In [1]:
import os, time, math
import math
from math import sqrt
from ann_visualizer.visualize import ann_viz
import numpy as np
import pandas as pd
import seaborn as sns
from keras.layers import Dense ,Dropout,BatchNormalization
from keras.models import Model
#from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
from keras.models import Sequential
from keras.utils.vis_utils import plot_model
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
%matplotlib inline
from matplotlib import pyplot as plt
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingClassifier
from sklearn import linear_model, tree, ensemble
import sklearn.metrics as metrics 
from matplotlib import pyplot
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
import statsmodels.api as sm
#from sklearn.metrics import mean_absolute_percentage_error
import xgboost as xgb
from xgboost import XGBRegressor
import math
from tune_sklearn import TuneGridSearchCV
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingClassifier
from sklearn import linear_model, tree, ensemble
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
reg_decision_model=DecisionTreeRegressor()
from sklearn.model_selection import StratifiedKFold
from hyperopt import tpe,hp,Trials
import xgboost as xgb
from hyperopt.fmin import fmin

In [2]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
# Helper Function
def rmse(predictions, targets):
    differences = predictions - targets                       #the DIFFERENCEs.
    differences_squared = differences ** 2                    #the SQUAREs of ^
    mean_of_differences_squared = differences_squared.mean()  #the MEAN of ^
    rmse_val = np.sqrt(mean_of_differences_squared)           #ROOT of ^
    return rmse_val

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f}.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    return accuracy

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def abline(slope, intercept):
    """Plot a line from slope and intercept"""
    axes = plt.gca()
    x_vals = np.array(axes.get_xlim())
    y_vals = intercept + slope * x_vals
    plt.plot(x_vals, y_vals, '--')

In [4]:
data=pd.read_csv("E:/Bike Fusion/New Plan/Circular Automatic Collected data for modeling/Buffer Paper/R_exported_buffer_paper_2019_Six_cities_data.csv")
data.head()

,Unnamed: 0,site_id,site_name,type,year,region,travel_pattern,AADBT,stv_adb,stv_c_adb,...,Bike.Share.best_tm,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi,rowIndex
0,1,P001BEN,Colorado Ave.,permanent,2019,Bend,2.0,46.209524,6.561644,2.616438,...,3056.0,1.248328,7.709046,10.644986,28.863905,35.305839,108.195110,0.512552,0.113341,1
1,2,P002BEN,Franklin Ave. Both Sides,permanent,2019,Bend,2.0,220.426625,6.958904,3.068493,...,3056.0,0.000000,0.000000,3.995612,49.828057,65.959899,230.465979,0.282730,0.426931,2
2,3,P003BEN,Galveston Ave. Both Sides,permanent,2019,Bend,3.0,127.953968,13.726027,4.369863,...,3056.0,1.191490,6.758242,27.855287,31.094049,41.330493,98.318063,0.499890,0.010863,3
3,4,P004BEN,Newport Ave. Both Sides,permanent,2019,Bend,1.0,109.378321,7.027397,2.178082,...,3056.0,0.722608,8.910716,26.495791,28.563976,54.080251,118.796144,0.344087,0.010368,4
4,5,P005BEN,Portland Ave. Both Sides,permanent,2019,Bend,2.0,61.295238,5.260274,1.698630,...,3056.0,5.137481,7.813417,12.608750,31.528644,55.880389,182.273542,0.559726,0.005038,5


In [5]:
track=data[['site_id','site_name','region', 'AADBT']]
track['index']=track.index
track.head()

C:\Users\mxm5116\AppData\Local\Temp/ipykernel_5408/2363949684.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  track['index']=track.index


,site_id,site_name,region,AADBT,index
0,P001BEN,Colorado Ave.,Bend,46.209524,0
1,P002BEN,Franklin Ave. Both Sides,Bend,220.426625,1
2,P003BEN,Galveston Ave. Both Sides,Bend,127.953968,2
3,P004BEN,Newport Ave. Both Sides,Bend,109.378321,3
4,P005BEN,Portland Ave. Both Sides,Bend,61.295238,4


In [6]:
# Clean the data
an_data=data._get_numeric_data()
nan_values = an_data. isna()
nan_columns = nan_values. any()
columns_with_nan = an_data. columns[nan_columns]. tolist()
print(columns_with_nan)

['travel_pattern', 'Bike.Share.Origin_otm', 'Bike.Share.Destination_otm', 'Bike.Share.Crossing_otm', 'Bike.Share.route_otm', 'Bike.Share.Origin_qm', 'Bike.Share.Destination_qm', 'Bike.Share.Crossing_qm', 'Bike.Share.route_qm', 'Bike.Share.Origin_hm', 'Bike.Share.Destination_hm', 'Bike.Share.Crossing_hm', 'Bike.Share.route_hm', 'Bike.Share.Origin_tfm', 'Bike.Share.Destination_tfm', 'Bike.Share.Crossing_tfm', 'Bike.Share.route_tfm', 'Bike.Share.Origin_om', 'Bike.Share.Destination_om', 'Bike.Share.Crossing_om', 'Bike.Share.route_om', 'Bike.Share.Origin_ohm', 'Bike.Share.Destination_ohm', 'Bike.Share.Crossing_ohm', 'Bike.Share.route_ohm', 'Bike.Share.Origin_tm', 'Bike.Share.Destination_tm', 'Bike.Share.Crossing_tm', 'Bike.Share.route_tm', 'Bike.Share.Origin_otm_net', 'Bike.Share.Destination_otm_net', 'Bike.Share.Crossing_otm_net', 'Bike.Share.route_otm_net', 'Bike.Share.Origin_qm_net', 'Bike.Share.Destination_qm_net', 'Bike.Share.Crossing_qm_net', 'Bike.Share.route_qm_net', 'Bike.Share.Ori

In [7]:
rf_data=an_data.drop(an_data. columns[nan_columns]. tolist(),axis=1)
rf_data=rf_data.drop(['Unnamed: 0','year','latitude','longitude','rowIndex'],axis=1)
rf_data.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,log_stl_raw,log_stv_stl,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,8.067776,10.090865,1.248328,7.709046,10.644986,28.863905,35.305839,108.195110,0.512552,0.113341
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,7.870166,9.944457,0.000000,0.000000,3.995612,49.828057,65.959899,230.465979,0.282730,0.426931
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,8.048469,10.738085,1.191490,6.758242,27.855287,31.094049,41.330493,98.318063,0.499890,0.010863
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,7.600402,9.683263,0.722608,8.910716,26.495791,28.563976,54.080251,118.796144,0.344087,0.010368
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,7.235619,9.069843,5.137481,7.813417,12.608750,31.528644,55.880389,182.273542,0.559726,0.005038


In [8]:
#corrMatrix = rf_data.corr()
#pd.DataFrame.to_csv(corrMatrix,'F:/Bike Fusion/New Plan/Circular Automatic Collected data for modeling/Buffer Paper/corrMatrix_AN_all.csv')

In [9]:
# rf_data1=rf_data[['AADBT','stv_c_adb','stl_raw','log_stv_stl','primary_binary','secondary_binary','tertiary_binary','residential_binary','path_binary',
# 'cycleway_binary','footway_binary','cycleway_lane_binary','Distance.to.Commercial.Area','Distance.to.Industrial.Area','Distance.to.Residential.Center',
# 'Distance.to.Retail.Center','Distance.to.Grass.Center','Distance.to.Park.Center','Distance.to.Water.Body','Distance.to.Forest.Center'
# ,'min_dist_to_school','Point.Speed','Bike.Commuter_tm','Intersection_Density_tm',
# 'Industrial.Area_qm_net','Residential_Road_otm_net','Grass.Area_hm_net','Park.Area_ohm','Water.Area_qm','Bicycle.Parking_tm_net','Bus.Stops_tm',
# 'School_tm','college_tm_net','University_tm','OSM_node_Density_tm','Primary_ohm','Secondary_tm','Tertiary_tm','Path_hm_net','Footway_tm',
# 'Student.Access_tm','pct_at_least_college_education_tm_net','Median_HH_income_otm','population_density_tm','employment_density_tm',
# 'Temp_avg_otm','Hum_avg_otm','Precipitation_otm','Number.of.jobs_tm_net','cycleway_lane_all_om','cycleway_track_all_tm','slope_ohm',
# 'pct_white_om','maxspeed_ohm_net','pct_male_ohm','sep_bikeway_otm_net','BikeFac_binary','BikeFac_ohm_net','BikeFac_onstreet_om',
# 'Distance_to_CBD_mi']]
#'Distance.to.CBD','Distance_to_Water_Body_mi'

# All buffer Sizes

In [10]:
rf_data1=rf_data.loc[:,~rf_data.columns.str.contains('_net', case=False)]
rf_data1.shape

(312, 398)

In [11]:
corrMatrix1 = rf_data1.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.35]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'Bus.Stops_qm',
       'Bus.Stops_hm', 'OSM_node_Density_hm', 'Primary_hm',
       'OSM_node_Density_tfm', 'Total.Commuter_om', 'Bike.Commuter_om',
       'bike.commuter.density_om', 'Intersection_Density_om',
       'Bicycle.Parking_om', 'Bus.Stops_om', 'college_om',
       'OSM_node_Density_om', 'Primary_om', 'HH_density_om',
       'population_density_om', 'employment_density_om', 'Total.Commuter_ohm',
       'Bike.Commuter_ohm', 'bike.commuter.density_ohm',
       'Intersection_Density_ohm', 'Bicycle.Parking_ohm', 'Bus.Stops_ohm',
       'college_ohm', 'OSM_node_Density_ohm', 'Primary_ohm', 'Tertiary_ohm',
       'cycleway_track_all_ohm', 'HH_density_ohm', 'population_density_ohm',
       'employment_density_ohm', 'Number.of.jobs_ohm', 'Total.Commuter_tm',
       'Bike.Commuter_tm', 'bike.commuter.density_tm',
       'Intersection_Density_tm', 'Bicycle.Parking_tm', 'Bus.Stops_tm',
       'School_tm', 'college_tm', 'OSM_node_Density

In [12]:
data_all=rf_data1[top_corr_features]
data_all1=data_all.drop(['OSM_node_Density_tm','stv_adb','OSM_node_Density_tfm','Primary_tm','Primary_ohm','cycleway_track_all_ohm','OSM_node_Density_hm','stv_nc_adb','stl_raw' ],axis=1)
data_all1.shape                     

(312, 50)

In [36]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = data_all1.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = data_all1[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 754 ms


,feature,VIF
3,Primary_hm,1.451356
10,log_stv_nc_adb,1.746248
11,log_stl_raw,1.749507
8,Student.Access_tm,1.959198
2,Bus.Stops_qm,1.964687
5,Bicycle.Parking_om,2.209164
6,Tertiary_ohm,2.569095
1,stv_c_adb,2.661211
7,School_tm,2.719679
4,bike.commuter.density_om,3.153324


In [37]:
vif_data['feature']

1                    stv_c_adb
2                 Bus.Stops_qm
3                   Primary_hm
4     bike.commuter.density_om
5           Bicycle.Parking_om
6                 Tertiary_ohm
7                    School_tm
8            Student.Access_tm
9            Number.of.jobs_tm
10              log_stv_nc_adb
11                 log_stl_raw
Name: feature, dtype: object

# Now lets select 0.1 mile buffer variabe

In [69]:
rf_data2=rf_data1.loc[:,~rf_data1.columns.str.contains('qm', case=False)] 
rf_data3=rf_data2.loc[:,~rf_data2.columns.str.contains('hm', case=False)] 
rf_data4=rf_data3.loc[:,~rf_data3.columns.str.contains('tfm', case=False)] 
rf_data5=rf_data4.loc[:,~rf_data4.columns.str.contains('om', case=False)] 
rf_data6=rf_data5.loc[:,~rf_data5.columns.str.contains('ohm', case=False)] 
rf_data7=rf_data6.loc[:,~rf_data6.columns.str.contains('tm', case=False)]
# keep the data for one-tenth mile buffer only
rf_data8=rf_data1.loc[:,rf_data1.columns.str.contains('otm', case=False)]
rf_data9=pd.concat([rf_data7, rf_data8.reindex(rf_data7.index)], axis=1)
rf_data9.columns.tolist()

['AADBT',
 'stv_adb',
 'stv_c_adb',
 'stl_raw',
 'primary_binary',
 'secondary_binary',
 'tertiary_binary',
 'residential_binary',
 'path_binary',
 'cycleway_binary',
 'footway_binary',
 'cycleway_lane_binary',
 'cycleway_track_all_binary',
 'lanes',
 'Distance.to.Industrial.Area',
 'Distance.to.Industrial.Center',
 'Distance.to.Residential.Area',
 'Distance.to.Residential.Center',
 'Distance.to.Retail.Area',
 'Distance.to.Retail.Center',
 'Distance.to.Grass',
 'Distance.to.Grass.Center',
 'Distance.to.Park',
 'Distance.to.Park.Center',
 'Distance.to.Water.Body',
 'Distance.to.Water.Center',
 'Distance.to.forest',
 'Distance.to.Forest.Center',
 'Distance.to.CBD',
 'min_dist_to_school',
 'min_dist_to_college',
 'min_dist_to_university',
 'Point.Speed',
 'Point.Bridge',
 'stv_nc_adb',
 'sep_bikeway_binary',
 'sep_onstreet_binary',
 'BikeFac_binary',
 'arterial_binary',
 'arterial_no_bike_lane_binary',
 'primary_no_bike_lane_binary',
 'secondary_no_bike_lane_binary',
 'tertiary_no_bike_la

In [70]:
corrMatrix1 = rf_data9.corr()
corrMatrix1

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,employment_density_otm,Number.of.jobs_otm,slope_otm,Temp_avg_otm,Hum_avg_otm,Precipitation_otm,sep_bikeway_otm,BikeFac_otm,BikeFac_onstreet_otm,Park_acres_otm
AADBT,1.000000,0.691349,0.734261,0.563353,-0.089504,-0.089987,-0.215239,0.060670,0.259477,0.089271,...,0.123861,0.088251,-0.031893,0.198497,0.128525,0.166397,0.166042,0.136017,0.067165,0.118860
stv_adb,0.691349,1.000000,0.763331,0.460712,-0.045819,-0.002297,-0.095177,0.024144,0.171482,-0.032325,...,-0.001883,-0.030000,-0.033344,0.280032,-0.077144,0.058352,0.030642,0.032979,-0.033526,0.236305
stv_c_adb,0.734261,0.763331,1.000000,0.614351,-0.072056,0.041851,-0.167485,0.175909,0.272769,-0.095544,...,0.124325,0.035289,-0.104572,0.090904,0.088822,0.009130,-0.006667,0.062001,0.014461,-0.048625
stl_raw,0.563353,0.460712,0.614351,1.000000,0.035996,0.163431,-0.186925,-0.128496,0.312011,-0.077424,...,0.195497,0.155987,-0.134061,0.021405,0.067893,-0.018278,0.080783,0.277861,0.248053,-0.034803
primary_binary,-0.089504,-0.045819,-0.072056,0.035996,1.000000,-0.084963,-0.085827,-0.097803,-0.032947,-0.084096,...,-0.091660,0.072900,0.031379,-0.074718,-0.128732,-0.124083,-0.038566,-0.002322,-0.024064,-0.055711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Precipitation_otm,0.166397,0.058352,0.009130,-0.018278,-0.124083,-0.149883,-0.159386,-0.071206,0.047262,0.323510,...,0.197096,0.077245,-0.147547,0.743313,0.617939,1.000000,0.143681,0.098656,0.160736,0.274578
sep_bikeway_otm,0.166042,0.030642,-0.006667,0.080783,-0.038566,-0.119157,-0.198023,-0.295350,0.228934,0.444811,...,-0.125386,0.034925,-0.012708,0.078215,0.058706,0.143681,1.000000,0.740767,0.377722,0.079599
BikeFac_otm,0.136017,0.032979,0.062001,0.277861,-0.002322,0.079319,-0.156169,-0.378630,0.248482,0.305606,...,0.007778,0.115276,-0.030597,-0.062562,0.147392,0.098656,0.740767,1.000000,0.827458,-0.046294
BikeFac_onstreet_otm,0.067165,-0.033526,0.014461,0.248053,-0.024064,0.161260,-0.134997,-0.350437,0.064445,0.366255,...,0.098983,0.142573,-0.037901,-0.080014,0.251823,0.160736,0.377722,0.827458,1.000000,-0.061590


In [71]:
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.2]
#plt.figure(figsize=(18,14))
#plt.title("2019_cor-relation>0.20 with AADBT")
#g = sns.heatmap(rf_data9[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [72]:
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.forest', 'Distance.to.Forest.Center',
       'Distance.to.CBD', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Percentage.of.Bike.Commuter_otm', 'Industrial.Area_otm',
       'Bus.Stops_otm', 'OSM_node_Density_otm', 'Footway_otm', 'bridge_otm',
       'pct_at_least_college_education_otm'],
      dtype='object')

In [81]:
data1=rf_data9[top_corr_features]
data11=data1.drop(['tertiary_no_bike_lane_binary','tertiary_bike_lane_binary','log_stv_nc_adb','stl_raw'],axis=1)

In [82]:
# rf_data99=rf_data[['AADBT','stv_c_adb', 'stl_raw', 'tertiary_binary',
#        'path_binary', 'Distance.to.forest', 'Distance.to.Forest.Center',
#         'min_dist_to_college', 'min_dist_to_university',
#        'stv_nc_adb', 
#        'tertiary_bike_lane_binary','log_stv_stl', 'Distance_to_CBD_mi',
#        'Percentage.of.Bike.Commuter_otm', 'Industrial.Area_otm',
#        'Bus.Stops_otm', 'OSM_node_Density_otm', 'Footway_otm', 'bridge_otm',
#        'pct_at_least_college_education_otm', 'Bike.Commuter_otm_net',
#        'Percentage.of.Bike.Commuter_otm_net', 'Industrial.Area_otm_net',
#        'Residential_Road_otm_net', 'Bus.Stops_otm_net', 'Path_otm_net',
#        'Footway_otm_net', 'sep_bikeway_otm_net']]

In [83]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = data11.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = data11[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

Wall time: 152 ms


C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,VIF
13,Footway_otm,1.159500
2,tertiary_binary,1.167131
11,Bus.Stops_otm,1.267171
12,OSM_node_Density_otm,1.309386
3,path_binary,1.344622
14,bridge_otm,1.362373
15,pct_at_least_college_education_otm,1.421599
10,Industrial.Area_otm,1.431497
6,stv_nc_adb,1.671984
9,Percentage.of.Bike.Commuter_otm,1.809610


In [84]:
vif_data['feature']

1                              stv_c_adb
2                        tertiary_binary
3                            path_binary
4                     Distance.to.forest
5                    min_dist_to_college
6                             stv_nc_adb
7                            log_stv_stl
8                     Distance_to_CBD_mi
9        Percentage.of.Bike.Commuter_otm
10                   Industrial.Area_otm
11                         Bus.Stops_otm
12                  OSM_node_Density_otm
13                           Footway_otm
14                            bridge_otm
15    pct_at_least_college_education_otm
Name: feature, dtype: object

# Now select 0.25 mile buffer

In [85]:
rf_data8=rf_data1.loc[:,rf_data1.columns.str.contains('qm', case=False)]
rf_data9=pd.concat([rf_data7, rf_data8.reindex(rf_data7.index)], axis=1)
rf_data9.columns.shape

(102,)

In [87]:
corrMatrix1 = rf_data9.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.2]
top_corr_features
#plt.figure(figsize=(18,14))
#plt.title("2019_cor-relation>0.20 with AADBT")
#g = sns.heatmap(rf_data9[top_corr_features].corr(),annot=True,cmap="RdYlGn")

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.forest', 'Distance.to.Forest.Center',
       'Distance.to.CBD', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Total.Commuter_qm', 'Bike.Commuter_qm',
       'Percentage.of.Bike.Commuter_qm', 'bike.commuter.density_qm',
       'Bicycle.Parking_qm', 'Bus.Stops_qm', 'OSM_node_Density_qm',
       'Primary_qm', 'Footway_qm', 'pct_at_least_college_education_qm',
       'Number.of.jobs_qm'],
      dtype='object')

In [91]:
data2=rf_data9[top_corr_features]
data22=data2.drop(['tertiary_no_bike_lane_binary','tertiary_bike_lane_binary','log_stv_nc_adb','stl_raw','log_stv_adb'],axis=1)

In [92]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = data22.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = data22[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

Wall time: 176 ms


C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,VIF
2,tertiary_binary,1.194793
15,Footway_qm,1.253506
3,path_binary,1.353681
16,pct_at_least_college_education_qm,1.450609
14,Primary_qm,1.456028
6,stv_nc_adb,1.476107
17,Number.of.jobs_qm,1.814356
11,Bicycle.Parking_qm,1.827695
13,OSM_node_Density_qm,1.967817
4,Distance.to.forest,2.047943


In [93]:
vif_data['feature']

1                             stv_c_adb
2                       tertiary_binary
3                           path_binary
4                    Distance.to.forest
5                   min_dist_to_college
6                            stv_nc_adb
7                           log_stl_raw
8                    Distance_to_CBD_mi
9                     Total.Commuter_qm
10       Percentage.of.Bike.Commuter_qm
11                   Bicycle.Parking_qm
12                         Bus.Stops_qm
13                  OSM_node_Density_qm
14                           Primary_qm
15                           Footway_qm
16    pct_at_least_college_education_qm
17                    Number.of.jobs_qm
Name: feature, dtype: object

# Run 0.5 mile buffer

In [94]:
rf_data8=rf_data1.loc[:,rf_data1.columns.str.contains('hm', case=False)]
rf_data9=pd.concat([rf_data7, rf_data8.reindex(rf_data7.index)], axis=1)
rf_data10=rf_data9.loc[:,~rf_data9.columns.str.contains('ohm', case=False)]
rf_data10.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,Number.of.jobs_hm,slope_hm,Temp_avg_hm,Hum_avg_hm,Precipitation_hm,sep_bikeway_hm,BikeFac_hm,BikeFac_onstreet_hm,school_college_hm,Park_acres_hm
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,4801.473846,1.990498,48.478082,61.352877,0.028959,16141.281360,5.241413,5.097525,3,10.644986
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,8320.262637,2.202706,48.478082,61.352877,0.028959,4091.843380,5.617990,5.429112,2,3.995612
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,2960.764681,2.346421,48.478082,61.352877,0.028959,10263.045300,3.231008,3.126321,3,27.855287
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,6237.743052,2.619904,48.478082,61.352877,0.028959,10591.062550,4.809335,4.766779,3,26.495791
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,6496.476827,0.000000,48.478082,61.352877,0.028959,7561.538615,5.373341,5.373341,0,12.608750


In [95]:
corrMatrix1 = rf_data10.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.2]
top_corr_features
#plt.figure(figsize=(18,14))
#plt.title("2019_cor-relation>0.20 with AADBT")
#g = sns.heatmap(rf_data9[top_corr_features].corr(),annot=True,cmap="RdYlGn")

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.forest', 'Distance.to.Forest.Center',
       'Distance.to.CBD', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Total.Commuter_hm', 'Bike.Commuter_hm',
       'Percentage.of.Bike.Commuter_hm', 'bike.commuter.density_hm',
       'Intersection_Density_hm', 'Bicycle.Parking_hm', 'Bus.Stops_hm',
       'college_hm', 'OSM_node_Density_hm', 'Primary_hm', 'Tertiary_hm',
       'Footway_hm', 'cycleway_track_all_hm', 'Student.Access_hm',
       'pct_at_least_college_education_hm', 'HH_density_hm',
       'population_density_hm', 'employment_density_hm', 'Number.of.jobs_hm'],
      dtype='object')

In [99]:
data3=rf_data10[top_corr_features]
data33=data3.drop(['stv_adb','log_stv_adb', 'log_stv_c_adb','log_stv_nc_adb', 'log_stl_raw','bike.commuter.density_hm','tertiary_no_bike_lane_binary','stl_raw','tertiary_bike_lane_binary','cycleway_track_all_hm'],axis=1)

In [100]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = data33.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = data33[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 231 ms


,feature,VIF
2,tertiary_binary,1.248571
3,path_binary,1.496964
15,Tertiary_hm,1.630240
18,pct_at_least_college_education_hm,1.709710
6,stv_nc_adb,1.848764
14,Primary_hm,1.863596
4,Distance.to.forest,2.124076
13,college_hm,2.483438
19,Number.of.jobs_hm,2.666148
17,Student.Access_hm,2.670618


In [101]:
vif_data['feature']

1                             stv_c_adb
2                       tertiary_binary
3                           path_binary
4                    Distance.to.forest
5                   min_dist_to_college
6                            stv_nc_adb
7                           log_stv_stl
8                    Distance_to_CBD_mi
9        Percentage.of.Bike.Commuter_hm
10              Intersection_Density_hm
11                   Bicycle.Parking_hm
12                         Bus.Stops_hm
13                           college_hm
14                           Primary_hm
15                          Tertiary_hm
16                           Footway_hm
17                    Student.Access_hm
18    pct_at_least_college_education_hm
19                    Number.of.jobs_hm
Name: feature, dtype: object

# 0.75 mile buffer variable selections

In [112]:
rf_data8=rf_data1.loc[:,rf_data1.columns.str.contains('tfm', case=False)]
rf_data9=pd.concat([rf_data7, rf_data8.reindex(rf_data7.index)], axis=1)
rf_data10=rf_data9.loc[:,~rf_data9.columns.str.contains('ohm', case=False)]
rf_data10.shape

(312, 102)

In [113]:
corrMatrix1 = rf_data10.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.2]
top_corr_features
#plt.figure(figsize=(18,14))
#plt.title("2019_cor-relation>0.20 with AADBT")
#g = sns.heatmap(rf_data9[top_corr_features].corr(),annot=True,cmap="RdYlGn")

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.forest', 'Distance.to.Forest.Center',
       'Distance.to.CBD', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Percentage.of.Bike.Commuter_tfm', 'bike.commuter.density_tfm',
       'Intersection_Density_tfm', 'OSM_node_Density_tfm',
       'pct_at_least_college_education_tfm', 'HH_density_tfm',
       'population_density_tfm', 'employment_density_tfm'],
      dtype='object')

In [125]:
data4=rf_data10[top_corr_features]
data44=data4.drop(['stv_adb','log_stv_adb', 'log_stv_c_adb','log_stv_nc_adb','stl_raw'],axis=1)

In [126]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = data44.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = data44[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 205 ms


,feature,VIF
2,tertiary_binary,1.147450
3,path_binary,1.301734
6,stv_nc_adb,1.566278
12,pct_at_least_college_education_tfm,1.571635
9,Percentage.of.Bike.Commuter_tfm,2.159853
13,HH_density_tfm,2.171572
4,Distance.to.forest,2.176007
1,stv_c_adb,2.224153
7,log_stl_raw,2.567347
11,OSM_node_Density_tfm,2.700090


In [127]:
vif_data['feature']

1                              stv_c_adb
2                        tertiary_binary
3                            path_binary
4                     Distance.to.forest
5                    min_dist_to_college
6                             stv_nc_adb
7                            log_stl_raw
8                     Distance_to_CBD_mi
9        Percentage.of.Bike.Commuter_tfm
10              Intersection_Density_tfm
11                  OSM_node_Density_tfm
12    pct_at_least_college_education_tfm
13                        HH_density_tfm
Name: feature, dtype: object

# Run 1.0 mile buffer Model

In [128]:
rf_data8=rf_data1.loc[:,rf_data1.columns.str.contains('_om', case=False)]
rf_data9=rf_data.loc[:,rf_data.columns.str.contains('_om_net', case=False)]
rf_data10=pd.concat([rf_data8, rf_data9.reindex(rf_data8.index)], axis=1)
rf_data12=pd.concat([rf_data7, rf_data8.reindex(rf_data7.index)], axis=1)
rf_data12.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,employment_density_om,Number.of.jobs_om,slope_om,Temp_avg_om,Hum_avg_om,Precipitation_om,sep_bikeway_om,BikeFac_om,BikeFac_onstreet_om,Park_acres_om
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,1870.153213,17496.87795,2.284414,48.478082,61.352877,0.028959,47477.56680,24.506152,23.845532,35.305839
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,2094.218655,17551.02958,2.433644,48.478082,61.352877,0.028959,36209.91970,19.693226,19.245858,65.959899
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,1843.183535,14569.45897,2.638941,48.478082,61.352877,0.028959,51530.47621,21.213084,17.486429,41.330493
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,2078.585330,14692.40912,2.682589,48.478082,61.352877,0.028959,32914.57296,20.598480,18.836272,54.080251
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,2135.456675,13994.75076,0.000000,48.478082,61.352877,0.028959,29151.19331,21.391827,19.822288,55.880389


In [129]:
corrMatrix1 = rf_data12.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.25]
#plt.figure(figsize=(18,14))
#plt.title("2019_cor-relation>0.20 with AADBT")
#g = sns.heatmap(rf_data12[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [138]:
data5=rf_data12[top_corr_features]
data55=data5.drop(['stv_adb','log_stv_adb', 'log_stv_c_adb','log_stv_nc_adb', 'stl_raw','log_stl_raw'],axis=1)

In [139]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = data55.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = data55[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

Wall time: 183 ms


C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,VIF
15,pct_at_least_college_education_om,1.529469
2,path_binary,1.548948
4,stv_nc_adb,1.908984
3,min_dist_to_college,1.924460
11,Tertiary_om,2.203690
14,Student.Access_om,2.613685
12,Footway_om,2.653557
10,Primary_om,2.890032
13,cycleway_track_all_om,2.937191
1,stv_c_adb,3.019905


In [140]:
vif_data['feature']

1                             stv_c_adb
2                           path_binary
3                   min_dist_to_college
4                            stv_nc_adb
5                           log_stv_stl
6        Percentage.of.Bike.Commuter_om
7               Intersection_Density_om
8                          Bus.Stops_om
9                            college_om
10                           Primary_om
11                          Tertiary_om
12                           Footway_om
13                cycleway_track_all_om
14                    Student.Access_om
15    pct_at_least_college_education_om
Name: feature, dtype: object

# Run 1.5 mile Buffer 

In [141]:
rf_data1=rf_data1.loc[:,~rf_data1.columns.str.contains('_otm', case=False)]
rf_data2=rf_data1.loc[:,~rf_data1.columns.str.contains('_qm', case=False)]
rf_data3=rf_data2.loc[:,~rf_data2.columns.str.contains('_hm', case=False)]
rf_data4=rf_data3.loc[:,~rf_data3.columns.str.contains('_tfm', case=False)]
rf_data5=rf_data4.loc[:,~rf_data4.columns.str.contains('_om', case=False)]
rf_data6=rf_data5.loc[:,~rf_data5.columns.str.contains('_tm', case=False)]
rf_data6.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,BikeFac_ohm,BikeFac_onstreet_ohm,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,52.810656,48.230205,2.023089,1.285490,1.598419,8.067776,10.090865,108.195110,0.512552,0.113341
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,50.171800,45.062402,2.074291,1.403273,1.587276,7.870166,9.944457,230.465979,0.282730,0.426931
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,47.404731,42.672877,2.689617,1.680802,2.337582,8.048469,10.738085,98.318063,0.499890,0.010863
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,47.405753,42.655687,2.082860,1.156278,1.766325,7.600402,9.683263,118.796144,0.344087,0.010368
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,45.532479,41.272569,1.834224,0.992744,1.517683,7.235619,9.069843,182.273542,0.559726,0.005038


In [142]:
corrMatrix1 = rf_data6.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
#plt.figure(figsize=(18,14))
#plt.title("2019_cor-relation>0.20 with AADBT")
#g = sns.heatmap(rf_data6[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [146]:
data6=rf_data6[top_corr_features]
data66=data6.drop(['stv_adb','log_stv_adb', 'log_stv_c_adb','log_stv_nc_adb', 'log_stl_raw','stl_raw'],axis=1)

In [147]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = data66.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = data66[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 502 ms


,feature,VIF
2,tertiary_binary,1.205357
3,path_binary,1.470815
14,pct_at_least_college_education_ohm,1.896666
7,Park.Area_ohm,2.149957
12,cycleway_track_all_ohm,2.286971
4,Distance.to.forest,2.308219
11,Footway_ohm,2.538974
13,Student.Access_ohm,2.647587
9,Primary_ohm,2.747711
10,Tertiary_ohm,2.761478


In [148]:
vif_data['feature']

1                              stv_c_adb
2                        tertiary_binary
3                            path_binary
4                     Distance.to.forest
5                    min_dist_to_college
6        Percentage.of.Bike.Commuter_ohm
7                          Park.Area_ohm
8                             School_ohm
9                            Primary_ohm
10                          Tertiary_ohm
11                           Footway_ohm
12                cycleway_track_all_ohm
13                    Student.Access_ohm
14    pct_at_least_college_education_ohm
15                            stv_nc_adb
16                           log_stv_stl
17                    Distance_to_CBD_mi
Name: feature, dtype: object

# Run 2.0 mile buffer

In [1254]:
# Run 2 mile Buffer RF Model

In [149]:
rf_data1=rf_data1.loc[:,~rf_data1.columns.str.contains('_otm', case=False)]
rf_data2=rf_data1.loc[:,~rf_data1.columns.str.contains('_qm', case=False)]
rf_data3=rf_data2.loc[:,~rf_data2.columns.str.contains('_hm', case=False)]
rf_data4=rf_data3.loc[:,~rf_data3.columns.str.contains('_tfm', case=False)]
rf_data5=rf_data4.loc[:,~rf_data4.columns.str.contains('_om', case=False)]
rf_data6=rf_data5.loc[:,~rf_data5.columns.str.contains('_ohm', case=False)]
rf_data6.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,tertiary_bike_lane_binary,BikeFac_tm,BikeFac_onstreet_tm,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Distance_to_CBD_mi,Distance_to_Water_Body_mi
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,0,85.007762,76.808645,2.023089,1.285490,1.598419,8.067776,10.090865,0.512552,0.113341
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,0,85.849684,76.675704,2.074291,1.403273,1.587276,7.870166,9.944457,0.282730,0.426931
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,0,84.479516,75.256963,2.689617,1.680802,2.337582,8.048469,10.738085,0.499890,0.010863
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,0,82.976764,73.221971,2.082860,1.156278,1.766325,7.600402,9.683263,0.344087,0.010368
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,1,81.722498,72.903232,1.834224,0.992744,1.517683,7.235619,9.069843,0.559726,0.005038


In [154]:
corrMatrix1 = rf_data6.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
#plt.figure(figsize=(18,14))
#plt.title("2019_cor-relation>0.20 with AADBT")
#g = sns.heatmap(rf_data6[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [155]:
data7=rf_data6[top_corr_features]
data77=data7.drop(['stv_adb','log_stv_adb', 'log_stv_c_adb','log_stv_nc_adb', 'log_stl_raw','stl_raw'],axis=1)

In [156]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = data77.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = data77[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 526 ms


,feature,VIF
16,tertiary_bike_lane_binary,1.243990
2,path_binary,1.371948
6,Residential_Road_tm,1.921707
9,University_tm,2.035537
7,Park.Area_tm,2.312072
14,pct_at_least_college_education_tm,2.333399
3,Distance.to.forest,2.450238
12,cycleway_track_all_tm,2.779412
15,stv_nc_adb,2.820953
11,Footway_tm,2.825765


In [158]:
vif_data['feature'][1:]

2                           path_binary
3                    Distance.to.forest
4                   min_dist_to_college
5        Percentage.of.Bike.Commuter_tm
6                   Residential_Road_tm
7                          Park.Area_tm
8                             School_tm
9                         University_tm
10                           Primary_tm
11                           Footway_tm
12                cycleway_track_all_tm
13                    Student.Access_tm
14    pct_at_least_college_education_tm
15                           stv_nc_adb
16            tertiary_bike_lane_binary
17                          log_stv_stl
18                   Distance_to_CBD_mi
Name: feature, dtype: object

# Portland Generalized Modal data

In [22]:
P_data=data[data['region']=='Portland']
P_data.head()

,Unnamed: 0,site_id,site_name,type,year,region,travel_pattern,AADBT,stv_adb,stv_c_adb,...,Bike.Share.best_tm,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi,rowIndex
224,225,P001POR,HAWTHORNE BR north side,permanent,2019,Portland,1.0,1775.363690,109.219178,80.945205,...,303222.0,2.532664,6.985360,24.117618,6.985360,66.650573,113.195792,0.315109,0.049033,225
225,226,P002POR,HAWTHORNE BR south side,permanent,2019,Portland,2.0,1537.211111,111.027397,74.904110,...,304246.0,0.000000,5.366635,23.795831,5.366635,70.113770,113.414613,0.403667,0.111882,226
226,227,P003POR,"SW MOODY AVE, WEST SIDE CYCLE TRACK",permanent,2019,Portland,1.0,1346.261310,98.109589,59.917808,...,259355.0,0.000000,0.027482,13.117664,0.027482,49.214257,81.480702,1.005728,0.848626,227
227,228,P004POR,STEEL BR lower deck,permanent,2019,Portland,1.0,1951.052146,128.041096,84.410959,...,310223.0,0.187564,3.716239,11.537631,3.716239,57.695648,111.057680,1.028972,0.015873,228
228,229,P005POR,BROADWAY BR north side,permanent,2019,Portland,1.0,1182.406584,120.712329,82.164384,...,300850.0,0.000000,0.000000,7.536517,0.000000,37.898346,107.614898,1.277080,0.154749,229


In [23]:
#P_data=P_data.loc[:,~P_data.columns.str.contains('_net', case=False)]
#P_data.shape

In [24]:
# Clean the data
an_data=P_data._get_numeric_data()
nan_values = an_data. isna()
nan_columns = nan_values. any()
columns_with_nan = an_data. columns[nan_columns]. tolist()
print(columns_with_nan)

['Bike.Share.route_otm', 'Bike.Share.route_qm', 'Bike.Share.route_hm', 'Bike.Share.route_tfm', 'Bike.Share.route_om', 'Bike.Share.route_ohm', 'Bike.Share.route_tm', 'Bike.Share.route_otm_net', 'Bike.Share.route_qm_net', 'Bike.Share.route_hm_net', 'Bike.Share.route_tfm_net', 'Bike.Share.route_om_net', 'Bike.Share.route_ohm_net', 'Bike.Share.route_tm_net']


In [26]:
p1_data=an_data.drop(an_data. columns[nan_columns]. tolist(),axis=1)
p1_data=p1_data.drop(['Unnamed: 0','year','latitude','longitude','rowIndex','Bike.Share.Origin_otm',
 'Bike.Share.Destination_otm',
 'Bike.Share.Crossing_otm','Bike.Share.Origin_qm',
 'Bike.Share.Destination_qm',
 'Bike.Share.Crossing_qm','Bike.Share.Origin_tfm',
 'Bike.Share.Destination_tfm',
 'Bike.Share.Crossing_tfm','Bike.Share.Origin_om',
 'Bike.Share.Destination_om',
 'Bike.Share.Crossing_om','Bike.Share.Origin_ohm',
 'Bike.Share.Destination_ohm',
 'Bike.Share.Crossing_ohm','Bike.Share.Origin_tm',
 'Bike.Share.Destination_tm',
 'Bike.Share.Crossing_tm','Bike.Share.Origin_qm_net',
 'Bike.Share.Destination_qm_net',
 'Bike.Share.Crossing_qm_net','Bike.Share.Origin_otm_net',
 'Bike.Share.Destination_otm_net',
 'Bike.Share.Crossing_otm_net','Bike.Share.Origin_hm_net',
 'Bike.Share.Destination_hm_net',
 'Bike.Share.Crossing_hm_net','Bike.Share.Origin_tfm_net',
 'Bike.Share.Destination_tfm_net',
 'Bike.Share.Crossing_tfm_net','Bike.Share.Origin_om_net',
 'Bike.Share.Destination_om_net',
 'Bike.Share.Crossing_om_net','Bike.Share.Origin_ohm_net',
 'Bike.Share.Destination_ohm_net',
 'Bike.Share.Crossing_ohm_net', 'Bike.Share.Origin_tm_net',
 'Bike.Share.Destination_tm_net',
 'Bike.Share.Crossing_tm_net','Bike.Share.best_otm',
 'Bike.Share.best_qm',
 'Bike.Share.best_hm',
 'Bike.Share.best_tfm',
 'Bike.Share.best_om',
 'Bike.Share.best_ohm',
 'Bike.Share.best_tm','Bike.Share.Origin_hm',
 'Bike.Share.Destination_hm',
 'Bike.Share.Crossing_hm'],axis=1)

p1_data.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,log_stl_raw,log_stv_stl,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
224,1.0,1775.363690,109.219178,80.945205,9915,0,0,0,0,1,...,9.201905,13.904376,2.532664,6.985360,24.117618,6.985360,66.650573,113.195792,0.315109,0.049033
225,2.0,1537.211111,111.027397,74.904110,6663,0,0,0,0,1,...,8.804475,13.523219,0.000000,5.366635,23.795831,5.366635,70.113770,113.414613,0.403667,0.111882
226,1.0,1346.261310,98.109589,59.917808,5466,0,0,0,0,0,...,8.606485,13.202711,0.000000,0.027482,13.117664,0.027482,49.214257,81.480702,1.005728,0.848626
227,1.0,1951.052146,128.041096,84.410959,17323,0,0,0,0,1,...,9.759848,14.619979,0.187564,3.716239,11.537631,3.716239,57.695648,111.057680,1.028972,0.015873
228,1.0,1182.406584,120.712329,82.164384,8077,0,0,0,0,1,...,8.996900,13.798560,0.000000,0.000000,7.536517,0.000000,37.898346,107.614898,1.277080,0.154749


In [85]:
p1_data=p1_data.loc[:,~p1_data.columns.str.contains('_net', case=False)] 
p1_data.shape

(88, 399)

# All Buffer

In [92]:
corrMatrix1 = p1_data.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
corrMatrix1.head(20)

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,log_stl_raw,log_stv_stl,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
travel_pattern,1.000000,-0.364762,-0.417670,-0.450843,-0.219514,NaN,0.039636,0.375572,-0.276314,0.015162,...,-0.250120,-0.427648,0.153206,0.150052,0.210850,0.150052,0.301859,0.293935,0.374064,-0.083484
AADBT,-0.364762,1.000000,0.794337,0.820675,0.577469,NaN,-0.039008,-0.254606,-0.059604,0.449860,...,0.580469,0.698989,-0.095360,0.001432,0.032192,0.001432,-0.150716,-0.136555,-0.451832,-0.034419
stv_adb,-0.417670,0.794337,1.000000,0.981994,0.604640,NaN,0.197546,-0.278858,-0.145738,0.354220,...,0.673644,0.868909,-0.044218,0.022592,0.041264,0.022592,-0.139710,-0.143198,-0.412943,0.051669
stv_c_adb,-0.450843,0.820675,0.981994,1.000000,0.596049,NaN,0.154720,-0.279389,-0.106741,0.368529,...,0.658785,0.842409,-0.060486,-0.026942,-0.001874,-0.026942,-0.178932,-0.181899,-0.427701,0.090610
stl_raw,-0.219514,0.577469,0.604640,0.596049,1.000000,NaN,0.238255,-0.195422,-0.499147,0.557755,...,0.809536,0.674864,-0.040800,-0.007171,0.027086,-0.007171,-0.076826,-0.162913,-0.458792,-0.217443
primary_binary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
secondary_binary,0.039636,-0.039008,0.197546,0.154720,0.238255,NaN,1.000000,-0.178174,-0.516398,-0.127515,...,0.366530,0.213125,-0.044145,0.036015,-0.008378,0.036015,-0.041904,-0.049958,-0.047932,0.198664
tertiary_binary,0.375572,-0.254606,-0.278858,-0.279389,-0.195422,NaN,-0.178174,1.000000,-0.414039,-0.102240,...,-0.135509,-0.227328,0.018053,-0.027667,0.094043,-0.027667,0.098765,0.047445,-0.105236,-0.186290
residential_binary,-0.276314,-0.059604,-0.145738,-0.106741,-0.499147,NaN,-0.516398,-0.414039,1.000000,-0.296319,...,-0.475817,-0.216833,-0.158605,-0.176499,-0.249893,-0.176499,-0.190567,-0.084369,0.143675,0.250083
path_binary,0.015162,0.449860,0.354220,0.368529,0.557755,NaN,-0.127515,-0.102240,-0.296319,1.000000,...,0.291264,0.251765,-0.021509,0.040609,0.106741,0.040609,0.160987,0.118191,0.032980,-0.337541


In [100]:
pdata0=p1_data[top_corr_features]
data00=pdata0.drop(['travel_pattern','cycleway_track_all_tm','tertiary_bike_lane_binary','tertiary_no_bike_lane_binary'],axis=1)

In [101]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = data00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = data00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\regression\linear_model.py:1738: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 38.9 s


,feature,VIF
9,University_tm,1.469382
7,Retail.Area_ohm,1.934355
3,Student.Access_om,2.592439
8,pct_African_American_ohm,2.689535
1,stv_c_adb,2.714587
14,sep_bikeway_tm,2.725720
6,Industrial.Area_ohm,2.735075
5,slope_om,3.050738
4,Median_HH_income_om,3.095854
13,sep_bikeway_tfm,3.164103


In [102]:
vif_data['feature']

1                    stv_c_adb
2                pct_female_om
3            Student.Access_om
4          Median_HH_income_om
5                     slope_om
6          Industrial.Area_ohm
7              Retail.Area_ohm
8     pct_African_American_ohm
9                University_tm
10                  Primary_tm
11                Secondary_tm
12              BikeFac_binary
13             sep_bikeway_tfm
14              sep_bikeway_tm
15                 BikeFac_otm
16                 log_stl_raw
Name: feature, dtype: object

# Portland 0.1 mile buffer

In [103]:
p1_data2=p1_data.loc[:,~p1_data.columns.str.contains('qm', case=False)] 
p1_data3=p1_data2.loc[:,~p1_data2.columns.str.contains('hm', case=False)] 
p1_data4=p1_data3.loc[:,~p1_data3.columns.str.contains('tfm', case=False)] 
p1_data5=p1_data4.loc[:,~p1_data4.columns.str.contains('om', case=False)] 
p1_data6=p1_data5.loc[:,~p1_data5.columns.str.contains('ohm', case=False)] 
p1_data7=p1_data6.loc[:,~p1_data6.columns.str.contains('tm', case=False)]
# keep the data for one-tenth mile buffer only
p1_data8=p1_data.loc[:,p1_data.columns.str.contains('otm', case=False)]
p1_data9=pd.concat([p1_data7, p1_data8.reindex(p1_data7.index)], axis=1)
p1_data9.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,employment_density_otm,Number.of.jobs_otm,slope_otm,Temp_avg_otm,Hum_avg_otm,Precipitation_otm,sep_bikeway_otm,BikeFac_otm,BikeFac_onstreet_otm,Park_acres_otm
224,1.0,1775.363690,109.219178,80.945205,9915,0,0,0,0,1,...,3838.854258,17.290086,0.000000,54.164932,69.331507,0.070274,4829.982437,1.131649,0.251502,2.532664
225,2.0,1537.211111,111.027397,74.904110,6663,0,0,0,0,1,...,3420.161346,0.000000,0.000000,54.164932,69.331507,0.070274,2101.268030,0.397967,0.000000,0.000000
226,1.0,1346.261310,98.109589,59.917808,5466,0,0,0,0,0,...,2638.546685,62.124073,2.047431,54.164932,69.331507,0.070274,1745.868816,0.330657,0.319540,0.000000
227,1.0,1951.052146,128.041096,84.410959,17323,0,0,0,0,1,...,4929.945224,0.000000,0.000000,54.164932,69.331507,0.070274,1693.885194,0.320812,0.000000,0.187564
228,1.0,1182.406584,120.712329,82.164384,8077,0,0,0,0,1,...,1387.509738,107.758651,0.000000,54.164932,69.331507,0.070274,1894.405743,1.516449,1.283489,0.000000


In [105]:
corrMatrix1 = p1_data9.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['travel_pattern', 'AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw',
       'tertiary_binary', 'path_binary', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Retail.Area',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.CBD',
       'min_dist_to_college', 'Point.Speed', 'Point.Bridge', 'stv_nc_adb',
       'sep_bikeway_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Total.Commuter_otm', 'Industrial.Area_otm', 'Residential_Road_otm',
       'Water.Area_otm', 'OSM_node_Density_otm', 'Primary_otm',
       'Secondary_otm', 'Path_otm', 'Cycleway_otm', 'maxspeed_otm',
       'bridge_otm', 'pct_male_otm', 'pct_female_otm', 'Median_HH_income_otm',
       'HH_density_otm', 'population_density_otm', 'employ

In [112]:
pdata0=p1_data9[top_corr_features]
pdata00=pdata0.drop(['stv_adb','travel_pattern','tertiary_no_bike_lane_binary','tertiary_bike_lane_binary','OSM_node_Density_otm','Distance.to.Industrial.Center','Distance.to.Grass.Center','Distance.to.Grass','log_stv_nc_adb','stv_nc_adb'],axis=1)

In [113]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = pdata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = pdata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\regression\linear_model.py:1738: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 440 ms


,feature,VIF
13,Water.Area_otm,1.541604
19,pct_female_otm,1.599219
20,Median_HH_income_otm,1.606563
5,Distance.to.Retail.Area,1.849049
1,tertiary_binary,1.893487
21,HH_density_otm,2.026332
4,Distance.to.Residential.Area,2.094764
6,min_dist_to_college,2.251852
17,maxspeed_otm,2.269136
3,Distance.to.Industrial.Area,2.599807


In [114]:
vif_data['feature']

1                  tertiary_binary
2                      path_binary
3      Distance.to.Industrial.Area
4     Distance.to.Residential.Area
5          Distance.to.Retail.Area
6              min_dist_to_college
7                   BikeFac_binary
8                    log_stv_c_adb
9                      log_stl_raw
10              Distance_to_CBD_mi
11             Industrial.Area_otm
12            Residential_Road_otm
13                  Water.Area_otm
14                     Primary_otm
15                   Secondary_otm
16                        Path_otm
17                    maxspeed_otm
18                      bridge_otm
19                  pct_female_otm
20            Median_HH_income_otm
21                  HH_density_otm
Name: feature, dtype: object

# Portland 0.25 mile buffer

In [115]:
p1_data8=p1_data.loc[:,p1_data.columns.str.contains('qm', case=False)]
p1_data9=pd.concat([p1_data7, p1_data8.reindex(p1_data7.index)], axis=1)
p1_data9.columns.shape

(103,)

In [116]:
corrMatrix1 = p1_data9.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['travel_pattern', 'AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw',
       'tertiary_binary', 'path_binary', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Retail.Area',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.CBD',
       'min_dist_to_college', 'Point.Speed', 'Point.Bridge', 'stv_nc_adb',
       'sep_bikeway_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Total.Commuter_qm', 'Industrial.Area_qm', 'Residential_Road_qm',
       'OSM_node_Density_qm', 'Primary_qm', 'Secondary_qm', 'Path_qm',
       'Cycleway_qm', 'maxspeed_qm', 'bridge_qm', 'population_density_qm',
       'employment_density_qm', 'Number.of.jobs_qm', 'sep_bikeway_qm',
       'BikeFac_qm'],
      dtype='object')

In [135]:
pdata1=p1_data9[top_corr_features]
pdata11=pdata1.drop(['travel_pattern','tertiary_no_bike_lane_binary','stv_nc_adb','OSM_node_Density_qm','Distance.to.Industrial.Center','Distance.to.Industrial.Center','Distance.to.Grass','Distance.to.Grass.Center','Distance.to.CBD','tertiary_bike_lane_binary','log_stv_nc_adb'],axis=1)
#pdata11=pdata1.drop(['travel_pattern','Bike.Share.best_otm','Bike.Share.Crossing_otm','Bike.Share.Origin_otm_net','Bike.Share.Destination_otm_net','Bike.Share.Crossing_otm_net','Distance.to.CBD','log_stv_adb' ,'log_stv_c_adb','log_stv_nc_adb','log_stl_raw','stl_raw','tertiary_no_bike_lane_binary','Path_otm_net','stv_adb','tertiary_bike_lane_binary','population_density_otm_net','population_density_otm','Distance.to.Grass.Center','sep_bikeway_otm','sep_bikeway_otm_net','sep_bikeway_binary','Primary_otm_net','Industrial.Area_otm_net','BikeFac_otm_net','Residential_Road_otm','log_stv_stl','bridge_otm','BikeFac_binary','employment_density_otm_net','Point.Bridge','stv_nc_adb'],axis=1)
pdata11.shape

(88, 34)

In [136]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = pdata11.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = pdata11[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 329 ms


,feature,VIF
16,maxspeed_qm,1.611299
4,Distance.to.Residential.Center,1.646190
18,employment_density_qm,1.838103
5,Distance.to.Retail.Area,1.897145
6,min_dist_to_college,1.925027
1,tertiary_binary,2.042645
11,Industrial.Area_qm,2.317699
12,Residential_Road_qm,2.656232
14,Secondary_qm,2.661523
8,BikeFac_binary,2.910472


In [137]:
vif_data['feature']

1                    tertiary_binary
2                        path_binary
3        Distance.to.Industrial.Area
4     Distance.to.Residential.Center
5            Distance.to.Retail.Area
6                min_dist_to_college
7                       Point.Bridge
8                     BikeFac_binary
9                      log_stv_c_adb
10                       log_stl_raw
11                Industrial.Area_qm
12               Residential_Road_qm
13                        Primary_qm
14                      Secondary_qm
15                           Path_qm
16                       maxspeed_qm
17                         bridge_qm
18             employment_density_qm
19                 Number.of.jobs_qm
Name: feature, dtype: object

# Portland 0.5 mile Buffer

In [138]:
p1_data8=p1_data.loc[:,p1_data.columns.str.contains('hm', case=False)]
p1_data9=pd.concat([p1_data7, p1_data8.reindex(p1_data7.index)], axis=1)
p1_data10=p1_data9.loc[:,~p1_data9.columns.str.contains('ohm', case=False)]
p1_data10.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,Number.of.jobs_hm,slope_hm,Temp_avg_hm,Hum_avg_hm,Precipitation_hm,sep_bikeway_hm,BikeFac_hm,BikeFac_onstreet_hm,school_college_hm,Park_acres_hm
224,1.0,1775.363690,109.219178,80.945205,9915,0,0,0,0,1,...,52845.177140,0.000000,54.164932,69.331507,0.070274,27328.10383,11.180662,6.315295,3,24.117618
225,2.0,1537.211111,111.027397,74.904110,6663,0,0,0,0,1,...,44662.672990,0.000000,54.164932,69.331507,0.070274,27814.88421,10.685974,5.818339,2,23.795831
226,1.0,1346.261310,98.109589,59.917808,5466,0,0,0,0,0,...,4730.761015,3.279166,54.164932,69.331507,0.070274,16861.87533,6.952576,5.829893,5,13.117664
227,1.0,1951.052146,128.041096,84.410959,17323,0,0,0,0,1,...,19755.915900,0.000000,54.164932,69.331507,0.070274,18778.04721,12.809169,9.675731,2,11.537631
228,1.0,1182.406584,120.712329,82.164384,8077,0,0,0,0,1,...,10427.793490,0.000000,54.164932,69.331507,0.070274,14167.53855,11.781868,9.715788,2,7.536517


In [139]:
corrMatrix1 = p1_data10.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.25]
top_corr_features

Index(['travel_pattern', 'AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw',
       'tertiary_binary', 'path_binary', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.CBD',
       'Point.Bridge', 'stv_nc_adb', 'sep_bikeway_binary', 'BikeFac_binary',
       'tertiary_no_bike_lane_binary', 'tertiary_bike_lane_binary',
       'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw',
       'log_stv_stl', 'Distance_to_CBD_mi', 'Commercial.Area_hm',
       'Retail.Area_hm', 'Bus.Stops_hm', 'OSM_node_Density_hm', 'Primary_hm',
       'Tertiary_hm', 'Path_hm', 'Cycleway_hm', 'pct_male_hm', 'pct_female_hm',
       'pct_at_least_college_education_hm', 'Number.of.jobs_hm',
       'sep_bikeway_hm', 'BikeFac_hm', 'BikeFac_onstreet_hm'],
      dtype='object')

In [142]:
pdata1=p1_data10[top_corr_features]
pdata11=pdata1.drop(['travel_pattern','stl_raw','tertiary_no_bike_lane_binary','log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw','stv_adb','Distance.to.CBD','tertiary_bike_lane_binary','sep_bikeway_hm','BikeFac_onstreet_hm','pct_female_hm','Distance.to.Grass','Point.Bridge','sep_bikeway_binary','stv_nc_adb','OSM_node_Density_hm','Distance.to.Grass.Center','log_stv_stl','BikeFac_hm','Primary_hm','Number.of.jobs_hm'],axis=1)

In [144]:
%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

pdata11
X_dataf = pdata11.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = pdata11[vif_data['feature'].tolist()]
vif_data

Wall time: 0 ns


,feature,VIF
1,stv_c_adb,3.707235
2,tertiary_binary,1.784519
3,path_binary,3.207257
4,Distance.to.Industrial.Area,2.996445
5,Distance.to.Industrial.Center,2.755604
6,Distance.to.Residential.Center,1.646210
7,BikeFac_binary,3.202380
8,Distance_to_CBD_mi,3.203842
9,Commercial.Area_hm,2.094970
10,Retail.Area_hm,1.744277


In [145]:
vif_data['feature']

1                             stv_c_adb
2                       tertiary_binary
3                           path_binary
4           Distance.to.Industrial.Area
5         Distance.to.Industrial.Center
6        Distance.to.Residential.Center
7                        BikeFac_binary
8                    Distance_to_CBD_mi
9                    Commercial.Area_hm
10                       Retail.Area_hm
11                         Bus.Stops_hm
12                          Tertiary_hm
13                              Path_hm
14                          Cycleway_hm
15                          pct_male_hm
16    pct_at_least_college_education_hm
Name: feature, dtype: object

# 0.75 mile buffer

In [146]:
p1_data8=p1_data.loc[:,p1_data.columns.str.contains('tfm', case=False)]
p1_data9=pd.concat([p1_data7, p1_data8.reindex(p1_data7.index)], axis=1)
p1_data10=p1_data9.loc[:,~p1_data9.columns.str.contains('ohm', case=False)]
p1_data10.shape

(88, 103)

In [147]:
corrMatrix1 = p1_data10.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.25]
top_corr_features

Index(['travel_pattern', 'AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw',
       'tertiary_binary', 'path_binary', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.CBD',
       'Point.Bridge', 'stv_nc_adb', 'sep_bikeway_binary', 'BikeFac_binary',
       'tertiary_no_bike_lane_binary', 'tertiary_bike_lane_binary',
       'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw',
       'log_stv_stl', 'Distance_to_CBD_mi', 'Industrial.Area_tfm',
       'Residential_Road_tfm', 'OSM_node_Density_tfm', 'Primary_tfm',
       'Secondary_tfm', 'Path_tfm', 'bridge_tfm', 'population_density_tfm',
       'employment_density_tfm', 'Number.of.jobs_tfm', 'sep_bikeway_tfm',
       'BikeFac_tfm'],
      dtype='object')

In [158]:
pdata1=p1_data10[top_corr_features]
pdata11=pdata1.drop(['log_stv_nc_adb','travel_pattern','stl_raw','tertiary_no_bike_lane_binary','log_stv_adb', 'stv_adb','Distance.to.CBD','tertiary_bike_lane_binary','sep_bikeway_tfm','Distance.to.Grass','Point.Bridge','sep_bikeway_binary','stv_nc_adb','OSM_node_Density_tfm','Distance.to.Grass.Center'],axis=1)

In [159]:
%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

pdata11
X_dataf = pdata11.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = pdata11[vif_data['feature'].tolist()]
vif_data

Wall time: 0 ns


,feature,VIF
1,tertiary_binary,1.966442
2,path_binary,2.930997
3,Distance.to.Industrial.Area,4.108410
4,Distance.to.Industrial.Center,3.321715
5,Distance.to.Residential.Center,1.723092
6,BikeFac_binary,3.556045
7,log_stv_c_adb,3.807156
8,log_stl_raw,4.268810
9,Distance_to_CBD_mi,3.908297
10,Industrial.Area_tfm,2.098057


In [160]:
vif_data['feature']

1                    tertiary_binary
2                        path_binary
3        Distance.to.Industrial.Area
4      Distance.to.Industrial.Center
5     Distance.to.Residential.Center
6                     BikeFac_binary
7                      log_stv_c_adb
8                        log_stl_raw
9                 Distance_to_CBD_mi
10               Industrial.Area_tfm
11              Residential_Road_tfm
12                       Primary_tfm
13                     Secondary_tfm
14                          Path_tfm
15                        bridge_tfm
16            employment_density_tfm
17                Number.of.jobs_tfm
18                       BikeFac_tfm
Name: feature, dtype: object

# portland 1 mile buffer

In [161]:
p1_data8=p1_data.loc[:,p1_data.columns.str.contains('_om', case=False)]
p1_data9=p1_data.loc[:,p1_data.columns.str.contains('_om_net', case=False)]
p1_data10=pd.concat([p1_data8, p1_data9.reindex(p1_data8.index)], axis=1)
p1_data12=pd.concat([p1_data7, p1_data8.reindex(p1_data7.index)], axis=1)
p1_data12.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,employment_density_om,Number.of.jobs_om,slope_om,Temp_avg_om,Hum_avg_om,Precipitation_om,sep_bikeway_om,BikeFac_om,BikeFac_onstreet_om,Park_acres_om
224,1.0,1775.363690,109.219178,80.945205,9915,0,0,0,0,1,...,7733.651969,114131.48600,0.000000,54.164932,69.331507,0.070274,59529.06164,34.356258,25.928075,66.650573
225,2.0,1537.211111,111.027397,74.904110,6663,0,0,0,0,1,...,7394.607673,112100.46520,0.000000,54.164932,69.331507,0.070274,59210.24525,33.761018,25.498225,70.113770
226,1.0,1346.261310,98.109589,59.917808,5466,0,0,0,0,0,...,5114.621634,63237.78023,3.153842,54.164932,69.331507,0.070274,54232.62402,27.637736,21.089580,49.214257
227,1.0,1951.052146,128.041096,84.410959,17323,0,0,0,0,1,...,6187.215677,109849.97880,0.000000,54.164932,69.331507,0.070274,38766.44078,37.317251,30.902969,57.695648
228,1.0,1182.406584,120.712329,82.164384,8077,0,0,0,0,1,...,6415.912593,87810.93053,0.000000,54.164932,69.331507,0.070274,31239.01252,34.001681,29.093835,37.898346


In [162]:
corrMatrix1 = p1_data12.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.25]
top_corr_features

Index(['travel_pattern', 'AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw',
       'tertiary_binary', 'path_binary', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.CBD',
       'Point.Bridge', 'stv_nc_adb', 'sep_bikeway_binary', 'BikeFac_binary',
       'tertiary_no_bike_lane_binary', 'tertiary_bike_lane_binary',
       'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw',
       'log_stv_stl', 'Distance_to_CBD_mi', 'Intersection_Density_om',
       'Bicycle.Parking_om', 'Bus.Stops_om', 'college_om',
       'OSM_node_Density_om', 'Primary_om', 'Secondary_om', 'Tertiary_om',
       'Path_om', 'Cycleway_om', 'cycleway_lane_all_om',
       'cycleway_track_all_om', 'pct_male_om', 'pct_female_om',
       'Student.Access_om', 'pct_at_least_college_education_om',
       'Number.of.jobs_om', 'sep_bikeway_om', 'BikeFac_om',
       'BikeFac_onstreet_om'],
      dtype

In [166]:
pdata1=p1_data12[top_corr_features]
pdata11=pdata1.drop(['Distance.to.Grass.Center','travel_pattern','stl_raw','tertiary_no_bike_lane_binary','log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw','stv_adb','Distance.to.CBD','tertiary_bike_lane_binary','Path_om','BikeFac_onstreet_om','pct_female_om','BikeFac_om','stv_nc_adb','log_stv_stl'],axis=1)

In [167]:
%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

pdata11
X_dataf = pdata11.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = pdata11[vif_data['feature'].tolist()]
vif_data

Wall time: 0 ns


,feature,VIF
1,stv_c_adb,2.903659
2,tertiary_binary,1.845758
3,path_binary,2.713233
4,Distance.to.Industrial.Center,2.259889
5,Distance.to.Residential.Center,2.224829
6,Distance.to.Grass,2.297360
7,Point.Bridge,2.815016
8,BikeFac_binary,2.298824
9,Intersection_Density_om,3.829914
10,Secondary_om,3.988168


In [168]:
vif_data['feature']

1                             stv_c_adb
2                       tertiary_binary
3                           path_binary
4         Distance.to.Industrial.Center
5        Distance.to.Residential.Center
6                     Distance.to.Grass
7                          Point.Bridge
8                        BikeFac_binary
9               Intersection_Density_om
10                         Secondary_om
11                          Cycleway_om
12                cycleway_track_all_om
13                          pct_male_om
14                    Student.Access_om
15    pct_at_least_college_education_om
Name: feature, dtype: object

# Portland 1.5 Mile Buffer

In [169]:
p1_data1=p1_data.loc[:,~p1_data.columns.str.contains('_otm', case=False)]
p1_data2=p1_data1.loc[:,~p1_data1.columns.str.contains('_qm', case=False)]
p1_data3=p1_data2.loc[:,~p1_data2.columns.str.contains('_hm', case=False)]
p1_data4=p1_data3.loc[:,~p1_data3.columns.str.contains('_tfm', case=False)]
p1_data5=p1_data4.loc[:,~p1_data4.columns.str.contains('_om', case=False)]
p1_data6=p1_data5.loc[:,~p1_data5.columns.str.contains('_tm', case=False)]
p1_data6.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,BikeFac_ohm,BikeFac_onstreet_ohm,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
224,1.0,1775.363690,109.219178,80.945205,9915,0,0,0,0,1,...,60.535084,48.735356,4.702471,4.406051,3.376699,9.201905,13.904376,113.195792,0.315109,0.049033
225,2.0,1537.211111,111.027397,74.904110,6663,0,0,0,0,1,...,59.854222,47.735910,4.718743,4.329471,3.614244,8.804475,13.523219,113.414613,0.403667,0.111882
226,1.0,1346.261310,98.109589,59.917808,5466,0,0,0,0,0,...,44.954221,34.402245,4.596226,4.109526,3.668467,8.606485,13.202711,81.480702,1.005728,0.848626
227,1.0,1951.052146,128.041096,84.410959,17323,0,0,0,0,1,...,62.103911,50.414415,4.860131,4.447474,3.798409,9.759848,14.619979,111.057680,1.028972,0.015873
228,1.0,1182.406584,120.712329,82.164384,8077,0,0,0,0,1,...,57.328453,46.782092,4.801660,4.420819,3.677514,8.996900,13.798560,107.614898,1.277080,0.154749


In [170]:
corrMatrix1 = p1_data6.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['travel_pattern', 'AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw',
       'tertiary_binary', 'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Retail.Area',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.CBD',
       'min_dist_to_college', 'Point.Speed', 'Point.Bridge',
       'Total.Commuter_ohm', 'Bike.Commuter_ohm',
       'Percentage.of.Bike.Commuter_ohm', 'bike.commuter.density_ohm',
       'Intersection_Density_ohm', 'Industrial.Area_ohm', 'Retail.Area_ohm',
       'Bicycle.Parking_ohm', 'Bus.Stops_ohm', 'college_ohm',
       'OSM_node_Density_ohm', 'Primary_ohm', 'Secondary_ohm', 'Tertiary_ohm',
       'Path_ohm', 'Cycleway_ohm', 'Footway_ohm', 'cycleway_lane_all_ohm',
       'cycleway_track_all_ohm', 'pct_white_ohm', 'pct_African_American_ohm',
       'pct_mal

In [185]:
pdata1=p1_data6[top_corr_features]
pdata11=pdata1.drop(['Distance.to.Industrial.Center','Distance.to.Commercial.Area','Distance.to.Commercial.Area.Center','Distance.to.Grass','Distance.to.Grass.Center','travel_pattern','stl_raw','tertiary_no_bike_lane_binary','log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw','stv_adb','Distance.to.CBD','tertiary_bike_lane_binary','Path_ohm','BikeFac_onstreet_ohm','pct_female_ohm','BikeFac_ohm','stv_nc_adb','log_stv_stl'],axis=1)

In [186]:
%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

pdata11
X_dataf = pdata11.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = pdata11[vif_data['feature'].tolist()]
vif_data

Wall time: 0 ns


C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,VIF
1,stv_c_adb,3.291265
2,tertiary_binary,1.988843
3,path_binary,3.949043
4,Distance.to.Industrial.Area,4.997968
5,Distance.to.Residential.Area,1.933591
6,Distance.to.Retail.Area,1.550051
7,min_dist_to_college,3.120588
8,Point.Speed,4.250525
9,Percentage.of.Bike.Commuter_ohm,3.003344
10,Industrial.Area_ohm,3.205776


In [187]:
vif_data['feature']

1                           stv_c_adb
2                     tertiary_binary
3                         path_binary
4         Distance.to.Industrial.Area
5        Distance.to.Residential.Area
6             Distance.to.Retail.Area
7                 min_dist_to_college
8                         Point.Speed
9     Percentage.of.Bike.Commuter_ohm
10                Industrial.Area_ohm
11                    Retail.Area_ohm
12                       Cycleway_ohm
13             cycleway_track_all_ohm
14           pct_African_American_ohm
15               Median_HH_income_ohm
16                          slope_ohm
17                 sep_bikeway_binary
18                    sep_bikeway_ohm
Name: feature, dtype: object

# Portland 2.0 mile buffer

In [188]:
p1_data1=p1_data.loc[:,~p1_data.columns.str.contains('_otm', case=False)]
p1_data2=p1_data1.loc[:,~p1_data1.columns.str.contains('_qm', case=False)]
p1_data3=p1_data2.loc[:,~p1_data2.columns.str.contains('_hm', case=False)]
p1_data4=p1_data3.loc[:,~p1_data3.columns.str.contains('_tfm', case=False)]
p1_data5=p1_data4.loc[:,~p1_data4.columns.str.contains('_om', case=False)]
p1_data6=p1_data5.loc[:,~p1_data5.columns.str.contains('_ohm', case=False)]
p1_data6.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,tertiary_bike_lane_binary,BikeFac_tm,BikeFac_onstreet_tm,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Distance_to_CBD_mi,Distance_to_Water_Body_mi
224,1.0,1775.363690,109.219178,80.945205,9915,0,0,0,0,1,...,0,84.252548,67.691203,4.702471,4.406051,3.376699,9.201905,13.904376,0.315109,0.049033
225,2.0,1537.211111,111.027397,74.904110,6663,0,0,0,0,1,...,0,84.252956,68.502935,4.718743,4.329471,3.614244,8.804475,13.523219,0.403667,0.111882
226,1.0,1346.261310,98.109589,59.917808,5466,0,0,0,0,0,...,0,69.327174,54.779167,4.596226,4.109526,3.668467,8.606485,13.202711,1.005728,0.848626
227,1.0,1951.052146,128.041096,84.410959,17323,0,0,0,0,1,...,0,80.388015,65.755245,4.860131,4.447474,3.798409,9.759848,14.619979,1.028972,0.015873
228,1.0,1182.406584,120.712329,82.164384,8077,0,0,0,0,1,...,0,78.827634,64.123416,4.801660,4.420819,3.677514,8.996900,13.798560,1.277080,0.154749


In [193]:
corrMatrix1 = p1_data6.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['travel_pattern', 'AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw',
       'tertiary_binary', 'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Retail.Area',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.CBD',
       'min_dist_to_college', 'Point.Speed', 'Point.Bridge',
       'Total.Commuter_tm', 'Bike.Commuter_tm',
       'Percentage.of.Bike.Commuter_tm', 'bike.commuter.density_tm',
       'Intersection_Density_tm', 'Industrial.Area_tm', 'Retail.Area_tm',
       'Bicycle.Parking_tm', 'Bus.Stops_tm', 'School_tm', 'college_tm',
       'University_tm', 'OSM_node_Density_tm', 'Primary_tm', 'Secondary_tm',
       'Tertiary_tm', 'Cycleway_tm', 'Footway_tm', 'cycleway_lane_all_tm',
       'cycleway_track_all_tm', 'pct_white_tm', 'pct_African_American_tm',
       'pct_male_

In [203]:
pdata1=p1_data6[top_corr_features]
pdata11=pdata1.drop(['Percentage.of.Bike.Commuter_tm','tertiary_no_bike_lane_binary','tertiary_bike_lane_binary','Distance.to.Grass','travel_pattern','stl_raw','log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw','stv_adb','Distance.to.CBD','pct_female_tm','BikeFac_tm','stv_nc_adb','log_stv_stl','bike.commuter.density_tm','Total.Commuter_tm','Distance.to.Grass.Center','HH_density_tm','OSM_node_Density_tm','cycleway_track_all_tm','cycleway_lane_all_tm'],axis=1)

In [204]:
%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

pdata11
X_dataf = pdata11.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = pdata11[vif_data['feature'].tolist()]
vif_data

Wall time: 0 ns


,feature,VIF
1,stv_c_adb,2.460862
2,tertiary_binary,1.854398
3,path_binary,3.389324
4,Distance.to.Commercial.Area.Center,1.988878
5,Distance.to.Residential.Area,1.617656
6,Distance.to.Retail.Area,1.539085
7,min_dist_to_college,1.959797
8,Industrial.Area_tm,2.352224
9,Retail.Area_tm,2.393934
10,University_tm,1.383981


In [205]:
vif_data['feature']

1                              stv_c_adb
2                        tertiary_binary
3                            path_binary
4     Distance.to.Commercial.Area.Center
5           Distance.to.Residential.Area
6                Distance.to.Retail.Area
7                    min_dist_to_college
8                     Industrial.Area_tm
9                         Retail.Area_tm
10                         University_tm
11                           Cycleway_tm
12                           pct_male_tm
13     pct_at_least_college_education_tm
14                              slope_tm
15                    sep_bikeway_binary
16                        BikeFac_binary
17                        sep_bikeway_tm
Name: feature, dtype: object

# Eugene all bufferes

In [209]:
e_data=data[data['region']=='Eugene']
e_data.head()

,Unnamed: 0,site_id,site_name,type,year,region,travel_pattern,AADBT,stv_adb,stv_c_adb,...,Bike.Share.best_tm,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi,rowIndex
147,148,P001EUG,Fern Ridge Path west of Chambers,permanent,2019,Eugene,2.0,513.408379,12.575342,3.780822,...,147890.0,2.57938,10.240767,38.828042,81.100254,109.983487,177.408835,1.582742,1.272067,148
148,149,P002EUG,Millrace Path @ Booth Kelly,permanent,2019,Eugene,3.0,109.807738,7.780822,2.397260,...,2374.0,0.00000,0.000000,7.688736,32.359430,60.783955,87.937981,3.765385,0.896833,149
149,150,P003EUG,Southbank Path west of Knickerbocker Bridge,permanent,2019,Eugene,2.0,330.297326,10.383562,4.863014,...,182248.0,0.00000,2.580734,52.336410,100.643748,160.837403,250.348289,1.934576,0.329983,150
150,151,P004EUG,North Bank Path south of Greenway Bridge,permanent,2019,Eugene,3.0,429.732467,19.589041,7.342466,...,83568.0,0.06550,26.300129,75.951888,133.116175,253.846733,388.687995,1.526089,0.171903,151
151,152,P005EUG,Alder north of 18th Ave,permanent,2019,Eugene,1.0,410.282143,6.972603,4.479452,...,192315.0,0.00000,0.000000,2.284680,20.413902,48.177451,241.359538,0.917704,0.394418,152


In [210]:
# Clean the data
an_data=e_data._get_numeric_data()
nan_values = an_data. isna()
nan_columns = nan_values. any()
columns_with_nan = an_data. columns[nan_columns]. tolist()
print(columns_with_nan)

[]


In [211]:
e1_data=an_data.drop(an_data. columns[nan_columns]. tolist(),axis=1)
e1_data=e1_data.drop(['Unnamed: 0','year','latitude','longitude','rowIndex','Bike.Share.Origin_otm',
 'Bike.Share.Destination_otm',
 'Bike.Share.Crossing_otm','Bike.Share.Origin_qm',
 'Bike.Share.Destination_qm',
 'Bike.Share.Crossing_qm','Bike.Share.Origin_tfm',
 'Bike.Share.Destination_tfm',
 'Bike.Share.Crossing_tfm','Bike.Share.Origin_om',
 'Bike.Share.Destination_om',
 'Bike.Share.Crossing_om','Bike.Share.Origin_ohm',
 'Bike.Share.Destination_ohm',
 'Bike.Share.Crossing_ohm','Bike.Share.Origin_tm',
 'Bike.Share.Destination_tm',
 'Bike.Share.Crossing_tm','Bike.Share.Origin_qm_net',
 'Bike.Share.Destination_qm_net',
 'Bike.Share.Crossing_qm_net','Bike.Share.Origin_otm_net',
 'Bike.Share.Destination_otm_net',
 'Bike.Share.Crossing_otm_net','Bike.Share.Origin_hm_net',
 'Bike.Share.Destination_hm_net',
 'Bike.Share.Crossing_hm_net','Bike.Share.Origin_tfm_net',
 'Bike.Share.Destination_tfm_net',
 'Bike.Share.Crossing_tfm_net','Bike.Share.Origin_om_net',
 'Bike.Share.Destination_om_net',
 'Bike.Share.Crossing_om_net','Bike.Share.Origin_ohm_net',
 'Bike.Share.Destination_ohm_net',
 'Bike.Share.Crossing_ohm_net', 'Bike.Share.Origin_tm_net',
 'Bike.Share.Destination_tm_net',
 'Bike.Share.Crossing_tm_net','Bike.Share.best_otm',
 'Bike.Share.best_qm',
 'Bike.Share.best_hm',
 'Bike.Share.best_tfm',
 'Bike.Share.best_om',
 'Bike.Share.best_ohm',
 'Bike.Share.best_tm','Bike.Share.Origin_hm',
 'Bike.Share.Destination_hm',
 'Bike.Share.Crossing_hm','Bike.Share.route_otm_net',
'Bike.Share.route_otm','Bike.Share.route_qm','Bike.Share.route_tfm','Bike.Share.route_qm_net',
'Bike.Share.route_hm','Bike.Share.route_om'],axis=1)

e1_data.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,log_stl_raw,log_stv_stl,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
147,2.0,513.408379,12.575342,3.780822,985,0,0,0,0,0,...,6.893656,9.501911,2.57938,10.240767,38.828042,81.100254,109.983487,177.408835,1.582742,1.272067
148,3.0,109.807738,7.780822,2.397260,203,0,0,0,0,0,...,5.318120,7.490690,0.00000,0.000000,7.688736,32.359430,60.783955,87.937981,3.765385,0.896833
149,2.0,330.297326,10.383562,4.863014,2766,0,0,0,0,0,...,7.925519,10.357689,0.00000,2.580734,52.336410,100.643748,160.837403,250.348289,1.934576,0.329983
150,3.0,429.732467,19.589041,7.342466,1233,0,0,0,0,0,...,7.118016,10.142775,0.06550,26.300129,75.951888,133.116175,253.846733,388.687995,1.526089,0.171903
151,1.0,410.282143,6.972603,4.479452,2154,0,0,0,0,0,...,7.675546,9.751557,0.00000,0.000000,2.284680,20.413902,48.177451,241.359538,0.917704,0.394418


In [212]:
e1_data=e1_data.loc[:,~e1_data.columns.str.contains('_net', case=False)] 
e1_data.shape

(77, 401)

In [220]:
corrMatrix1 = e1_data.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.40]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'Distance.to.Park.Center',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_college',
       'min_dist_to_university', 'Grass.Area_otm',
       ...
       'BikeFac_onstreet_tfm', 'BikeFac_onstreet_om', 'BikeFac_onstreet_ohm',
       'BikeFac_onstreet_tm', 'log_stv_adb', 'log_stv_c_adb', 'log_stl_raw',
       'log_stv_stl', 'Park_acres_ohm', 'Distance_to_CBD_mi'],
      dtype='object', length=121)

In [233]:
edata0=e1_data[top_corr_features]
#pdata00=pdata0.drop(['log_stl_raw','stv_adb','travel_pattern','Bike.Share.best_otm','Bike.Share.Destination_otm_net','Bike.Share.Crossing_otm_net','Bike.Share.Origin_otm_net','Distance.to.CBD','Path_otm_net','tertiary_no_bike_lane_binary','tertiary_bike_lane_binary','employment_density_otm_net','Distance.to.Grass.Center','log_stv_adb','employment_density_otm','sep_bikeway_otm','sep_bikeway_otm_net','stl_raw','Primary_otm','log_stv_c_adb','log_stv_nc_adb','sep_bikeway_binary','Residential_Road_otm','stv_nc_adb','log_stv_stl','BikeFac_otm','Industrial.Area_otm','bridge_otm','Bike.Share.Crossing_otm','BikeFac_binary','OSM_node_Density_otm_net'],axis=1)
edata00=edata0.drop(['Grass.Area_hm','Percentage.of.Bike.Commuter_tm','Grass.Area_tfm','Grass.Area_ohm','stv_c_adb','Path_hm','Median_HH_income_om','Cycleway_tfm','Cycleway_hm','Student.Access_hm','Footway_tfm','University_qm','University_om'],axis=1)
#plt.figure(figsize=(18,14))
#plt.title("2019_cor-relation>0.20 with AADBT")
#g = sns.heatmap(edata00[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [234]:
%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

edata00
X_dataf = edata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = edata00[vif_data['feature'].tolist()]
vif_data

Wall time: 0 ns


C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\regression\linear_model.py:1738: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,VIF
1,Bus.Stops_qm,1.670356
2,Primary_hm,2.613455
3,Footway_hm,4.137535
4,pct_white_tfm,3.843018
5,pct_African_American_tfm,4.840492
6,Bicycle.Parking_om,3.128242
7,Percentage.of.Bike.Commuter_ohm,4.283886
8,Grass.Area_tm,4.081135
9,college_tm,3.194905
10,Median_HH_income_tm,2.847868


In [235]:
vif_data['feature']

1                        Bus.Stops_qm
2                          Primary_hm
3                          Footway_hm
4                       pct_white_tfm
5            pct_African_American_tfm
6                  Bicycle.Parking_om
7     Percentage.of.Bike.Commuter_ohm
8                       Grass.Area_tm
9                          college_tm
10                Median_HH_income_tm
11                     sep_bikeway_qm
12                BikeFac_onstreet_hm
13                      log_stv_c_adb
14                        log_stl_raw
15                     Park_acres_ohm
Name: feature, dtype: object

# 0.1 mile Buffer

In [236]:
e1_data2=e1_data.loc[:,~e1_data.columns.str.contains('qm', case=False)] 
e1_data3=e1_data2.loc[:,~e1_data2.columns.str.contains('hm', case=False)] 
e1_data4=e1_data3.loc[:,~e1_data3.columns.str.contains('tfm', case=False)] 
e1_data5=e1_data4.loc[:,~e1_data4.columns.str.contains('om', case=False)] 
e1_data6=e1_data5.loc[:,~e1_data5.columns.str.contains('ohm', case=False)] 
e1_data7=e1_data6.loc[:,~e1_data6.columns.str.contains('tm', case=False)]
# keep the data for one-tenth mile buffer only
e1_data8=e1_data.loc[:,e1_data.columns.str.contains('otm', case=False)]
e1_data9=pd.concat([e1_data7, e1_data8.reindex(e1_data7.index)], axis=1)
e1_data9.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,employment_density_otm,Number.of.jobs_otm,slope_otm,Temp_avg_otm,Hum_avg_otm,Precipitation_otm,sep_bikeway_otm,BikeFac_otm,BikeFac_onstreet_otm,Park_acres_otm
147,2.0,513.408379,12.575342,3.780822,985,0,0,0,0,0,...,9877.842291,58.534922,1.019620,52.194247,75.815616,0.099863,1560.926300,0.541816,0.541816,2.57938
148,3.0,109.807738,7.780822,2.397260,203,0,0,0,0,0,...,1573.053115,62.914584,0.000000,52.194247,75.815616,0.099863,676.098089,0.225905,0.225905,0.00000
149,2.0,330.297326,10.383562,4.863014,2766,0,0,0,0,0,...,3001.492056,14.808355,3.970095,52.194247,75.815616,0.099863,3073.451994,0.582093,0.409458,0.00000
150,3.0,429.732467,19.589041,7.342466,1233,0,0,0,0,0,...,1308.626049,270.641179,1.001548,52.194247,75.815616,0.099863,1618.465744,0.306528,0.306528,0.06550
151,1.0,410.282143,6.972603,4.479452,2154,0,0,0,0,0,...,14284.316280,35.311578,2.643780,52.194247,75.815616,0.099863,1054.833556,0.816226,0.816226,0.00000


In [237]:
corrMatrix1 = e1_data9.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.Water.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_college',
       'min_dist_to_university', 'stv_nc_adb', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Residential_Road_otm', 'Grass.Area_otm',
       'Forest.Area_otm', 'Bicycle.Parking_otm', 'Bus.Stops_otm',
       'University_otm', 'Cycleway_otm', 'Footway_otm', 'bridge_otm',
       'pct_white_otm', 'pct_African_American_otm', 'pct_male_otm',
       'pct_female_

In [247]:
edata0=e1_data[top_corr_features]
edata00=edata0.drop(['stv_c_adb','tertiary_no_bike_lane_binary','Grass.Area_otm','Distance.to.Grass.Center','Distance.to.Grass','tertiary_bike_lane_binary','Distance.to.Grass.Center'],axis=1)

In [248]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

edata00
X_dataf = edata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = edata00[vif_data['feature'].tolist()]
vif_data

C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\regression\linear_model.py:1738: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 594 ms


,feature,VIF
1,stl_raw,2.168323
2,tertiary_binary,2.157570
3,Distance.to.Industrial.Center,3.358705
4,Distance.to.Retail.Center,3.636250
5,Distance.to.Park,1.493258
6,Distance.to.Water.Center,3.857039
7,Distance.to.forest,2.817462
8,stv_nc_adb,1.803587
9,Residential_Road_otm,2.812244
10,Forest.Area_otm,2.277345


In [252]:
vif_data['feature']

1                                stl_raw
2                        tertiary_binary
3          Distance.to.Industrial.Center
4              Distance.to.Retail.Center
5                       Distance.to.Park
6               Distance.to.Water.Center
7                     Distance.to.forest
8                             stv_nc_adb
9                   Residential_Road_otm
10                       Forest.Area_otm
11                   Bicycle.Parking_otm
12                         Bus.Stops_otm
13                        University_otm
14                          Cycleway_otm
15                            bridge_otm
16                         pct_white_otm
17              pct_African_American_otm
18                        pct_female_otm
19                    Student.Access_otm
20    pct_at_least_college_education_otm
21                        Median.Age_otm
22                  Median_HH_income_otm
23                             slope_otm
Name: feature, dtype: object

# Eugene 0.25 mile buffer

In [253]:
e1_data8=e1_data.loc[:,e1_data.columns.str.contains('qm', case=False)]
e1_data9=pd.concat([e1_data7, e1_data8.reindex(e1_data7.index)], axis=1)
e1_data9.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,employment_density_qm,Number.of.jobs_qm,slope_qm,Temp_avg_qm,Hum_avg_qm,Precipitation_qm,sep_bikeway_qm,BikeFac_qm,BikeFac_onstreet_qm,Park_acres_qm
147,2.0,513.408379,12.575342,3.780822,985,0,0,0,0,0,...,6545.850017,2348.849945,1.072945,52.194247,75.815616,0.099863,7166.254349,3.881654,3.881654,10.240767
148,3.0,109.807738,7.780822,2.397260,203,0,0,0,0,0,...,1812.188737,563.460101,0.000000,52.194247,75.815616,0.099863,1514.051190,0.815407,0.815407,0.000000
149,2.0,330.297326,10.383562,4.863014,2766,0,0,0,0,0,...,2177.814063,115.958424,2.905628,52.194247,75.815616,0.099863,10673.515950,2.021499,1.654200,2.580734
150,3.0,429.732467,19.589041,7.342466,1233,0,0,0,0,0,...,1587.129122,1403.268007,1.365528,52.194247,75.815616,0.099863,8142.031078,1.831853,1.831853,26.300129
151,1.0,410.282143,6.972603,4.479452,2154,0,0,0,0,0,...,15074.112770,184.408601,2.061826,52.194247,75.815616,0.099863,3406.484822,2.619919,2.619919,0.000000


In [254]:
corrMatrix1 = e1_data9.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.Water.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_college',
       'min_dist_to_university', 'stv_nc_adb', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Percentage.of.Bike.Commuter_qm',
       'Residential_Road_qm', 'Grass.Area_qm', 'Forest.Area_qm',
       'Bicycle.Parking_qm', 'Bus.Stops_qm', 'University_qm', 'Primary_qm',
       'Path_qm', 'Cycleway_qm', 'Footway_qm', 'bridge_qm', 'pct_white_qm',
       'pct_Af

In [261]:
edata0=e1_data[top_corr_features]
edata00=edata0.drop(['stv_c_adb','Distance.to.Grass','Distance.to.Grass.Center','Grass.Area_qm','stv_nc_adb','tertiary_bike_lane_binary','tertiary_no_bike_lane_binary'],axis=1)

In [262]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

edata00
X_dataf = edata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = edata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\regression\linear_model.py:1738: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 859 ms


,feature,VIF
5,Distance.to.Park,1.572447
23,slope_qm,1.783238
2,tertiary_binary,1.901450
1,stl_raw,2.125312
7,log_stv_nc_adb,2.238439
14,Primary_qm,2.270767
13,University_qm,2.389327
15,bridge_qm,2.498791
20,pct_at_least_college_education_qm,2.746726
10,Forest.Area_qm,2.791499


In [263]:
vif_data['feature']

1                               stl_raw
2                       tertiary_binary
3           Distance.to.Industrial.Area
4             Distance.to.Retail.Center
5                      Distance.to.Park
6                    Distance.to.forest
7                        log_stv_nc_adb
8        Percentage.of.Bike.Commuter_qm
9                   Residential_Road_qm
10                       Forest.Area_qm
11                   Bicycle.Parking_qm
12                         Bus.Stops_qm
13                        University_qm
14                           Primary_qm
15                            bridge_qm
16                         pct_white_qm
17              pct_African_American_qm
18                        pct_female_qm
19                    Student.Access_qm
20    pct_at_least_college_education_qm
21                        Median.Age_qm
22                    Number.of.jobs_qm
23                             slope_qm
24                       sep_bikeway_qm
Name: feature, dtype: object

# Eugene 0.5 mile buffer

In [264]:
e1_data8=e1_data.loc[:,e1_data.columns.str.contains('hm', case=False)]
e1_data9=pd.concat([e1_data7, e1_data8.reindex(e1_data7.index)], axis=1)
e1_data10=e1_data9.loc[:,~e1_data9.columns.str.contains('ohm', case=False)]
e1_data10.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,Number.of.jobs_hm,slope_hm,Temp_avg_hm,Hum_avg_hm,Precipitation_hm,sep_bikeway_hm,BikeFac_hm,BikeFac_onstreet_hm,school_college_hm,Park_acres_hm
147,2.0,513.408379,12.575342,3.780822,985,0,0,0,0,0,...,6657.828784,1.077072,52.194247,75.815616,0.099863,19537.849010,9.315511,9.315511,3,38.828042
148,3.0,109.807738,7.780822,2.397260,203,0,0,0,0,0,...,2134.928769,0.000000,52.194247,75.815616,0.099863,3996.555127,3.080564,3.080564,5,7.688736
149,2.0,330.297326,10.383562,4.863014,2766,0,0,0,0,0,...,627.799125,3.122945,52.194247,75.815616,0.099863,44661.588740,8.561418,4.479115,0,52.336410
150,3.0,429.732467,19.589041,7.342466,1233,0,0,0,0,0,...,4578.448797,1.087621,52.194247,75.815616,0.099863,21230.193220,7.421639,7.216028,0,75.951888
151,1.0,410.282143,6.972603,4.479452,2154,0,0,0,0,0,...,8648.800244,2.186660,52.194247,75.815616,0.099863,15015.418970,8.275185,8.110776,2,2.284680


In [265]:
corrMatrix1 = e1_data10.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.Water.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_college',
       'min_dist_to_university', 'stv_nc_adb', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Total.Commuter_hm', 'Bike.Commuter_hm',
       'Percentage.of.Bike.Commuter_hm', 'bike.commuter.density_hm',
       'Residential_Road_hm', 'Grass.Area_hm', 'Water.Area_hm',
       'Bicycle.Parking_hm', 'Bus.Stops_hm', 'University_hm', 'Primary_hm',
       'Path_hm

In [273]:
edata0=e1_data[top_corr_features]
edata00=edata0.drop(['tertiary_bike_lane_binary','tertiary_no_bike_lane_binary','cycleway_lane_all_hm','stv_adb','Distance.to.Grass','Distance.to.Grass.Center','Grass.Area_hm','stv_nc_adb','stl_raw'],axis=1)

In [274]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

edata00
X_dataf = edata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = edata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 885 ms


,feature,VIF
1,tertiary_binary,1.961198
10,Water.Area_hm,2.003791
4,Distance.to.Park,2.072761
6,log_stv_nc_adb,2.238721
5,Distance.to.forest,2.254787
15,bridge_hm,2.292900
7,log_stl_raw,2.415079
12,Bus.Stops_hm,2.661799
18,pct_at_least_college_education_hm,2.702241
13,University_hm,2.779971


In [275]:
vif_data['feature']

1                       tertiary_binary
2         Distance.to.Industrial.Center
3             Distance.to.Retail.Center
4                      Distance.to.Park
5                    Distance.to.forest
6                        log_stv_nc_adb
7                           log_stl_raw
8        Percentage.of.Bike.Commuter_hm
9                   Residential_Road_hm
10                        Water.Area_hm
11                   Bicycle.Parking_hm
12                         Bus.Stops_hm
13                        University_hm
14                           Primary_hm
15                            bridge_hm
16                         pct_white_hm
17              pct_African_American_hm
18    pct_at_least_college_education_hm
19                  BikeFac_onstreet_hm
Name: feature, dtype: object

# Eugene 0.75 mile buffer

In [276]:
e1_data8=e1_data.loc[:,e1_data.columns.str.contains('tfm', case=False)]
e1_data9=pd.concat([e1_data7, e1_data8.reindex(e1_data7.index)], axis=1)
e1_data10=e1_data9.loc[:,~e1_data9.columns.str.contains('ohm', case=False)]
e1_data10.shape

(77, 103)

In [277]:
corrMatrix1 = e1_data10.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.30]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'Distance.to.Retail.Area',
       'Distance.to.Retail.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.Water.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_college',
       'min_dist_to_university', 'stv_nc_adb', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Total.Commuter_tfm', 'Bike.Commuter_tfm',
       'Percentage.of.Bike.Commuter_tfm', 'bike.commuter.density_tfm',
       'Grass.Area_tfm', 'Bicycle.Parking_tfm', 'University_tfm',
       'Primary_tfm', 'Path_tfm', 'Cycleway_tfm', 'Footway_tfm',
       'pct_white_tfm', 'pct_African_American_tfm', 'Student.Access_tfm',
       'pct_at_least_college_education_tfm', 'Median.Age_tfm',
       'Median_HH_income_tfm', 'populatio

In [278]:
edata0=e1_data[top_corr_features]
edata00=edata0.drop(['stv_adb','Distance.to.Grass','Distance.to.Grass.Center','Grass.Area_tfm','stv_nc_adb','stl_raw'],axis=1)

In [279]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

edata00
X_dataf = edata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = edata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 647 ms


,feature,VIF
2,Distance.to.Park,1.544621
4,Distance.to.forest,2.227516
3,Distance.to.Water.Center,2.291907
9,Primary_tfm,2.699601
8,Bicycle.Parking_tfm,3.183536
12,pct_at_least_college_education_tfm,3.217340
1,Distance.to.Retail.Area,3.323588
10,pct_white_tfm,3.390359
5,log_stv_nc_adb,3.439465
7,Total.Commuter_tfm,3.535809


In [280]:
vif_data['feature']

1                Distance.to.Retail.Area
2                       Distance.to.Park
3               Distance.to.Water.Center
4                     Distance.to.forest
5                         log_stv_nc_adb
6                            log_stv_stl
7                     Total.Commuter_tfm
8                    Bicycle.Parking_tfm
9                            Primary_tfm
10                         pct_white_tfm
11              pct_African_American_tfm
12    pct_at_least_college_education_tfm
13                       sep_bikeway_tfm
Name: feature, dtype: object

# Eugene 1 mile buffer

In [281]:
e1_data8=e1_data.loc[:,e1_data.columns.str.contains('_om', case=False)]
e1_data9=e1_data.loc[:,e1_data.columns.str.contains('_om_net', case=False)]
e1_data10=pd.concat([e1_data8, e1_data9.reindex(e1_data8.index)], axis=1)
e1_data12=pd.concat([e1_data7, e1_data8.reindex(e1_data7.index)], axis=1)
e1_data12.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,employment_density_om,Number.of.jobs_om,slope_om,Temp_avg_om,Hum_avg_om,Precipitation_om,sep_bikeway_om,BikeFac_om,BikeFac_onstreet_om,Park_acres_om
147,2.0,513.408379,12.575342,3.780822,985,0,0,0,0,0,...,4356.692399,11938.058150,1.726534,52.194247,75.815616,0.099863,32418.59213,20.269532,20.264638,109.983487
148,3.0,109.807738,7.780822,2.397260,203,0,0,0,0,0,...,2382.369313,5574.641856,0.000000,52.194247,75.815616,0.099863,17612.09841,11.401768,10.966742,60.783955
149,2.0,330.297326,10.383562,4.863014,2766,0,0,0,0,0,...,2389.527596,3953.920444,2.590774,52.194247,75.815616,0.099863,109530.01290,27.616772,15.885294,160.837403
150,3.0,429.732467,19.589041,7.342466,1233,0,0,0,0,0,...,2479.498873,11191.042050,1.231185,52.194247,75.815616,0.099863,51207.41772,27.362905,26.993746,253.846733
151,1.0,410.282143,6.972603,4.479452,2154,0,0,0,0,0,...,7596.847670,22706.941500,2.052708,52.194247,75.815616,0.099863,84027.55882,35.973329,31.055170,48.177451


In [282]:
corrMatrix1 = e1_data12.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.25]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.Water.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_college',
       'min_dist_to_university', 'stv_nc_adb', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Total.Commuter_om', 'Bike.Commuter_om',
       'Percentage.of.Bike.Commuter_om', 'bike.commuter.density_om',
       'Retail.Area_om', 'Grass.Area_om', 'Bicycle.Parking_om', 'college_om',
       'University_om', 'OSM_node_Density_om', 'Primary_om', 'Secondary_om',
       'Path_om', 'Cycleway_om', 'Footway_om', 'cycleway_lane_all_om

In [291]:
edata0=e1_data[top_corr_features]
edata00=edata0.drop([ 'Distance_to_Water_Body_mi','OSM_node_Density_om','stv_adb','Distance.to.Grass','Distance.to.Grass.Center','stv_nc_adb','stl_raw','Grass.Area_om','cycleway_lane_all_om'],axis=1)

In [292]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

edata00
X_dataf = edata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = edata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 971 ms


,feature,VIF
4,Distance.to.Park,1.618867
2,tertiary_binary,1.660270
6,Distance.to.forest,1.858802
13,maxspeed_om,1.928432
9,Retail.Area_om,2.026527
8,log_stl_raw,2.142841
5,Distance.to.Water.Body,2.192475
10,college_om,2.596132
14,pct_at_least_college_education_om,3.041030
11,Primary_om,3.320261


In [294]:
vif_data['feature']

1                             stv_c_adb
2                       tertiary_binary
3         Distance.to.Industrial.Center
4                      Distance.to.Park
5                Distance.to.Water.Body
6                    Distance.to.forest
7                        log_stv_nc_adb
8                           log_stl_raw
9                        Retail.Area_om
10                           college_om
11                           Primary_om
12                         Secondary_om
13                          maxspeed_om
14    pct_at_least_college_education_om
Name: feature, dtype: object

# Eugene 1.5 mile buffer Test

In [1371]:
e1_data1=e1_data.loc[:,~e1_data.columns.str.contains('_otm', case=False)]
e1_data2=e1_data1.loc[:,~e1_data1.columns.str.contains('_qm', case=False)]
e1_data3=e1_data2.loc[:,~e1_data2.columns.str.contains('_hm', case=False)]
e1_data4=e1_data3.loc[:,~e1_data3.columns.str.contains('_tfm', case=False)]
e1_data5=e1_data4.loc[:,~e1_data4.columns.str.contains('_om', case=False)]
e1_data6=e1_data5.loc[:,~e1_data5.columns.str.contains('_tm', case=False)]
e1_data6.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,sep_bikeway_ohm_net,BikeFac_ohm_net,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
147,2.0,513.408379,12.575342,3.780822,985,0,0,0,0,0,...,41592.31486,31.157091,2.608255,1.564612,2.281823,6.893656,9.501911,177.408835,1.582742,1.272067
148,3.0,109.807738,7.780822,2.397260,203,0,0,0,0,0,...,43380.68769,27.171879,2.172570,1.222969,1.853726,5.318120,7.490690,87.937981,3.765385,0.896833
149,2.0,330.297326,10.383562,4.863014,2766,0,0,0,0,0,...,137974.29400,36.479439,2.432170,1.768664,1.874958,7.925519,10.357689,250.348289,1.934576,0.329983
150,3.0,429.732467,19.589041,7.342466,1233,0,0,0,0,0,...,60210.99915,32.882896,3.024759,2.121359,2.583739,7.118016,10.142775,388.687995,1.526089,0.171903
151,1.0,410.282143,6.972603,4.479452,2154,0,0,0,0,0,...,230364.07390,92.258786,2.076011,1.701005,1.250804,7.675546,9.751557,241.359538,0.917704,0.394418


In [1372]:
corrMatrix1 = e1_data6.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.25]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.Water.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_college',
       'min_dist_to_university', 'Total.Commuter_ohm', 'Bike.Commuter_ohm',
       'Percentage.of.Bike.Commuter_ohm', 'bike.commuter.density_ohm',
       'Retail.Area_ohm', 'Grass.Area_ohm', 'Park.Area_ohm',
       'Bicycle.Parking_ohm', 'college_ohm', 'University_ohm',
       'OSM_node_Density_ohm', 'Primary_ohm', 'Secondary_ohm', 'Path_ohm',
       'Cycleway_ohm', 'Footway_ohm', 'cycleway_lane_all_ohm', 'maxspeed_ohm',
       'pct_white_ohm', 'pct_African_American_ohm', 'pct_male_ohm',
       'pct_female_ohm', 'Studen

In [1373]:
edata0=e1_data[top_corr_features]
edata00=edata0.drop(['stv_adb','Distance.to.Grass','Distance.to.Grass.Center','stv_nc_adb','stl_raw','Grass.Area_ohm_net','Grass.Area_ohm','pct_female_ohm','pct_male_ohm','OSM_node_Density_ohm','cycleway_lane_all_ohm'],axis=1)

In [1374]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

edata00
X_dataf = edata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = edata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 3.73 s


,feature,VIF
5,Distance.to.Park,1.773455
2,tertiary_binary,1.925182
17,log_stl_raw,2.157802
7,Distance.to.Forest.Center,2.479320
6,Distance.to.Water.Center,3.302058
13,pct_white_ohm_net,3.358109
11,Water.Area_ohm_net,3.436196
15,HH_density_ohm_net,3.465803
8,Retail.Area_ohm,3.511533
3,Distance.to.Industrial.Area,3.556994


In [1375]:
vif_data['feature']

1                                  stv_c_adb
2                            tertiary_binary
3                Distance.to.Industrial.Area
4                    Distance.to.Retail.Area
5                           Distance.to.Park
6                   Distance.to.Water.Center
7                  Distance.to.Forest.Center
8                            Retail.Area_ohm
9                        Bicycle.Parking_ohm
10                              maxspeed_ohm
11                        Water.Area_ohm_net
12                           Primary_ohm_net
13                         pct_white_ohm_net
14    pct_at_least_college_education_ohm_net
15                        HH_density_ohm_net
16                            log_stv_nc_adb
17                               log_stl_raw
Name: feature, dtype: object

# Eugene 2 mile buffer

In [1376]:
e1_data1=e1_data.loc[:,~e1_data.columns.str.contains('_otm', case=False)]
e1_data2=e1_data1.loc[:,~e1_data1.columns.str.contains('_qm', case=False)]
e1_data3=e1_data2.loc[:,~e1_data2.columns.str.contains('_hm', case=False)]
e1_data4=e1_data3.loc[:,~e1_data3.columns.str.contains('_tfm', case=False)]
e1_data5=e1_data4.loc[:,~e1_data4.columns.str.contains('_om', case=False)]
e1_data6=e1_data5.loc[:,~e1_data5.columns.str.contains('_ohm', case=False)]
e1_data6.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,BikeFac_onstreet_tm,sep_bikeway_tm_net,BikeFac_tm_net,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Distance_to_CBD_mi,Distance_to_Water_Body_mi
147,2.0,513.408379,12.575342,3.780822,985,0,0,0,0,0,...,93.130737,86955.45017,64.614200,2.608255,1.564612,2.281823,6.893656,9.501911,1.582742,1.272067
148,3.0,109.807738,7.780822,2.397260,203,0,0,0,0,0,...,47.390156,133682.92850,61.216622,2.172570,1.222969,1.853726,5.318120,7.490690,3.765385,0.896833
149,2.0,330.297326,10.383562,4.863014,2766,0,0,0,0,0,...,69.595302,191007.29350,57.279205,2.432170,1.768664,1.874958,7.925519,10.357689,1.934576,0.329983
150,3.0,429.732467,19.589041,7.342466,1233,0,0,0,0,0,...,104.989063,113423.05130,64.485386,3.024759,2.121359,2.583739,7.118016,10.142775,1.526089,0.171903
151,1.0,410.282143,6.972603,4.479452,2154,0,0,0,0,0,...,92.617217,423183.61790,154.650904,2.076011,1.701005,1.250804,7.675546,9.751557,0.917704,0.394418


In [1377]:
corrMatrix1 = e1_data6.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass',
       ...
       'BikeFac_onstreet_tm', 'sep_bikeway_tm_net', 'BikeFac_tm_net',
       'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw',
       'log_stv_stl', 'Distance_to_CBD_mi', 'Distance_to_Water_Body_mi'],
      dtype='object', length=101)

In [1378]:
edata0=e1_data[top_corr_features]
edata00=edata0.drop(['stv_adb','Distance.to.Grass','Distance.to.Grass.Center','stv_nc_adb','stl_raw','Grass.Area_tm_net','Grass.Area_tm','pct_female_tm','tertiary_bike_lane_binary','tertiary_no_bike_lane_binary'],axis=1)

In [1379]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

edata00
X_dataf = edata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = edata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\regression\linear_model.py:1738: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 5.14 s


,feature,VIF
13,log_stv_nc_adb,1.460047
2,Distance.to.Park.Center,1.674616
14,log_stl_raw,2.112740
8,pct_male_tm,2.118244
12,HH_density_tm_net,2.234786
4,Distance.to.Forest.Center,2.562482
3,Distance.to.Water.Center,2.766022
9,Retail.Area_tm_net,2.792514
6,University_tm,2.796561
10,Primary_tm_net,2.963624


In [1380]:
vif_data['feature']

1     Distance.to.Retail.Center
2       Distance.to.Park.Center
3      Distance.to.Water.Center
4     Distance.to.Forest.Center
5                  Park.Area_tm
6                 University_tm
7                   Tertiary_tm
8                   pct_male_tm
9            Retail.Area_tm_net
10               Primary_tm_net
11              maxspeed_tm_net
12            HH_density_tm_net
13               log_stv_nc_adb
14                  log_stl_raw
Name: feature, dtype: object

# Bend Variable Selections

In [10]:
be_data=data[data['region']=='Bend']
be_data.head()

,Unnamed: 0,site_id,site_name,type,year,region,travel_pattern,AADBT,stv_adb,stv_c_adb,...,Bike.Share.best_tm,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi,rowIndex
0,1,P001BEN,Colorado Ave.,permanent,2019,Bend,2.0,46.209524,6.561644,2.616438,...,3056.0,1.248328,7.709046,10.644986,28.863905,35.305839,108.195110,0.512552,0.113341,1
1,2,P002BEN,Franklin Ave. Both Sides,permanent,2019,Bend,2.0,220.426625,6.958904,3.068493,...,3056.0,0.000000,0.000000,3.995612,49.828057,65.959899,230.465979,0.282730,0.426931,2
2,3,P003BEN,Galveston Ave. Both Sides,permanent,2019,Bend,3.0,127.953968,13.726027,4.369863,...,3056.0,1.191490,6.758242,27.855287,31.094049,41.330493,98.318063,0.499890,0.010863,3
3,4,P004BEN,Newport Ave. Both Sides,permanent,2019,Bend,1.0,109.378321,7.027397,2.178082,...,3056.0,0.722608,8.910716,26.495791,28.563976,54.080251,118.796144,0.344087,0.010368,4
4,5,P005BEN,Portland Ave. Both Sides,permanent,2019,Bend,2.0,61.295238,5.260274,1.698630,...,3056.0,5.137481,7.813417,12.608750,31.528644,55.880389,182.273542,0.559726,0.005038,5


In [11]:
# Clean the data
an_data=be_data._get_numeric_data()
nan_values = an_data. isna()
nan_columns = nan_values. any()
columns_with_nan = an_data. columns[nan_columns]. tolist()
print(columns_with_nan)

[]


In [12]:
be1_data=an_data.drop(an_data. columns[nan_columns]. tolist(),axis=1)
be1_data=be1_data.drop(['Unnamed: 0','year','latitude','longitude','rowIndex','Bike.Share.Origin_otm',
 'Bike.Share.Destination_otm',
 'Bike.Share.Crossing_otm','Bike.Share.Origin_qm',
 'Bike.Share.Destination_qm',
 'Bike.Share.Crossing_qm','Bike.Share.Origin_tfm',
 'Bike.Share.Destination_tfm',
 'Bike.Share.Crossing_tfm','Bike.Share.Origin_om',
 'Bike.Share.Destination_om',
 'Bike.Share.Crossing_om','Bike.Share.Origin_ohm',
 'Bike.Share.Destination_ohm',
 'Bike.Share.Crossing_ohm','Bike.Share.Origin_tm',
 'Bike.Share.Destination_tm',
 'Bike.Share.Crossing_tm','Bike.Share.Origin_qm_net',
 'Bike.Share.Destination_qm_net',
 'Bike.Share.Crossing_qm_net','Bike.Share.Origin_otm_net',
 'Bike.Share.Destination_otm_net',
 'Bike.Share.Crossing_otm_net','Bike.Share.Origin_hm_net',
 'Bike.Share.Destination_hm_net',
 'Bike.Share.Crossing_hm_net','Bike.Share.Origin_tfm_net',
 'Bike.Share.Destination_tfm_net',
 'Bike.Share.Crossing_tfm_net','Bike.Share.Origin_om_net',
 'Bike.Share.Destination_om_net',
 'Bike.Share.Crossing_om_net','Bike.Share.Origin_ohm_net',
 'Bike.Share.Destination_ohm_net',
 'Bike.Share.Crossing_ohm_net', 'Bike.Share.Origin_tm_net',
 'Bike.Share.Destination_tm_net',
 'Bike.Share.Crossing_tm_net','Bike.Share.best_otm',
 'Bike.Share.best_qm',
 'Bike.Share.best_hm',
 'Bike.Share.best_tfm',
 'Bike.Share.best_om',
 'Bike.Share.best_ohm',
 'Bike.Share.best_tm','Bike.Share.Origin_hm',
 'Bike.Share.Destination_hm',
 'Bike.Share.Crossing_hm','Bike.Share.route_otm_net',
'Bike.Share.route_otm','Bike.Share.route_qm','Bike.Share.route_tfm','Bike.Share.route_qm_net',
'Bike.Share.route_hm','Bike.Share.route_om','Bike.Share.route_hm_net','Bike.Share.route_tfm_net','Bike.Share.route_ohm'],axis=1)

be1_data.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,log_stl_raw,log_stv_stl,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
0,2.0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,...,8.067776,10.090865,1.248328,7.709046,10.644986,28.863905,35.305839,108.195110,0.512552,0.113341
1,2.0,220.426625,6.958904,3.068493,2617,0,1,0,0,0,...,7.870166,9.944457,0.000000,0.000000,3.995612,49.828057,65.959899,230.465979,0.282730,0.426931
2,3.0,127.953968,13.726027,4.369863,3128,0,1,0,0,0,...,8.048469,10.738085,1.191490,6.758242,27.855287,31.094049,41.330493,98.318063,0.499890,0.010863
3,1.0,109.378321,7.027397,2.178082,1998,0,1,0,0,0,...,7.600402,9.683263,0.722608,8.910716,26.495791,28.563976,54.080251,118.796144,0.344087,0.010368
4,2.0,61.295238,5.260274,1.698630,1387,0,0,1,0,0,...,7.235619,9.069843,5.137481,7.813417,12.608750,31.528644,55.880389,182.273542,0.559726,0.005038


In [13]:
be1_data=be1_data.loc[:,~be1_data.columns.str.contains('_net', case=False)] 
be1_data.shape

(63, 400)

In [14]:
corrMatrix1 = be1_data.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.30]
top_corr_features

Index(['AADBT', 'stv_c_adb', 'cycleway_binary', 'Park.Area_otm',
       'Water.Area_otm', 'Cycleway_otm', 'bridge_otm', 'Bicycle.Parking_qm',
       'Bus.Stops_qm', 'bridge_qm', 'pct_at_least_college_education_qm',
       'Intersection_Density_hm', 'Bicycle.Parking_hm', 'School_hm',
       'pct_at_least_college_education_hm', 'Number.of.jobs_hm',
       'Intersection_Density_tfm', 'Bicycle.Parking_tfm',
       'OSM_node_Density_tfm', 'Secondary_tfm', 'Number.of.jobs_tfm',
       'Intersection_Density_om', 'Bicycle.Parking_om', 'OSM_node_Density_om',
       'Secondary_om', 'Number.of.jobs_om', 'Percentage.of.Bike.Commuter_ohm',
       'Secondary_ohm', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'school_college_hm', 'log_stv_c_adb', 'Park_acres_otm'],
      dtype='object')

In [15]:
bedata0=be_data[top_corr_features]
bedata00=bedata0.drop(['stv_c_adb','Percentage.of.Bike.Commuter_ohm','OSM_node_Density_om','Park_acres_otm','OSM_node_Density_tfm'],axis=1)

In [16]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bedata00
X_dataf = bedata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bedata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

Wall time: 155 ms


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,VIF
8,pct_at_least_college_education_qm,1.832130
6,Bus.Stops_qm,1.874325
14,sep_onstreet_binary,1.878948
16,log_stv_c_adb,1.929874
9,Intersection_Density_hm,1.943854
15,school_college_hm,2.108196
1,Park.Area_otm,2.267083
7,bridge_qm,2.416832
4,bridge_otm,2.649892
3,Cycleway_otm,2.886827


In [17]:
vif_data['feature']

1                         Park.Area_otm
2                        Water.Area_otm
3                          Cycleway_otm
4                            bridge_otm
5                    Bicycle.Parking_qm
6                          Bus.Stops_qm
7                             bridge_qm
8     pct_at_least_college_education_qm
9               Intersection_Density_hm
10                   Bicycle.Parking_hm
11                    Number.of.jobs_hm
12                        Secondary_tfm
13                   Bicycle.Parking_om
14                  sep_onstreet_binary
15                    school_college_hm
16                        log_stv_c_adb
Name: feature, dtype: object

# Bend 0.1 mile buffer

In [24]:
be1_data2=be1_data.loc[:,~be1_data.columns.str.contains('_qm', case=False)] 
be1_data3=be1_data2.loc[:,~be1_data2.columns.str.contains('_hm', case=False)] 
be1_data4=be1_data3.loc[:,~be1_data3.columns.str.contains('_tfm', case=False)] 
be1_data5=be1_data4.loc[:,~be1_data4.columns.str.contains('_om', case=False)] 
be1_data6=be1_data5.loc[:,~be1_data5.columns.str.contains('_ohm', case=False)] 
be1_data7=be1_data6.loc[:,~be1_data6.columns.str.contains('_tm', case=False)]
be1_data8=be1_data7.loc[:,~be1_data7.columns.str.contains('_otm', case=False)]
#be1_data8=be1_data7.loc[:,~be1_data7.columns.str.contains('_otm', case=False)]
# keep the data for one-tenth mile buffer only
#be1_data9=be1_data.loc[:,be1_data.columns.str.contains('_otm', case=False)]
#be1_data10=pd.concat([be1_data8, be1_data9.reindex(e1_data8.index)], axis=1)
be1_data7.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,BikeFac_otm,BikeFac_onstreet_otm,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Park_acres_otm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
0,2.0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,...,0.614959,0.614959,2.023089,1.285490,1.598419,8.067776,10.090865,1.248328,0.512552,0.113341
1,2.0,220.426625,6.958904,3.068493,2617,0,1,0,0,0,...,0.439320,0.270844,2.074291,1.403273,1.587276,7.870166,9.944457,0.000000,0.282730,0.426931
2,3.0,127.953968,13.726027,4.369863,3128,0,1,0,0,0,...,0.100906,0.100906,2.689617,1.680802,2.337582,8.048469,10.738085,1.191490,0.499890,0.010863
3,1.0,109.378321,7.027397,2.178082,1998,0,1,0,0,0,...,0.091312,0.091312,2.082860,1.156278,1.766325,7.600402,9.683263,0.722608,0.344087,0.010368
4,2.0,61.295238,5.260274,1.698630,1387,0,0,1,0,0,...,0.328949,0.328949,1.834224,0.992744,1.517683,7.235619,9.069843,5.137481,0.559726,0.005038


In [25]:
corrMatrix1 = be1_data7.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['travel_pattern', 'AADBT', 'stv_adb', 'stv_c_adb', 'cycleway_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_college', 'min_dist_to_university',
       'Industrial.Area_otm', 'Park.Area_otm', 'Water.Area_otm',
       'Cycleway_otm', 'maxspeed_otm', 'bridge_otm',
       'pct_at_least_college_education_otm', 'stv_nc_adb',
       'sep_bikeway_binary', 'sep_onstreet_binary', 'BikeFac_binary',
       'sep_bikeway_otm', 'BikeFac_onstreet_otm', 'log_stv_adb',
       'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl',
       'Park_acres_otm', 'Distance_to_CBD_mi', 'Distance_to_Water_Body_mi'],
      dtype='object

In [27]:
bedata0=be_data[top_corr_features]
bedata00=bedata0.drop(['travel_pattern','Distance.to.Grass', 'Distance.to.Grass.Center','stv_adb'],axis=1)

In [28]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bedata00
X_dataf = bedata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bedata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 280 ms


,feature,VIF
2,Industrial.Area_otm,1.208822
8,pct_at_least_college_education_otm,1.633063
3,Park.Area_otm,1.808753
1,Distance.to.Water.Body,1.885047
12,sep_bikeway_otm,2.326406
4,Water.Area_otm,2.482149
15,log_stl_raw,2.587885
11,BikeFac_binary,2.588386
10,sep_onstreet_binary,2.614267
13,BikeFac_onstreet_otm,2.832954


In [29]:
vif_data['feature']

1                 Distance.to.Water.Body
2                    Industrial.Area_otm
3                          Park.Area_otm
4                         Water.Area_otm
5                           Cycleway_otm
6                           maxspeed_otm
7                             bridge_otm
8     pct_at_least_college_education_otm
9                             stv_nc_adb
10                   sep_onstreet_binary
11                        BikeFac_binary
12                       sep_bikeway_otm
13                  BikeFac_onstreet_otm
14                         log_stv_c_adb
15                           log_stl_raw
Name: feature, dtype: object

# 0.25 mile buffer

In [30]:
be1_data9=be1_data.loc[:,be1_data.columns.str.contains('_qm', case=False)]
be1_data9=pd.concat([be1_data8, be1_data9.reindex(be1_data8.index)], axis=1)
be1_data9.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,employment_density_qm,Number.of.jobs_qm,slope_qm,Temp_avg_qm,Hum_avg_qm,Precipitation_qm,sep_bikeway_qm,BikeFac_qm,BikeFac_onstreet_qm,Park_acres_qm
0,2.0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,...,1636.347447,1020.446745,2.636041,48.478082,61.352877,0.028959,5707.918217,1.797717,1.685536,7.709046
1,2.0,220.426625,6.958904,3.068493,2617,0,1,0,0,0,...,1899.674482,1637.249344,1.868355,48.478082,61.352877,0.028959,889.554467,1.601985,1.433508,0.000000
2,3.0,127.953968,13.726027,4.369863,3128,0,1,0,0,0,...,2303.639755,439.811604,2.321501,48.478082,61.352877,0.028959,1772.157330,0.335636,0.335636,6.758242
3,1.0,109.378321,7.027397,2.178082,1998,0,1,0,0,0,...,2203.832992,2578.449263,3.075359,48.478082,61.352877,0.028959,4166.990400,1.459830,1.459830,8.910716
4,2.0,61.295238,5.260274,1.698630,1387,0,0,1,0,0,...,1708.138331,1436.153915,0.000000,48.478082,61.352877,0.028959,2857.017310,1.815494,1.815494,7.813417


In [31]:
corrMatrix1 = be1_data9.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['travel_pattern', 'AADBT', 'stv_adb', 'stv_c_adb', 'cycleway_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb',
       'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Park.Area_qm', 'Water.Area_qm',
       'Bicycle.Parking_qm', 'Bus.Stops_qm', 'Cycleway_qm', 'bridge_qm',
       'pct_at_least_college_education_qm', 'Park_acres_qm'],
      dtype='object')

In [32]:
bedata0=be_data[top_corr_features]
bedata00=bedata0.drop(['travel_pattern','Distance.to.Grass', 'Distance.to.Grass.Center'],axis=1)

In [33]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bedata00
X_dataf = bedata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bedata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 265 ms


,feature,VIF
11,Bus.Stops_qm,1.498120
8,Park.Area_qm,1.610335
12,Cycleway_qm,1.610829
13,bridge_qm,1.763975
4,BikeFac_binary,2.184858
14,pct_at_least_college_education_qm,2.228575
10,Bicycle.Parking_qm,2.257489
3,sep_onstreet_binary,2.384888
7,Distance_to_Water_Body_mi,2.488783
9,Water.Area_qm,2.553893


In [34]:
vif_data['feature']

1                      Distance.to.Park
2                            stv_nc_adb
3                   sep_onstreet_binary
4                        BikeFac_binary
5                         log_stv_c_adb
6                           log_stl_raw
7             Distance_to_Water_Body_mi
8                          Park.Area_qm
9                         Water.Area_qm
10                   Bicycle.Parking_qm
11                         Bus.Stops_qm
12                          Cycleway_qm
13                            bridge_qm
14    pct_at_least_college_education_qm
Name: feature, dtype: object

# Bend 0.5 mile buffer

In [35]:
be1_data9=be1_data.loc[:,be1_data.columns.str.contains('hm', case=False)]
be1_data10=pd.concat([be1_data8, be1_data9.reindex(be1_data8.index)], axis=1)
be1_data11=be1_data10.loc[:,~be1_data10.columns.str.contains('ohm', case=False)]
be1_data11.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,Number.of.jobs_hm,slope_hm,Temp_avg_hm,Hum_avg_hm,Precipitation_hm,sep_bikeway_hm,BikeFac_hm,BikeFac_onstreet_hm,school_college_hm,Park_acres_hm
0,2.0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,...,4801.473846,1.990498,48.478082,61.352877,0.028959,16141.281360,5.241413,5.097525,3,10.644986
1,2.0,220.426625,6.958904,3.068493,2617,0,1,0,0,0,...,8320.262637,2.202706,48.478082,61.352877,0.028959,4091.843380,5.617990,5.429112,2,3.995612
2,3.0,127.953968,13.726027,4.369863,3128,0,1,0,0,0,...,2960.764681,2.346421,48.478082,61.352877,0.028959,10263.045300,3.231008,3.126321,3,27.855287
3,1.0,109.378321,7.027397,2.178082,1998,0,1,0,0,0,...,6237.743052,2.619904,48.478082,61.352877,0.028959,10591.062550,4.809335,4.766779,3,26.495791
4,2.0,61.295238,5.260274,1.698630,1387,0,0,1,0,0,...,6496.476827,0.000000,48.478082,61.352877,0.028959,7561.538615,5.373341,5.373341,0,12.608750


In [36]:
corrMatrix1 = be1_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['travel_pattern', 'AADBT', 'stv_adb', 'stv_c_adb', 'cycleway_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb',
       'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Intersection_Density_hm',
       'Bicycle.Parking_hm', 'School_hm', 'OSM_node_Density_hm',
       'Secondary_hm', 'bridge_hm', 'pct_at_least_college_education_hm',
       'Number.of.jobs_hm', 'school_college_hm'],
      dtype='object')

In [37]:
bedata0=be_data[top_corr_features]
bedata00=bedata0.drop(['travel_pattern','Distance.to.Grass', 'Distance.to.Grass.Center','OSM_node_Density_hm'],axis=1)

In [38]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bedata00
X_dataf = bedata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bedata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 280 ms


,feature,VIF
2,stv_nc_adb,1.580132
9,School_hm,1.797956
4,BikeFac_binary,2.052207
3,sep_onstreet_binary,2.285513
11,bridge_hm,2.306781
10,Secondary_hm,2.353698
6,Distance_to_Water_Body_mi,2.484093
12,pct_at_least_college_education_hm,2.733601
8,Bicycle.Parking_hm,2.889357
1,Distance.to.Park,3.145090


In [39]:
vif_data['feature']

1                      Distance.to.Park
2                            stv_nc_adb
3                   sep_onstreet_binary
4                        BikeFac_binary
5                           log_stl_raw
6             Distance_to_Water_Body_mi
7               Intersection_Density_hm
8                    Bicycle.Parking_hm
9                             School_hm
10                         Secondary_hm
11                            bridge_hm
12    pct_at_least_college_education_hm
13                    Number.of.jobs_hm
Name: feature, dtype: object

# 0.75 mile buffer

In [40]:
be1_data9=be1_data.loc[:,be1_data.columns.str.contains('tfm', case=False)]
be1_data10=pd.concat([be1_data8, be1_data9.reindex(be1_data8.index)], axis=1)
be1_data11=be1_data10.loc[:,~be1_data10.columns.str.contains('ohm', case=False)]
be1_data11.shape

(63, 105)

In [41]:
corrMatrix1 = be1_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features
#plt.figure(figsize=(18,14))
#plt.title("2019_cor-relation>0.20 with AADBT")
#g = sns.heatmap(be1_data11[top_corr_features].corr(),annot=True,cmap="RdYlGn")

Index(['travel_pattern', 'AADBT', 'stv_adb', 'stv_c_adb', 'cycleway_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb',
       'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Intersection_Density_tfm',
       'Bicycle.Parking_tfm', 'Bus.Stops_tfm', 'School_tfm',
       'OSM_node_Density_tfm', 'Secondary_tfm', 'maxspeed_tfm',
       'pct_at_least_college_education_tfm', 'Number.of.jobs_tfm'],
      dtype='object')

In [42]:
bedata0=be_data[top_corr_features]
bedata00=bedata0.drop(['travel_pattern','Distance.to.Grass', 'Distance.to.Grass.Center','OSM_node_Density_tfm'],axis=1)

In [43]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bedata00
X_dataf = bedata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bedata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 273 ms


,feature,VIF
4,BikeFac_binary,1.846578
10,School_tfm,2.232663
7,Distance_to_Water_Body_mi,2.291634
3,sep_onstreet_binary,2.338135
11,Secondary_tfm,2.500708
9,Bus.Stops_tfm,2.546117
12,pct_at_least_college_education_tfm,2.991499
2,stv_nc_adb,3.870995
1,Distance.to.Industrial.Center,4.108026
5,log_stv_c_adb,4.192475


In [44]:
vif_data['feature']

1          Distance.to.Industrial.Center
2                             stv_nc_adb
3                    sep_onstreet_binary
4                         BikeFac_binary
5                          log_stv_c_adb
6                            log_stl_raw
7              Distance_to_Water_Body_mi
8                    Bicycle.Parking_tfm
9                          Bus.Stops_tfm
10                            School_tfm
11                         Secondary_tfm
12    pct_at_least_college_education_tfm
Name: feature, dtype: object

# Bend 1.0 mile buffer

In [45]:
be1_data9=be1_data.loc[:,be1_data.columns.str.contains('_om', case=False)]
#be1_data10=be1_data.loc[:,be1_data.columns.str.contains('_om_net', case=False)]
be1_data11=pd.concat([be1_data9, be1_data8.reindex(be1_data9.index)], axis=1)
#be1_data12=pd.concat([be1_data8, be1_data11.reindex(be1_data8.index)], axis=1)
be1_data11.head()

,Total.Commuter_om,Bike.Commuter_om,Percentage.of.Bike.Commuter_om,bike.commuter.density_om,Intersection_Density_om,Commercial.Area_om,Industrial.Area_om,Residential_Road_om,Retail.Area_om,Grass.Area_om,...,primary_bike_lane_binary,secondary_bike_lane_binary,tertiary_bike_lane_binary,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Distance_to_CBD_mi,Distance_to_Water_Body_mi
0,3838.240755,177.551513,4.625857,56.516867,208.813002,321006.60190,9.857878e+05,206485.7054,4364639.646,548494.2582,...,0,1,0,2.023089,1.285490,1.598419,8.067776,10.090865,0.512552,0.113341
1,4355.174831,213.919687,4.911851,68.093311,201.491815,0.00000,4.482083e+06,369212.8390,2991600.243,425792.8806,...,0,0,0,2.074291,1.403273,1.587276,7.870166,9.944457,0.282730,0.426931
2,3951.140124,190.656367,4.825351,60.688306,197.672065,321006.60190,1.973648e+05,214142.3334,3446050.246,544347.1726,...,0,0,0,2.689617,1.680802,2.337582,8.048469,10.738085,0.499890,0.010863
3,4397.794652,234.766749,5.338284,74.729192,205.948190,53466.15922,2.696412e+05,247609.5115,1528528.414,287757.6812,...,0,0,0,2.082860,1.156278,1.766325,7.600402,9.683263,0.344087,0.010368
4,4487.555898,229.699490,5.118588,73.116220,197.990377,285515.60530,8.033127e+05,328446.7320,1119307.450,163259.4537,...,0,0,1,1.834224,0.992744,1.517683,7.235619,9.069843,0.559726,0.005038


In [46]:
corrMatrix1 = be1_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['Percentage.of.Bike.Commuter_om', 'Intersection_Density_om',
       'Retail.Area_om', 'Bicycle.Parking_om', 'Bus.Stops_om', 'School_om',
       'OSM_node_Density_om', 'Secondary_om', 'Tertiary_om', 'maxspeed_om',
       'pct_African_American_om', 'pct_at_least_college_education_om',
       'Number.of.jobs_om', 'travel_pattern', 'AADBT', 'stv_adb', 'stv_c_adb',
       'cycleway_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Retail.Area',
       'Distance.to.Retail.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb',
       'log_stl_raw'

In [48]:
bedata0=be_data[top_corr_features]
bedata00=bedata0.drop(['OSM_node_Density_om','travel_pattern'],axis=1)

In [49]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bedata00
X_dataf = bedata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bedata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 403 ms


,feature,VIF
6,pct_African_American_om,1.742783
9,sep_onstreet_binary,1.839042
7,Distance.to.Grass.Center,1.875421
2,Retail.Area_om,1.923182
5,Tertiary_om,2.055816
12,Distance_to_Water_Body_mi,2.084071
10,BikeFac_binary,2.220100
1,Percentage.of.Bike.Commuter_om,2.670754
11,log_stv_c_adb,3.207036
3,Bicycle.Parking_om,3.460343


In [50]:
vif_data['feature']

1     Percentage.of.Bike.Commuter_om
2                     Retail.Area_om
3                 Bicycle.Parking_om
4                          School_om
5                        Tertiary_om
6            pct_African_American_om
7           Distance.to.Grass.Center
8                         stv_nc_adb
9                sep_onstreet_binary
10                    BikeFac_binary
11                     log_stv_c_adb
12         Distance_to_Water_Body_mi
Name: feature, dtype: object

# bend 1.5 mile buffer

In [51]:
be1_data1=be1_data.loc[:,~be1_data.columns.str.contains('_otm', case=False)]
be1_data2=be1_data1.loc[:,~be1_data1.columns.str.contains('_qm', case=False)]
be1_data3=be1_data2.loc[:,~be1_data2.columns.str.contains('_hm', case=False)]
be1_data4=be1_data3.loc[:,~be1_data3.columns.str.contains('_tfm', case=False)]
be1_data5=be1_data4.loc[:,~be1_data4.columns.str.contains('_om', case=False)]
be1_data6=be1_data5.loc[:,~be1_data5.columns.str.contains('_tm', case=False)]
be1_data6.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,BikeFac_ohm,BikeFac_onstreet_ohm,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
0,2.0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,...,52.810656,48.230205,2.023089,1.285490,1.598419,8.067776,10.090865,108.195110,0.512552,0.113341
1,2.0,220.426625,6.958904,3.068493,2617,0,1,0,0,0,...,50.171800,45.062402,2.074291,1.403273,1.587276,7.870166,9.944457,230.465979,0.282730,0.426931
2,3.0,127.953968,13.726027,4.369863,3128,0,1,0,0,0,...,47.404731,42.672877,2.689617,1.680802,2.337582,8.048469,10.738085,98.318063,0.499890,0.010863
3,1.0,109.378321,7.027397,2.178082,1998,0,1,0,0,0,...,47.405753,42.655687,2.082860,1.156278,1.766325,7.600402,9.683263,118.796144,0.344087,0.010368
4,2.0,61.295238,5.260274,1.698630,1387,0,0,1,0,0,...,45.532479,41.272569,1.834224,0.992744,1.517683,7.235619,9.069843,182.273542,0.559726,0.005038


In [52]:
corrMatrix1 = be1_data6.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['travel_pattern', 'AADBT', 'stv_adb', 'stv_c_adb', 'cycleway_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_college', 'min_dist_to_university',
       'Bike.Commuter_ohm', 'Percentage.of.Bike.Commuter_ohm',
       'bike.commuter.density_ohm', 'Intersection_Density_ohm',
       'Retail.Area_ohm', 'Bicycle.Parking_ohm', 'Bus.Stops_ohm', 'School_ohm',
       'University_ohm', 'OSM_node_Density_ohm', 'Secondary_ohm',
       'Tertiary_ohm', 'Cycleway_ohm', 'Footway_ohm', 'cycleway_lane_all_ohm',
       'maxspeed_ohm', 'Median.Age_ohm', 'Number.of.jobs_ohm', 'stv_nc_adb',
       'sep_bikeway_binary', 'sep_onstreet_binary',

In [54]:
bedata0=be_data[top_corr_features]
bedata00=bedata0.drop(['Distance.to.Grass', 'Distance.to.Grass.Center','travel_pattern'],axis=1)

In [55]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bedata00
X_dataf = bedata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bedata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 564 ms


,feature,VIF
12,Distance_to_Water_Body_mi,1.458592
4,University_ohm,1.693577
8,BikeFac_binary,1.870445
7,sep_onstreet_binary,2.390970
9,BikeFac_ohm,3.053320
6,stv_nc_adb,3.629239
2,Percentage.of.Bike.Commuter_ohm,3.660862
1,Distance.to.Industrial.Center,3.700827
10,log_stv_c_adb,3.786821
3,Bicycle.Parking_ohm,4.378713


In [56]:
vif_data['feature']

1       Distance.to.Industrial.Center
2     Percentage.of.Bike.Commuter_ohm
3                 Bicycle.Parking_ohm
4                      University_ohm
5               cycleway_lane_all_ohm
6                          stv_nc_adb
7                 sep_onstreet_binary
8                      BikeFac_binary
9                         BikeFac_ohm
10                      log_stv_c_adb
11                        log_stl_raw
12          Distance_to_Water_Body_mi
Name: feature, dtype: object

# Bend 2.0 mile buffer

In [57]:
be1_data1=be1_data.loc[:,~be1_data.columns.str.contains('_otm', case=False)]
be1_data2=be1_data1.loc[:,~be1_data1.columns.str.contains('_qm', case=False)]
be1_data3=be1_data2.loc[:,~be1_data2.columns.str.contains('_hm', case=False)]
be1_data4=be1_data3.loc[:,~be1_data3.columns.str.contains('_tfm', case=False)]
be1_data5=be1_data4.loc[:,~be1_data4.columns.str.contains('_om', case=False)]
be1_data6=be1_data5.loc[:,~be1_data5.columns.str.contains('_ohm', case=False)]
be1_data6.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,tertiary_bike_lane_binary,BikeFac_tm,BikeFac_onstreet_tm,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Distance_to_CBD_mi,Distance_to_Water_Body_mi
0,2.0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,...,0,85.007762,76.808645,2.023089,1.285490,1.598419,8.067776,10.090865,0.512552,0.113341
1,2.0,220.426625,6.958904,3.068493,2617,0,1,0,0,0,...,0,85.849684,76.675704,2.074291,1.403273,1.587276,7.870166,9.944457,0.282730,0.426931
2,3.0,127.953968,13.726027,4.369863,3128,0,1,0,0,0,...,0,84.479516,75.256963,2.689617,1.680802,2.337582,8.048469,10.738085,0.499890,0.010863
3,1.0,109.378321,7.027397,2.178082,1998,0,1,0,0,0,...,0,82.976764,73.221971,2.082860,1.156278,1.766325,7.600402,9.683263,0.344087,0.010368
4,2.0,61.295238,5.260274,1.698630,1387,0,0,1,0,0,...,1,81.722498,72.903232,1.834224,0.992744,1.517683,7.235619,9.069843,0.559726,0.005038


In [58]:
corrMatrix1 = be1_data6.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['travel_pattern', 'AADBT', 'stv_adb', 'stv_c_adb', 'cycleway_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_college', 'min_dist_to_university',
       'Bike.Commuter_tm', 'Percentage.of.Bike.Commuter_tm',
       'bike.commuter.density_tm', 'Intersection_Density_tm',
       'Residential_Road_tm', 'Retail.Area_tm', 'Bicycle.Parking_tm',
       'School_tm', 'college_tm', 'University_tm', 'OSM_node_Density_tm',
       'Secondary_tm', 'Tertiary_tm', 'Cycleway_tm', 'Footway_tm',
       'cycleway_lane_all_tm', 'maxspeed_tm', 'Number.of.jobs_tm',
       'Bike.Share.route_tm', 'stv_nc_adb', 'sep_bikeway_binary',
       'sep_o

In [59]:
bedata0=be_data[top_corr_features]
bedata00=bedata0.drop(['Distance.to.Grass', 'Distance.to.Grass.Center','travel_pattern'],axis=1)

In [60]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bedata00
X_dataf = bedata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bedata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 593 ms


,feature,VIF
11,Distance_to_Water_Body_mi,1.472922
1,stv_c_adb,1.679356
8,BikeFac_binary,2.008837
7,sep_onstreet_binary,2.150377
6,University_tm,2.435610
2,min_dist_to_university,3.102483
5,college_tm,3.333837
9,BikeFac_tm,3.424969
4,Retail.Area_tm,3.919920
3,Percentage.of.Bike.Commuter_tm,3.932403


In [61]:
vif_data['feature']

1                          stv_c_adb
2             min_dist_to_university
3     Percentage.of.Bike.Commuter_tm
4                     Retail.Area_tm
5                         college_tm
6                      University_tm
7                sep_onstreet_binary
8                     BikeFac_binary
9                         BikeFac_tm
10                       log_stl_raw
11         Distance_to_Water_Body_mi
Name: feature, dtype: object

# Dallas all Buffers

In [75]:
da_data=data[data['region']=='Dallas']
da_data.head()

,Unnamed: 0,site_id,site_name,type,year,region,travel_pattern,AADBT,stv_adb,stv_c_adb,...,Bike.Share.best_tm,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi,rowIndex
116,117,P001DAL,AT&T Trail at Trinity Forest Trail,permanent,2019,Dallas,3.0,11.805952,0.986301,0.013699,...,NaN,0.000000,0.000000,0.000000,25.512511,43.453987,150.472751,6.966430,0.025024,117
117,118,P002DAL,Coombs Creek Trail,permanent,2019,Dallas,3.0,17.944163,5.794521,1.273973,...,NaN,0.217983,2.214549,2.278466,15.979778,59.050920,140.723741,2.805968,0.501749,118
118,119,P003DAL,Cottonwood Creek Trail at Hamilton Park,permanent,2019,Dallas,3.0,63.346667,5.726027,1.383562,...,NaN,16.780093,42.889590,56.251345,72.315602,105.249238,172.150190,10.005480,0.545815,119
119,120,P004DAL,Crawford Memorial Park Loop Trail,permanent,2019,Dallas,2.0,43.761310,0.150685,0.000000,...,NaN,40.133036,197.745903,509.494083,526.322986,559.045079,611.094139,8.688990,1.622354,120
120,121,P006DAL,Gateway Park Loop Trail,permanent,2019,Dallas,3.0,26.274603,0.000000,0.000000,...,NaN,12.408895,62.431443,105.934496,127.755741,134.067319,204.293522,5.762422,0.452393,121


In [76]:
# Clean the data
an_data=da_data._get_numeric_data()
nan_values = an_data. isna()
nan_columns = nan_values. any()
columns_with_nan = an_data. columns[nan_columns]. tolist()
print(columns_with_nan)

['Bike.Share.Origin_otm', 'Bike.Share.Destination_otm', 'Bike.Share.Crossing_otm', 'Bike.Share.route_otm', 'Bike.Share.Origin_qm', 'Bike.Share.Destination_qm', 'Bike.Share.Crossing_qm', 'Bike.Share.route_qm', 'Bike.Share.Origin_hm', 'Bike.Share.Destination_hm', 'Bike.Share.Crossing_hm', 'Bike.Share.route_hm', 'Bike.Share.Origin_tfm', 'Bike.Share.Destination_tfm', 'Bike.Share.Crossing_tfm', 'Bike.Share.route_tfm', 'Bike.Share.Origin_om', 'Bike.Share.Destination_om', 'Bike.Share.Crossing_om', 'Bike.Share.route_om', 'Bike.Share.Origin_ohm', 'Bike.Share.Destination_ohm', 'Bike.Share.Crossing_ohm', 'Bike.Share.route_ohm', 'Bike.Share.Origin_tm', 'Bike.Share.Destination_tm', 'Bike.Share.Crossing_tm', 'Bike.Share.route_tm', 'Bike.Share.Origin_otm_net', 'Bike.Share.Destination_otm_net', 'Bike.Share.Crossing_otm_net', 'Bike.Share.route_otm_net', 'Bike.Share.Origin_qm_net', 'Bike.Share.Destination_qm_net', 'Bike.Share.Crossing_qm_net', 'Bike.Share.route_qm_net', 'Bike.Share.Origin_hm_net', 'Bike

In [77]:
da1_data=an_data.drop(an_data. columns[nan_columns]. tolist(),axis=1)
da1_data=da1_data.drop(['Unnamed: 0','year','latitude','longitude','rowIndex'],axis=1)
da1_data.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,log_stl_raw,log_stv_stl,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
116,3.0,11.805952,0.986301,0.013699,25,0,0,0,0,0,...,3.258097,3.944371,0.000000,0.000000,0.000000,25.512511,43.453987,150.472751,6.966430,0.025024
117,3.0,17.944163,5.794521,1.273973,12,0,0,0,0,1,...,2.564949,4.481066,0.217983,2.214549,2.278466,15.979778,59.050920,140.723741,2.805968,0.501749
118,3.0,63.346667,5.726027,1.383562,220,0,0,0,0,0,...,5.398163,7.304147,16.780093,42.889590,56.251345,72.315602,105.249238,172.150190,10.005480,0.545815
119,2.0,43.761310,0.150685,0.000000,129,0,0,0,0,0,...,4.867534,5.007892,40.133036,197.745903,509.494083,526.322986,559.045079,611.094139,8.688990,1.622354
120,3.0,26.274603,0.000000,0.000000,8,0,0,0,0,0,...,2.197225,2.197225,12.408895,62.431443,105.934496,127.755741,134.067319,204.293522,5.762422,0.452393


In [78]:
da1_data=da1_data.loc[:,~da1_data.columns.str.contains('_net', case=False)] 
da1_data.shape

(31, 399)

In [79]:
corrMatrix1 = da1_data.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.50]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Water.Area_otm',
       'pct_white_otm', 'pct_African_American_otm',
       'pct_at_least_college_education_otm', 'Water.Area_qm', 'pct_white_qm',
       'pct_African_American_qm', 'pct_at_least_college_education_qm',
       'Water.Area_hm', 'pct_white_hm', 'pct_at_least_college_education_hm',
       'Park.Area_tfm', 'Water.Area_tfm', 'pct_white_tfm',
       'pct_at_least_college_education_tfm', 'Park.Area_om', 'Water.Area_om',
       'pct_white_om', 'pct_at_least_college_education_om', 'Park.Area_ohm',
       'Water.Area_ohm', 'pct_white_ohm', 'pct_at_least_college_education_ohm',
       'Median_HH_income_ohm', 'Park.Area_tm', 'Water.Area_tm',
       'pct_at_least_college_education_tm', 'Median_HH_income_tm',
       'stv_nc_adb', 'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb',
       'log_stl_raw', 'log_stv

In [80]:
dadata0=da1_data[top_corr_features]
dadata00=dadata0.drop(['Distance.to.Grass', 'Distance.to.Grass.Center'],axis=1)

In [81]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

dadata00
X_dataf = dadata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = dadata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 350 ms


,feature,VIF
3,pct_white_tfm,1.712513
7,stv_nc_adb,2.340839
8,log_stl_raw,2.660818
1,Water.Area_qm,3.531008
6,pct_at_least_college_education_tm,3.569471
5,Water.Area_tm,3.621919
2,pct_at_least_college_education_qm,3.692662
4,Park.Area_tm,4.437461


In [82]:
vif_data['feature']

1                        Water.Area_qm
2    pct_at_least_college_education_qm
3                        pct_white_tfm
4                         Park.Area_tm
5                        Water.Area_tm
6    pct_at_least_college_education_tm
7                           stv_nc_adb
8                          log_stl_raw
Name: feature, dtype: object

# Dallas 0.1 mile buffer

In [83]:
da1_data2=da1_data.loc[:,~da1_data.columns.str.contains('_qm', case=False)] 
da1_data3=da1_data2.loc[:,~da1_data2.columns.str.contains('_hm', case=False)] 
da1_data4=da1_data3.loc[:,~da1_data3.columns.str.contains('_tfm', case=False)] 
da1_data5=da1_data4.loc[:,~da1_data4.columns.str.contains('_om', case=False)] 
da1_data6=da1_data5.loc[:,~da1_data5.columns.str.contains('_ohm', case=False)] 
da1_data7=da1_data6.loc[:,~da1_data6.columns.str.contains('_tm', case=False)]
da1_data8=da1_data7.loc[:,~da1_data7.columns.str.contains('_otm', case=False)]
da1_data7.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,BikeFac_otm,BikeFac_onstreet_otm,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Park_acres_otm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
116,3.0,11.805952,0.986301,0.013699,25,0,0,0,0,0,...,0.246991,0.246991,0.686274,0.013606,0.679354,3.258097,3.944371,0.000000,6.966430,0.025024
117,3.0,17.944163,5.794521,1.273973,12,0,0,0,0,1,...,0.643486,0.434198,1.916116,0.821528,1.708477,2.564949,4.481066,0.217983,2.805968,0.501749
118,3.0,63.346667,5.726027,1.383562,220,0,0,0,0,0,...,0.222310,0.000000,1.905985,0.868596,1.675687,5.398163,7.304147,16.780093,10.005480,0.545815
119,2.0,43.761310,0.150685,0.000000,129,0,0,0,0,0,...,0.000000,0.000000,0.140357,0.000000,0.140357,4.867534,5.007892,40.133036,8.688990,1.622354
120,3.0,26.274603,0.000000,0.000000,8,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.197225,2.197225,12.408895,5.762422,0.452393


In [84]:
corrMatrix1 = da1_data7.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Water.Body', 'Distance.to.Water.Center',
       'Distance.to.forest', 'Distance.to.Forest.Center',
       'Commercial.Area_otm', 'Retail.Area_otm', 'Grass.Area_otm',
       'Water.Area_otm', 'Tertiary_otm', 'Path_otm', 'Footway_otm',
       'pct_white_otm', 'pct_African_American_otm', 'pct_male_otm',
       'pct_female_otm', 'pct_at_least_college_education_otm',
       'Number.of.jobs_otm', 'slope_otm', 'stv_nc_adb', 'sep_bikeway_binary',
       'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb',

In [89]:
dadata0=da1_data[top_corr_features]
dadata00=dadata0.drop(['stv_c_adb','Distance.to.Grass','tertiary_bike_lane_binary'],axis=1)

In [90]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

dadata00
X_dataf = dadata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = dadata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 312 ms


,feature,VIF
5,Distance.to.Forest.Center,1.808712
8,Path_otm,1.835127
7,Tertiary_otm,1.845334
6,Commercial.Area_otm,1.910026
14,BikeFac_binary,2.024723
3,Distance.to.Park,2.280648
9,Footway_otm,2.405873
4,Distance.to.Water.Center,2.770434
12,slope_otm,2.817781
10,pct_African_American_otm,2.849686


In [91]:
vif_data['feature']

1     Distance.to.Residential.Center
2           Distance.to.Grass.Center
3                   Distance.to.Park
4           Distance.to.Water.Center
5          Distance.to.Forest.Center
6                Commercial.Area_otm
7                       Tertiary_otm
8                           Path_otm
9                        Footway_otm
10          pct_African_American_otm
11                Number.of.jobs_otm
12                         slope_otm
13                        stv_nc_adb
14                    BikeFac_binary
15      tertiary_no_bike_lane_binary
16                       log_stl_raw
Name: feature, dtype: object

# Dallas 0.25 mile buffer

In [92]:
da1_data9=da1_data.loc[:,da1_data.columns.str.contains('_qm', case=False)]
da1_data9=pd.concat([da1_data8, da1_data9.reindex(da1_data8.index)], axis=1)
da1_data9.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,employment_density_qm,Number.of.jobs_qm,slope_qm,Temp_avg_qm,Hum_avg_qm,Precipitation_qm,sep_bikeway_qm,BikeFac_qm,BikeFac_onstreet_qm,Park_acres_qm
116,3.0,11.805952,0.986301,0.013699,25,0,0,0,0,0,...,176.566311,9.867018,4.116102,67.624384,62.316438,0.106521,3272.649196,0.619820,0.619820,0.000000
117,3.0,17.944163,5.794521,1.273973,12,0,0,0,0,1,...,2079.823959,49.313668,3.632782,67.624384,62.316438,0.106521,4158.230920,1.763853,1.224471,2.214549
118,3.0,63.346667,5.726027,1.383562,220,0,0,0,0,0,...,2001.387474,1705.970090,2.300775,67.624384,62.316438,0.106521,1811.048556,0.343002,0.000000,42.889590
119,2.0,43.761310,0.150685,0.000000,129,0,0,0,0,0,...,748.260630,12.728310,2.390345,67.624384,62.316438,0.106521,0.000000,0.000000,0.000000,197.745903
120,3.0,26.274603,0.000000,0.000000,8,0,0,0,0,0,...,1957.084652,17.699832,2.389081,67.624384,62.316438,0.106521,0.000000,0.000000,0.000000,62.431443


In [93]:
corrMatrix1 = da1_data9.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Water.Body', 'Distance.to.Water.Center',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'stv_nc_adb',
       'sep_bikeway_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl',
       'Distance_to_Water_Body_mi', 'Bike.Commuter_qm',
       'bike.commuter.density_qm', 'Commercial.Area_qm', 'Residential_Road_qm',
       'Grass.Area_qm', 'Park.Area_qm', 'Water.Area_qm', 'School_qm',
       'Secondary_qm', 'Tertiary_qm', 'Path_qm', 'pct_white_qm',
      

In [94]:
dadata0=da1_data[top_corr_features]
dadata00=dadata0.drop(['stv_c_adb','log_stv_c_adb','log_stv_nc_adb','stl_raw','log_stv_stl','log_stv_adb','tertiary_no_bike_lane_binary','bike.commuter.density_qm','tertiary_bike_lane_binary'],axis=1)

In [95]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

dadata00
X_dataf = dadata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = dadata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 265 ms


,feature,VIF
12,School_qm,1.387462
7,BikeFac_binary,1.736949
9,Bike.Commuter_qm,1.849356
3,Distance.to.Residential.Area,1.849420
6,stv_nc_adb,2.142844
11,Residential_Road_qm,2.156937
17,slope_qm,2.388692
5,Distance.to.Forest.Center,2.557782
4,Distance.to.Park,2.589939
13,Secondary_qm,2.779376


In [96]:
vif_data['feature']

1     Distance.to.Commercial.Area.Center
2            Distance.to.Industrial.Area
3           Distance.to.Residential.Area
4                       Distance.to.Park
5              Distance.to.Forest.Center
6                             stv_nc_adb
7                         BikeFac_binary
8                            log_stl_raw
9                       Bike.Commuter_qm
10                    Commercial.Area_qm
11                   Residential_Road_qm
12                             School_qm
13                          Secondary_qm
14                               Path_qm
15                          pct_white_qm
16     pct_at_least_college_education_qm
17                              slope_qm
18                         Park_acres_qm
Name: feature, dtype: object

# 0.5 mile buffer

In [97]:
da1_data9=da1_data.loc[:,da1_data.columns.str.contains('hm', case=False)]
da1_data10=pd.concat([da1_data8, da1_data9.reindex(da1_data8.index)], axis=1)
da1_data11=da1_data10.loc[:,~da1_data10.columns.str.contains('ohm', case=False)]
da1_data11.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,Number.of.jobs_hm,slope_hm,Temp_avg_hm,Hum_avg_hm,Precipitation_hm,sep_bikeway_hm,BikeFac_hm,BikeFac_onstreet_hm,school_college_hm,Park_acres_hm
116,3.0,11.805952,0.986301,0.013699,25,0,0,0,0,0,...,37.340708,2.106917,67.624384,62.316438,0.106521,6988.052131,1.323495,1.323495,0,0.000000
117,3.0,17.944163,5.794521,1.273973,12,0,0,0,0,1,...,475.088147,7.634648,67.624384,62.316438,0.106521,23277.761440,6.753795,5.601872,1,2.278466
118,3.0,63.346667,5.726027,1.383562,220,0,0,0,0,0,...,14867.756250,5.457969,67.624384,62.316438,0.106521,4656.252142,0.881866,0.000000,2,56.251345
119,2.0,43.761310,0.150685,0.000000,129,0,0,0,0,0,...,151.652910,2.806963,67.624384,62.316438,0.106521,0.000000,0.000000,0.000000,0,509.494083
120,3.0,26.274603,0.000000,0.000000,8,0,0,0,0,0,...,239.316478,5.420494,67.624384,62.316438,0.106521,58.472531,0.011074,0.000000,0,105.934496


In [98]:
corrMatrix1 = da1_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Water.Body', 'Distance.to.Water.Center',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'stv_nc_adb',
       'sep_bikeway_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl',
       'Distance_to_Water_Body_mi', 'Commercial.Area_hm',
       'Residential_Road_hm', 'Grass.Area_hm', 'Park.Area_hm', 'Water.Area_hm',
       'Forest.Area_hm', 'Secondary_hm', 'Tertiary_hm', 'Path_hm',
       'pct_white_hm', 'pct_African_American_hm', 'pct_male_hm',
       

In [100]:
dadata0=da1_data[top_corr_features]
dadata00=dadata0.drop(['stv_c_adb','log_stv_c_adb','log_stv_nc_adb','stl_raw','log_stv_stl','log_stv_adb','tertiary_no_bike_lane_binary','tertiary_bike_lane_binary'],axis=1)

In [101]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

dadata00
X_dataf = dadata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = dadata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 264 ms


,feature,VIF
7,BikeFac_binary,1.473736
2,Distance.to.Residential.Area,2.173667
16,Median.Age_hm,2.244355
17,Median_HH_income_hm,2.274614
4,Distance.to.Water.Center,2.286393
6,stv_nc_adb,2.329497
9,Residential_Road_hm,2.347237
5,Distance.to.Forest.Center,2.619327
13,pct_white_hm,2.848859
12,Tertiary_hm,2.936567


In [102]:
vif_data['feature']

1           Distance.to.Industrial.Area
2          Distance.to.Residential.Area
3                      Distance.to.Park
4              Distance.to.Water.Center
5             Distance.to.Forest.Center
6                            stv_nc_adb
7                        BikeFac_binary
8                           log_stl_raw
9                   Residential_Road_hm
10                       Forest.Area_hm
11                         Secondary_hm
12                          Tertiary_hm
13                         pct_white_hm
14                        pct_female_hm
15    pct_at_least_college_education_hm
16                        Median.Age_hm
17                  Median_HH_income_hm
Name: feature, dtype: object

# Dallas 0.75 mile buffer

In [103]:
da1_data9=da1_data.loc[:,da1_data.columns.str.contains('tfm', case=False)]
da1_data10=pd.concat([da1_data8, da1_data9.reindex(da1_data8.index)], axis=1)
da1_data11=da1_data10.loc[:,~da1_data10.columns.str.contains('ohm', case=False)]
da1_data11.shape

(31, 105)

In [104]:
corrMatrix1 = da1_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Water.Body', 'Distance.to.Water.Center',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'stv_nc_adb',
       'sep_bikeway_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl',
       'Distance_to_Water_Body_mi', 'Commercial.Area_tfm',
       'Industrial.Area_tfm', 'Grass.Area_tfm', 'Park.Area_tfm',
       'Water.Area_tfm', 'Forest.Area_tfm', 'Bus.Stops_tfm', 'Secondary_tfm',
       'Path_tfm', 'cycleway_lane_all_tfm', 'bridge_tfm', 'pct_white_tfm',


In [105]:
dadata0=da1_data[top_corr_features]
dadata00=dadata0.drop(['stv_c_adb','log_stv_c_adb','log_stv_nc_adb','stl_raw','log_stv_stl','log_stv_adb','tertiary_no_bike_lane_binary','tertiary_bike_lane_binary','cycleway_lane_all_tfm','Grass.Area_tfm'],axis=1)

In [106]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

dadata00
X_dataf = dadata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = dadata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 270 ms


,feature,VIF
6,Distance.to.Forest.Center,1.652064
11,Industrial.Area_tfm,2.114020
13,Bus.Stops_tfm,2.176793
8,BikeFac_binary,2.291651
5,Distance.to.Water.Center,2.436212
17,slope_tfm,2.674832
14,bridge_tfm,2.680249
4,Distance.to.Park,2.707710
2,Distance.to.Commercial.Area.Center,2.815320
7,stv_nc_adb,2.871351


In [107]:
vif_data['feature']

1                        tertiary_binary
2     Distance.to.Commercial.Area.Center
3           Distance.to.Residential.Area
4                       Distance.to.Park
5               Distance.to.Water.Center
6              Distance.to.Forest.Center
7                             stv_nc_adb
8                         BikeFac_binary
9                            log_stl_raw
10                   Commercial.Area_tfm
11                   Industrial.Area_tfm
12                        Water.Area_tfm
13                         Bus.Stops_tfm
14                            bridge_tfm
15              pct_African_American_tfm
16                  Median_HH_income_tfm
17                             slope_tfm
Name: feature, dtype: object

# Dallas 1 mile buffer

In [108]:
da1_data9=da1_data.loc[:,da1_data.columns.str.contains('_om', case=False)]
#be1_data10=be1_data.loc[:,be1_data.columns.str.contains('_om_net', case=False)]
da1_data11=pd.concat([da1_data9, da1_data8.reindex(da1_data9.index)], axis=1)
#be1_data12=pd.concat([be1_data8, be1_data11.reindex(be1_data8.index)], axis=1)
da1_data11.head()

,Total.Commuter_om,Bike.Commuter_om,Percentage.of.Bike.Commuter_om,bike.commuter.density_om,Intersection_Density_om,Commercial.Area_om,Industrial.Area_om,Residential_Road_om,Retail.Area_om,Grass.Area_om,...,primary_bike_lane_binary,secondary_bike_lane_binary,tertiary_bike_lane_binary,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Distance_to_CBD_mi,Distance_to_Water_Body_mi
116,2465.019940,0.000000,0.000000,0.000000,33.422813,0.000000e+00,4.891287e+04,69783.8417,0.000000e+00,3.500120e+06,...,0,0,0,0.686274,0.013606,0.679354,3.258097,3.944371,6.966430,0.025024
117,6127.140212,11.286789,0.184210,3.592726,179.846565,3.128246e+05,6.079316e+04,352116.3439,0.000000e+00,1.059447e+06,...,0,0,0,1.916116,0.821528,1.708477,2.564949,4.481066,2.805968,0.501749
118,9554.793209,35.578643,0.372364,11.325127,110.772751,1.238092e+07,6.671257e+06,956629.2570,4.444337e+06,1.838831e+06,...,0,0,0,1.905985,0.868596,1.675687,5.398163,7.304147,10.005480,0.545815
119,4228.732010,0.000000,0.000000,0.000000,101.860001,9.705156e+05,1.131547e+06,121565.3124,0.000000e+00,1.308147e+05,...,0,0,0,0.140357,0.000000,0.140357,4.867534,5.007892,8.688990,1.622354
120,5116.580963,0.073396,0.001434,0.023363,83.716188,0.000000e+00,0.000000e+00,188817.5410,5.046587e+05,1.115217e+06,...,0,0,0,0.000000,0.000000,0.000000,2.197225,2.197225,5.762422,0.452393


In [109]:
corrMatrix1 = da1_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['Commercial.Area_om', 'Industrial.Area_om', 'Grass.Area_om',
       'Park.Area_om', 'Water.Area_om', 'Forest.Area_om', 'School_om',
       'Secondary_om', 'cycleway_lane_all_om', 'maxspeed_om', 'pct_white_om',
       'pct_African_American_om', 'pct_male_om', 'pct_female_om',
       'pct_at_least_college_education_om', 'Median.Age_om',
       'Median_HH_income_om', 'slope_om', 'Park_acres_om', 'AADBT', 'stv_adb',
       'stv_c_adb', 'stl_raw', 'tertiary_binary', 'path_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Residential.Area', 'Distance.to.Residential.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Water.Body', 'Distance.to.Water.Center',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'stv_nc_adb',
       'sep_bikeway_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
    

In [113]:
dadata0=da1_data[top_corr_features]
dadata00=dadata0.drop(['stv_adb','stv_c_adb','log_stv_c_adb','log_stv_nc_adb','stl_raw','log_stv_stl','log_stv_adb','tertiary_no_bike_lane_binary','tertiary_bike_lane_binary','cycleway_lane_all_om','Grass.Area_om','Distance.to.Residential.Center','tertiary_binary'],axis=1)

In [114]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

dadata00
X_dataf = dadata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = dadata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 249 ms


,feature,VIF
14,BikeFac_binary,1.304818
12,Distance.to.forest,1.403922
11,Distance.to.Water.Center,1.812039
9,Distance.to.Residential.Area,1.938164
3,maxspeed_om,1.972263
13,stv_nc_adb,2.143756
8,Distance.to.Commercial.Area.Center,2.170023
10,Distance.to.Park,2.178964
7,slope_om,2.345891
1,Industrial.Area_om,2.457834


In [115]:
vif_data['feature']

1                     Industrial.Area_om
2                              School_om
3                            maxspeed_om
4                pct_African_American_om
5                          Median.Age_om
6                    Median_HH_income_om
7                               slope_om
8     Distance.to.Commercial.Area.Center
9           Distance.to.Residential.Area
10                      Distance.to.Park
11              Distance.to.Water.Center
12                    Distance.to.forest
13                            stv_nc_adb
14                        BikeFac_binary
15                           log_stl_raw
Name: feature, dtype: object

# Dallas 1.50 mile buffer

In [116]:
da1_data1=da1_data.loc[:,~da1_data.columns.str.contains('_otm', case=False)]
da1_data2=da1_data1.loc[:,~da1_data1.columns.str.contains('_qm', case=False)]
da1_data3=da1_data2.loc[:,~da1_data2.columns.str.contains('_hm', case=False)]
da1_data4=da1_data3.loc[:,~da1_data3.columns.str.contains('_tfm', case=False)]
da1_data5=da1_data4.loc[:,~da1_data4.columns.str.contains('_om', case=False)]
da1_data6=da1_data5.loc[:,~da1_data5.columns.str.contains('_tm', case=False)]
da1_data6.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,BikeFac_ohm,BikeFac_onstreet_ohm,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
116,3.0,11.805952,0.986301,0.013699,25,0,0,0,0,0,...,7.012964,6.975809,0.686274,0.013606,0.679354,3.258097,3.944371,150.472751,6.966430,0.025024
117,3.0,17.944163,5.794521,1.273973,12,0,0,0,0,1,...,18.461246,15.416667,1.916116,0.821528,1.708477,2.564949,4.481066,140.723741,2.805968,0.501749
118,3.0,63.346667,5.726027,1.383562,220,0,0,0,0,0,...,12.089785,5.018198,1.905985,0.868596,1.675687,5.398163,7.304147,172.150190,10.005480,0.545815
119,2.0,43.761310,0.150685,0.000000,129,0,0,0,0,0,...,0.000000,0.000000,0.140357,0.000000,0.140357,4.867534,5.007892,611.094139,8.688990,1.622354
120,3.0,26.274603,0.000000,0.000000,8,0,0,0,0,0,...,0.011074,0.000000,0.000000,0.000000,0.000000,2.197225,2.197225,204.293522,5.762422,0.452393


In [117]:
corrMatrix1 = da1_data6.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Water.Body', 'Distance.to.Water.Center',
       'Distance.to.forest', 'Distance.to.Forest.Center',
       'Commercial.Area_ohm', 'Industrial.Area_ohm', 'Grass.Area_ohm',
       'Park.Area_ohm', 'Water.Area_ohm', 'Forest.Area_ohm', 'School_ohm',
       'University_ohm', 'Secondary_ohm', 'cycleway_lane_all_ohm',
       'maxspeed_ohm', 'pct_white_ohm', 'pct_African_American_ohm',
       'pct_male_ohm', 'pct_female_ohm', 'pct_at_least_college_education_ohm',
       'Median.Age_ohm', 'Median_HH_income_ohm', 'slope_ohm', 'stv_nc_adb',
       'sep_bikeway_binary', 'BikeFac_binary', 'tertiary_

In [118]:
dadata0=da1_data[top_corr_features]
dadata00=dadata0.drop(['stv_adb','stv_c_adb','log_stv_c_adb','log_stv_nc_adb','stl_raw','log_stv_stl','log_stv_adb','tertiary_no_bike_lane_binary','tertiary_bike_lane_binary','cycleway_lane_all_ohm','Grass.Area_ohm'],axis=1)

In [119]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

dadata00
X_dataf = dadata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = dadata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 285 ms


,feature,VIF
14,BikeFac_binary,1.330292
5,Distance.to.Forest.Center,1.520768
4,Distance.to.Water.Center,1.822885
9,maxspeed_ohm,1.892394
10,pct_white_ohm,2.050618
13,stv_nc_adb,2.067145
8,University_ohm,2.202330
2,Distance.to.Residential.Area,2.404357
1,Distance.to.Commercial.Area,2.576291
3,Distance.to.Park,2.643605


In [120]:
vif_data['feature']

1            Distance.to.Commercial.Area
2           Distance.to.Residential.Area
3                       Distance.to.Park
4               Distance.to.Water.Center
5              Distance.to.Forest.Center
6                    Industrial.Area_ohm
7                             School_ohm
8                         University_ohm
9                           maxspeed_ohm
10                         pct_white_ohm
11    pct_at_least_college_education_ohm
12                        Median.Age_ohm
13                            stv_nc_adb
14                        BikeFac_binary
15                           log_stl_raw
Name: feature, dtype: object

# Dallas 2 miles buffer

In [121]:
da1_data1=da1_data.loc[:,~da1_data.columns.str.contains('_otm', case=False)]
da1_data2=da1_data1.loc[:,~da1_data1.columns.str.contains('_qm', case=False)]
da1_data3=da1_data2.loc[:,~da1_data2.columns.str.contains('_hm', case=False)]
da1_data4=da1_data3.loc[:,~da1_data3.columns.str.contains('_tfm', case=False)]
da1_data5=da1_data4.loc[:,~da1_data4.columns.str.contains('_om', case=False)]
da1_data6=da1_data5.loc[:,~da1_data5.columns.str.contains('_ohm', case=False)]
da1_data6.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,tertiary_bike_lane_binary,BikeFac_tm,BikeFac_onstreet_tm,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Distance_to_CBD_mi,Distance_to_Water_Body_mi
116,3.0,11.805952,0.986301,0.013699,25,0,0,0,0,0,...,0,9.313861,9.276706,0.686274,0.013606,0.679354,3.258097,3.944371,6.966430,0.025024
117,3.0,17.944163,5.794521,1.273973,12,0,0,0,0,1,...,0,30.733131,21.188085,1.916116,0.821528,1.708477,2.564949,4.481066,2.805968,0.501749
118,3.0,63.346667,5.726027,1.383562,220,0,0,0,0,0,...,0,21.865746,11.617794,1.905985,0.868596,1.675687,5.398163,7.304147,10.005480,0.545815
119,2.0,43.761310,0.150685,0.000000,129,0,0,0,0,0,...,0,0.034977,0.000000,0.140357,0.000000,0.140357,4.867534,5.007892,8.688990,1.622354
120,3.0,26.274603,0.000000,0.000000,8,0,0,0,0,0,...,0,0.011074,0.000000,0.000000,0.000000,0.000000,2.197225,2.197225,5.762422,0.452393


In [122]:
corrMatrix1 = da1_data6.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Water.Body', 'Distance.to.Water.Center',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'Industrial.Area_tm',
       'Grass.Area_tm', 'Park.Area_tm', 'Water.Area_tm', 'Forest.Area_tm',
       'School_tm', 'University_tm', 'Secondary_tm', 'cycleway_lane_all_tm',
       'maxspeed_tm', 'pct_white_tm', 'pct_African_American_tm', 'pct_male_tm',
       'pct_female_tm', 'pct_at_least_college_education_tm', 'Median.Age_tm',
       'Median_HH_income_tm', 'slope_tm', 'stv_nc_adb', 'sep_bikeway_binary',
       'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane

In [124]:
dadata0=da1_data[top_corr_features]
dadata00=dadata0.drop(['stv_adb','stv_c_adb','log_stv_c_adb','log_stv_nc_adb','stl_raw','log_stv_stl','log_stv_adb','tertiary_no_bike_lane_binary','tertiary_bike_lane_binary','cycleway_lane_all_tm','Grass.Area_tm'],axis=1)

In [125]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

dadata00
X_dataf = dadata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = dadata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 262 ms


,feature,VIF
12,maxspeed_tm,1.326946
14,BikeFac_binary,1.452846
5,Distance.to.Water.Center,1.501707
11,University_tm,1.795493
6,Distance.to.forest,1.897898
13,stv_nc_adb,2.203851
4,Distance.to.Park,2.403923
9,Forest.Area_tm,2.522930
3,Distance.to.Residential.Center,2.685958
15,log_stl_raw,2.914978


In [126]:
vif_data['feature']

1     Distance.to.Commercial.Area.Center
2          Distance.to.Industrial.Center
3         Distance.to.Residential.Center
4                       Distance.to.Park
5               Distance.to.Water.Center
6                     Distance.to.forest
7                     Industrial.Area_tm
8                          Water.Area_tm
9                         Forest.Area_tm
10                             School_tm
11                         University_tm
12                           maxspeed_tm
13                            stv_nc_adb
14                        BikeFac_binary
15                           log_stl_raw
Name: feature, dtype: object

# Boulder All buffer Model

In [22]:
bo_data=data[data['region']=='Boulder']
bo_data.head()

,Unnamed: 0,site_id,site_name,type,year,region,travel_pattern,AADBT,stv_adb,stv_c_adb,...,Bike.Share.best_tm,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi,rowIndex
63,64,P001BOU,B90004 US 36 North of Boulder,permanent,2019,Boulder,3.0,206.745402,102.547945,14.027397,...,NaN,0.000000,0.000000,4.544397,12.165644,53.529893,85.960580,3.691834,0.883893,64
64,65,P002BOU,Folsom at Boulder Creek,permanent,2019,Boulder,2.0,479.543452,29.739726,10.452055,...,NaN,0.000000,0.000000,21.179435,27.822404,48.958050,105.032549,0.903281,0.392384,65
65,66,P003BOU,Folsom at Pine,permanent,2019,Boulder,1.0,607.273809,52.397260,16.424658,...,NaN,0.183242,1.815874,10.706058,23.420727,49.016319,104.389696,1.048750,0.245676,66
66,67,P004BOU,Folsom at South St.,permanent,2019,Boulder,2.0,502.851786,29.904110,9.972603,...,NaN,0.085188,2.151470,6.359348,32.578822,46.306943,99.337266,0.929956,0.572558,67
67,68,P005BOU,Eco-Totem 13th at Walnut,permanent,2019,Boulder,2.0,601.530238,20.808219,9.095890,...,NaN,0.035249,4.509611,7.963230,15.517555,42.385664,121.454864,0.159742,0.398035,68


In [23]:
# Clean the data
an_data=bo_data._get_numeric_data()
nan_values = an_data. isna()
nan_columns = nan_values. any()
columns_with_nan = an_data. columns[nan_columns]. tolist()
print(columns_with_nan)

['travel_pattern', 'Bike.Share.Origin_otm', 'Bike.Share.Destination_otm', 'Bike.Share.Crossing_otm', 'Bike.Share.route_otm', 'Bike.Share.Origin_qm', 'Bike.Share.Destination_qm', 'Bike.Share.Crossing_qm', 'Bike.Share.route_qm', 'Bike.Share.Origin_hm', 'Bike.Share.Destination_hm', 'Bike.Share.Crossing_hm', 'Bike.Share.route_hm', 'Bike.Share.Origin_tfm', 'Bike.Share.Destination_tfm', 'Bike.Share.Crossing_tfm', 'Bike.Share.route_tfm', 'Bike.Share.Origin_om', 'Bike.Share.Destination_om', 'Bike.Share.Crossing_om', 'Bike.Share.route_om', 'Bike.Share.Origin_ohm', 'Bike.Share.Destination_ohm', 'Bike.Share.Crossing_ohm', 'Bike.Share.route_ohm', 'Bike.Share.Origin_tm', 'Bike.Share.Destination_tm', 'Bike.Share.Crossing_tm', 'Bike.Share.route_tm', 'Bike.Share.Origin_otm_net', 'Bike.Share.Destination_otm_net', 'Bike.Share.Crossing_otm_net', 'Bike.Share.route_otm_net', 'Bike.Share.Origin_qm_net', 'Bike.Share.Destination_qm_net', 'Bike.Share.Crossing_qm_net', 'Bike.Share.route_qm_net', 'Bike.Share.Ori

In [24]:
bo1_data=an_data.drop(an_data. columns[nan_columns]. tolist(),axis=1)
bo1_data=bo1_data.drop(['Unnamed: 0','year','latitude','longitude','rowIndex'],axis=1)
bo1_data.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,log_stl_raw,log_stv_stl,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
63,206.745402,102.547945,14.027397,218,1,0,0,0,0,0,...,5.389072,10.029106,0.000000,0.000000,4.544397,12.165644,53.529893,85.960580,3.691834,0.883893
64,479.543452,29.739726,10.452055,5359,0,0,1,0,0,0,...,8.586719,12.012275,0.000000,0.000000,21.179435,27.822404,48.958050,105.032549,0.903281,0.392384
65,607.273809,52.397260,16.424658,4518,0,1,0,0,0,0,...,8.416046,12.393805,0.183242,1.815874,10.706058,23.420727,49.016319,104.389696,1.048750,0.245676
66,502.851786,29.904110,9.972603,3750,0,1,0,0,0,0,...,8.229778,11.660667,0.085188,2.151470,6.359348,32.578822,46.306943,99.337266,0.929956,0.572558
67,601.530238,20.808219,9.095890,2382,0,0,0,1,0,0,...,7.776115,10.858402,0.035249,4.509611,7.963230,15.517555,42.385664,121.454864,0.159742,0.398035


In [25]:
bo1_data=bo1_data.loc[:,~bo1_data.columns.str.contains('_net', case=False)] 
bo1_data.shape

(39, 398)

In [26]:
corrMatrix1 = bo1_data.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.50]
top_corr_features

Index(['AADBT', 'stv_c_adb', 'stl_raw', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Retail.Area',
       'Distance.to.Retail.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park.Center',
       ...
       'BikeFac_onstreet_tfm', 'BikeFac_onstreet_om', 'BikeFac_onstreet_ohm',
       'BikeFac_onstreet_tm', 'school_college_hm', 'log_stv_c_adb',
       'log_stl_raw', 'log_stv_stl', 'Park_acres_ohm', 'Distance_to_CBD_mi'],
      dtype='object', length=218)

In [27]:
bodata0=bo1_data[top_corr_features]
bodata00=bodata0.drop(['Distance.to.Grass', 'Distance.to.Grass.Center','BikeFac_onstreet_otm'],axis=1)

In [28]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bodata00
X_dataf = bodata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bodata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 14.5 s


,feature,VIF
7,school_college_hm,2.085895
5,pct_at_least_college_education_tm,2.516390
1,stl_raw,2.798264
3,Park.Area_tm,3.638451
8,log_stv_c_adb,3.796951
2,Industrial.Area_tm,3.903991
6,Median.Age_tm,3.950135
4,pct_white_tm,4.645327


In [29]:
vif_data['feature']

1                              stl_raw
2                   Industrial.Area_tm
3                         Park.Area_tm
4                         pct_white_tm
5    pct_at_least_college_education_tm
6                        Median.Age_tm
7                    school_college_hm
8                        log_stv_c_adb
Name: feature, dtype: object

# Boulder 0.1 mile buffer

In [30]:
bo1_data2=bo1_data.loc[:,~bo1_data.columns.str.contains('_qm', case=False)] 
bo1_data3=bo1_data2.loc[:,~bo1_data2.columns.str.contains('_hm', case=False)] 
bo1_data4=bo1_data3.loc[:,~bo1_data3.columns.str.contains('_tfm', case=False)] 
bo1_data5=bo1_data4.loc[:,~bo1_data4.columns.str.contains('_om', case=False)] 
bo1_data6=bo1_data5.loc[:,~bo1_data5.columns.str.contains('_ohm', case=False)] 
bo1_data7=bo1_data6.loc[:,~bo1_data6.columns.str.contains('_tm', case=False)]
bo1_data8=bo1_data7.loc[:,~bo1_data7.columns.str.contains('_otm', case=False)]
bo1_data7.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,BikeFac_otm,BikeFac_onstreet_otm,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Park_acres_otm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
63,206.745402,102.547945,14.027397,218,1,0,0,0,0,0,...,0.786453,0.000000,4.640035,2.709875,4.494468,5.389072,10.029106,0.000000,3.691834,0.883893
64,479.543452,29.739726,10.452055,5359,0,0,1,0,0,0,...,0.497695,0.471697,3.425556,2.438169,3.010013,8.586719,12.012275,0.000000,0.903281,0.392384
65,607.273809,52.397260,16.424658,4518,0,1,0,0,0,0,...,0.646729,0.646729,3.977759,2.857886,3.610177,8.416046,12.393805,0.183242,1.048750,0.245676
66,502.851786,29.904110,9.972603,3750,0,1,0,0,0,0,...,0.779142,0.736982,3.430889,2.395402,3.041256,8.229778,11.660667,0.085188,0.929956,0.572558
67,601.530238,20.808219,9.095890,2382,0,0,0,1,0,0,...,0.373082,0.174569,3.082287,2.312128,2.542572,7.776115,10.858402,0.035249,0.159742,0.398035


In [31]:
corrMatrix1 = bo1_data7.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'cycleway_lane_binary', 'cycleway_track_all_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Residential.Area', 'Distance.to.Residential.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_school',
       'min_dist_to_college', 'min_dist_to_university', 'Point.Speed',
       'Point.Bridge', 'Total.Commuter_otm', 'Bike.Commuter_otm',
       'Percentage.of.Bike.Commuter_otm', 'bike.commuter.density_otm',
       'Intersection_Density_otm', 'Residential_Road_otm', 'Retail.Area_otm',
       'Grass.Area_otm', 'Park.Area_otm', 'Forest.Area_otm',
       'Bicycle.Parking_otm', '

In [32]:
bodata0=bo1_data[top_corr_features]
bodata00=bodata0.drop(['Percentage.of.Bike.Commuter_otm','pct_at_least_college_education_otm','pct_female_otm','tertiary_no_bike_lane_binary','cycleway_lane_all_otm','secondary_bike_lane_binary','tertiary_bike_lane_binary','cycleway_track_all_otm','Distance.to.Grass', 'Distance.to.Grass.Center','sep_onstreet_binary','sep_bikeway_otm','stv_nc_adb'],axis=1)

In [33]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bodata00
X_dataf = bodata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bodata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 1.01 s


,feature,VIF
6,Footway_otm,1.508930
2,Point.Speed,1.552668
13,log_stv_nc_adb,1.567601
12,BikeFac_otm,1.639840
4,Primary_otm,1.657284
8,Number.of.jobs_otm,1.906396
9,slope_otm,1.994113
7,pct_male_otm,2.046960
1,min_dist_to_college,2.213474
10,sep_bikeway_binary,2.273378


In [34]:
vif_data['feature']

1      min_dist_to_college
2              Point.Speed
3     Residential_Road_otm
4              Primary_otm
5            Secondary_otm
6              Footway_otm
7             pct_male_otm
8       Number.of.jobs_otm
9                slope_otm
10      sep_bikeway_binary
11          BikeFac_binary
12             BikeFac_otm
13          log_stv_nc_adb
14          Park_acres_otm
Name: feature, dtype: object

# Boulder 0.25 mile buffer

In [35]:
bo1_data9=bo1_data.loc[:,bo1_data.columns.str.contains('_qm', case=False)]
bo1_data9=pd.concat([bo1_data8, bo1_data9.reindex(bo1_data8.index)], axis=1)
bo1_data9.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,employment_density_qm,Number.of.jobs_qm,slope_qm,Temp_avg_qm,Hum_avg_qm,Precipitation_qm,sep_bikeway_qm,BikeFac_qm,BikeFac_onstreet_qm,Park_acres_qm
63,206.745402,102.547945,14.027397,218,1,0,0,0,0,0,...,81.530840,21.827873,3.217897,47.49533,60.606044,0.021209,12869.251370,2.437358,0.000000,0.000000
64,479.543452,29.739726,10.452055,5359,0,0,1,0,0,0,...,6309.599983,1665.253458,1.844764,47.49533,60.606044,0.021209,9465.540674,2.801242,1.501732,0.000000
65,607.273809,52.397260,16.424658,4518,0,1,0,0,0,0,...,5105.803364,1132.579913,1.635591,47.49533,60.606044,0.021209,1318.443150,2.607746,2.454726,1.815874
66,502.851786,29.904110,9.972603,3750,0,1,0,0,0,0,...,7919.241800,2839.898950,1.477742,47.49533,60.606044,0.021209,1082.637253,2.975947,2.770902,2.151470
67,601.530238,20.808219,9.095890,2382,0,0,0,1,0,0,...,2743.084502,12535.304590,2.083201,47.49533,60.606044,0.021209,10853.140420,2.974885,1.543866,4.509611


In [36]:
corrMatrix1 = bo1_data9.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'cycleway_lane_binary', 'cycleway_track_all_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Residential.Area', 'Distance.to.Residential.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_school',
       'min_dist_to_college', 'min_dist_to_university', 'Point.Speed',
       'Point.Bridge', 'stv_nc_adb', 'sep_bikeway_binary',
       'sep_onstreet_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'secondary_bike_lane_binary', 'tertiary_bike_lane_binary',
       'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw',
       'log_stv_stl', 'Distanc

In [37]:
bodata0=bo1_data[top_corr_features]
bodata00=bodata0.drop(['Distance.to.Grass.Center','cycleway_track_all_qm'],axis=1)

In [38]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bodata00
X_dataf = bodata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bodata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 1.55 s


,feature,VIF
16,Park_acres_qm,1.435027
5,University_qm,1.559952
8,cycleway_track_all_qm,2.002882
6,Primary_qm,2.057520
14,slope_qm,2.278048
3,Bicycle.Parking_qm,2.294446
4,School_qm,2.537963
1,log_stv_nc_adb,2.577664
2,Residential_Road_qm,2.743810
7,Secondary_qm,2.825335


In [39]:
vif_data['feature']

1                        log_stv_nc_adb
2                   Residential_Road_qm
3                    Bicycle.Parking_qm
4                             School_qm
5                         University_qm
6                            Primary_qm
7                          Secondary_qm
8                 cycleway_track_all_qm
9                             bridge_qm
10              pct_African_American_qm
11                        pct_female_qm
12    pct_at_least_college_education_qm
13                  Median_HH_income_qm
14                             slope_qm
15                           BikeFac_qm
16                        Park_acres_qm
Name: feature, dtype: object

# Boulder 0.5 mile buffer

In [50]:
bo1_data9=bo1_data.loc[:,bo1_data.columns.str.contains('hm', case=False)]
bo1_data10=pd.concat([bo1_data8, bo1_data9.reindex(bo1_data8.index)], axis=1)
bo1_data11=bo1_data10.loc[:,~bo1_data10.columns.str.contains('ohm', case=False)]
bo1_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,Number.of.jobs_hm,slope_hm,Temp_avg_hm,Hum_avg_hm,Precipitation_hm,sep_bikeway_hm,BikeFac_hm,BikeFac_onstreet_hm,school_college_hm,Park_acres_hm
63,206.745402,102.547945,14.027397,218,1,0,0,0,0,0,...,412.013702,2.983636,47.49533,60.606044,0.021209,35007.19918,7.298870,0.668719,0,4.544397
64,479.543452,29.739726,10.452055,5359,0,0,1,0,0,0,...,5155.731854,2.074227,47.49533,60.606044,0.021209,33618.25806,10.890763,5.771450,0,21.179435
65,607.273809,52.397260,16.424658,4518,0,1,0,0,0,0,...,5790.518700,1.561718,47.49533,60.606044,0.021209,17086.05231,15.414093,13.408015,2,10.706058
66,502.851786,29.904110,9.972603,3750,0,1,0,0,0,0,...,9030.987679,1.437182,47.49533,60.606044,0.021209,17011.86770,12.607496,9.385551,2,6.359348
67,601.530238,20.808219,9.095890,2382,0,0,0,1,0,0,...,18290.047100,2.446765,47.49533,60.606044,0.021209,19604.90471,9.752128,7.577656,6,7.963230


In [51]:
corrMatrix1 = bo1_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'cycleway_lane_binary', 'cycleway_track_all_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Residential.Area', 'Distance.to.Residential.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_school',
       'min_dist_to_college', 'min_dist_to_university', 'Point.Speed',
       'Point.Bridge', 'stv_nc_adb', 'sep_bikeway_binary',
       'sep_onstreet_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'secondary_bike_lane_binary', 'tertiary_bike_lane_binary',
       'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw',
       'log_stv_stl', 'Distanc

In [52]:
bodata0=bo1_data[top_corr_features]
bodata00=bodata0.drop(['Distance.to.Grass', 'Distance.to.Grass.Center','cycleway_track_all_hm'],axis=1)

In [53]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bodata00
X_dataf = bodata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bodata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 1.4 s


,feature,VIF
11,pct_African_American_hm,1.594677
4,Water.Area_hm,1.688707
14,slope_hm,1.935718
10,bridge_hm,2.369987
1,log_stv_nc_adb,2.658010
5,Forest.Area_hm,2.764053
16,Park_acres_hm,3.144570
7,University_hm,3.437641
8,Secondary_hm,3.735473
12,pct_at_least_college_education_hm,3.991760


In [54]:
vif_data['feature']

1                        log_stv_nc_adb
2                   Residential_Road_hm
3                        Retail.Area_hm
4                         Water.Area_hm
5                        Forest.Area_hm
6                    Bicycle.Parking_hm
7                         University_hm
8                          Secondary_hm
9                           Tertiary_hm
10                            bridge_hm
11              pct_African_American_hm
12    pct_at_least_college_education_hm
13                  Median_HH_income_hm
14                             slope_hm
15                    school_college_hm
16                        Park_acres_hm
Name: feature, dtype: object

# Boulder 0.75 mile Boulder Model

In [55]:
bo1_data9=bo1_data.loc[:,bo1_data.columns.str.contains('tfm', case=False)]
bo1_data10=pd.concat([bo1_data8, bo1_data9.reindex(bo1_data8.index)], axis=1)
bo1_data11=bo1_data10.loc[:,~bo1_data10.columns.str.contains('ohm', case=False)]
bo1_data11.shape

(39, 104)

In [56]:
corrMatrix1 = bo1_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'cycleway_lane_binary', 'cycleway_track_all_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Residential.Area', 'Distance.to.Residential.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_school',
       'min_dist_to_college', 'min_dist_to_university', 'Point.Speed',
       'Point.Bridge', 'stv_nc_adb', 'sep_bikeway_binary',
       'sep_onstreet_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'secondary_bike_lane_binary', 'tertiary_bike_lane_binary',
       'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw',
       'log_stv_stl', 'Distanc

In [57]:
bodata0=bo1_data[top_corr_features]
bodata00=bodata0.drop(['Distance.to.Grass', 'Distance.to.Grass.Center','cycleway_track_all_tfm'],axis=1)

In [58]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bodata00
X_dataf = bodata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bodata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 1.52 s


,feature,VIF
10,pct_African_American_tfm,1.223921
3,Water.Area_tfm,1.708142
5,college_tfm,2.241621
9,bridge_tfm,2.249102
1,log_stv_nc_adb,2.290642
11,pct_female_tfm,2.688530
15,Park_acres_tfm,2.810125
14,slope_tfm,2.822956
6,Secondary_tfm,2.960108
7,Tertiary_tfm,3.015586


In [59]:
vif_data['feature']

1                         log_stv_nc_adb
2                   Residential_Road_tfm
3                         Water.Area_tfm
4                        Forest.Area_tfm
5                            college_tfm
6                          Secondary_tfm
7                           Tertiary_tfm
8                           Cycleway_tfm
9                             bridge_tfm
10              pct_African_American_tfm
11                        pct_female_tfm
12    pct_at_least_college_education_tfm
13                  Median_HH_income_tfm
14                             slope_tfm
15                        Park_acres_tfm
Name: feature, dtype: object

# Boulder 1 mile Buffer

In [193]:
bo1_data9=bo1_data.loc[:,bo1_data.columns.str.contains('_om', case=False)]
#be1_data10=be1_data.loc[:,be1_data.columns.str.contains('_om_net', case=False)]
bo1_data11=pd.concat([bo1_data9, bo1_data8.reindex(bo1_data9.index)], axis=1)
#be1_data12=pd.concat([be1_data8, be1_data11.reindex(be1_data8.index)], axis=1)
bo1_data11.head()

,Total.Commuter_om,Bike.Commuter_om,Percentage.of.Bike.Commuter_om,bike.commuter.density_om,Intersection_Density_om,Commercial.Area_om,Industrial.Area_om,Residential_Road_om,Retail.Area_om,Grass.Area_om,...,primary_bike_lane_binary,secondary_bike_lane_binary,tertiary_bike_lane_binary,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Distance_to_CBD_mi,Distance_to_Water_Body_mi
63,1857.862959,88.055666,4.739621,28.029219,55.386376,8.133637e+04,257519.2215,105247.8242,0.000,8.152082e+05,...,0,0,0,4.640035,2.709875,4.494468,5.389072,10.029106,3.691834,0.883893
64,15723.588120,1579.706755,10.046732,502.840412,191.942440,3.249894e+06,0.0000,157027.4368,7481906.117,1.293629e+06,...,0,0,0,3.425556,2.438169,3.010013,8.586719,12.012275,0.903281,0.392384
65,13503.120980,1768.437077,13.096506,562.915634,202.765065,2.134284e+06,0.0000,190615.1409,8382654.775,9.245083e+04,...,0,1,0,3.977759,2.857886,3.610177,8.416046,12.393805,1.048750,0.245676
66,13139.479300,1560.865835,11.879206,496.843112,198.627002,3.542691e+06,0.0000,146591.2964,7794634.399,5.589955e+05,...,0,1,0,3.430889,2.395402,3.041256,8.229778,11.660667,0.929956,0.572558
67,14794.994590,1635.206192,11.052428,520.506577,196.398815,1.191767e+05,0.0000,231031.2740,1810569.391,3.934445e+05,...,0,0,0,3.082287,2.312128,2.542572,7.776115,10.858402,0.159742,0.398035


In [194]:
corrMatrix1 = bo1_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['Total.Commuter_om', 'Bike.Commuter_om',
       'Percentage.of.Bike.Commuter_om', 'bike.commuter.density_om',
       'Intersection_Density_om', 'Commercial.Area_om', 'Residential_Road_om',
       'Retail.Area_om', 'Grass.Area_om', 'Park.Area_om', 'Water.Area_om',
       'Forest.Area_om', 'Bicycle.Parking_om', 'Bus.Stops_om', 'School_om',
       'college_om', 'University_om', 'OSM_node_Density_om', 'Primary_om',
       'Secondary_om', 'Tertiary_om', 'Cycleway_om', 'Footway_om',
       'cycleway_lane_all_om', 'cycleway_track_all_om', 'bridge_om',
       'pct_white_om', 'pct_African_American_om', 'Student.Access_om',
       'pct_at_least_college_education_om', 'Median.Age_om',
       'Median_HH_income_om', 'HH_density_om', 'population_density_om',
       'employment_density_om', 'Number.of.jobs_om', 'slope_om', 'BikeFac_om',
       'BikeFac_onstreet_om', 'Park_acres_om', 'AADBT', 'stv_adb', 'stv_c_adb',
       'stl_raw', 'tertiary_binary', 'cycleway_lane_binary',
       'cycleway_t

In [205]:
bodata0=bo1_data[top_corr_features]
bodata00=bodata0.drop(['Point.Bridge','sep_bikeway_binary','secondary_bike_lane_binary','Distance.to.Grass', 'Distance.to.Grass.Center','cycleway_track_all_binary','tertiary_bike_lane_binary','tertiary_no_bike_lane_binary','sep_onstreet_binary'],axis=1)

In [206]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bodata00
X_dataf = bodata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bodata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 1.23 s


,feature,VIF
14,Point.Speed,1.593214
7,tertiary_binary,1.667497
8,cycleway_lane_binary,1.981655
3,BikeFac_om,2.030592
4,Park_acres_om,2.240936
9,Distance.to.Commercial.Area,2.526305
15,stv_nc_adb,2.736475
6,stl_raw,2.919588
2,slope_om,2.962973
10,Distance.to.Industrial.Center,3.066769


In [207]:
vif_data['feature']

1               Median_HH_income_om
2                          slope_om
3                        BikeFac_om
4                     Park_acres_om
5                         stv_c_adb
6                           stl_raw
7                   tertiary_binary
8              cycleway_lane_binary
9       Distance.to.Commercial.Area
10    Distance.to.Industrial.Center
11          Distance.to.Retail.Area
12               Distance.to.forest
13               min_dist_to_school
14                      Point.Speed
15                       stv_nc_adb
Name: feature, dtype: object

# Boulder 1.5 miles Buffer

In [60]:
bo1_data1=bo1_data.loc[:,~bo1_data.columns.str.contains('_otm', case=False)]
bo1_data2=bo1_data1.loc[:,~bo1_data1.columns.str.contains('_qm', case=False)]
bo1_data3=bo1_data2.loc[:,~bo1_data2.columns.str.contains('_hm', case=False)]
bo1_data4=bo1_data3.loc[:,~bo1_data3.columns.str.contains('_tfm', case=False)]
bo1_data5=bo1_data4.loc[:,~bo1_data4.columns.str.contains('_om', case=False)]
bo1_data6=bo1_data5.loc[:,~bo1_data5.columns.str.contains('_tm', case=False)]
bo1_data6.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,BikeFac_ohm,BikeFac_onstreet_ohm,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
63,206.745402,102.547945,14.027397,218,1,0,0,0,0,0,...,40.843556,6.514619,4.640035,2.709875,4.494468,5.389072,10.029106,85.960580,3.691834,0.883893
64,479.543452,29.739726,10.452055,5359,0,0,1,0,0,0,...,80.718483,55.581171,3.425556,2.438169,3.010013,8.586719,12.012275,105.032549,0.903281,0.392384
65,607.273809,52.397260,16.424658,4518,0,1,0,0,0,0,...,90.331497,58.920963,3.977759,2.857886,3.610177,8.416046,12.393805,104.389696,1.048750,0.245676
66,502.851786,29.904110,9.972603,3750,0,1,0,0,0,0,...,90.401462,59.817759,3.430889,2.395402,3.041256,8.229778,11.660667,99.337266,0.929956,0.572558
67,601.530238,20.808219,9.095890,2382,0,0,0,1,0,0,...,74.707239,44.329796,3.082287,2.312128,2.542572,7.776115,10.858402,121.454864,0.159742,0.398035


In [61]:
corrMatrix1 = bo1_data6.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'cycleway_lane_binary', 'cycleway_track_all_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Residential.Area', 'Distance.to.Residential.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_school',
       'min_dist_to_college', 'min_dist_to_university', 'Point.Speed',
       'Point.Bridge', 'Total.Commuter_ohm', 'Bike.Commuter_ohm',
       'Percentage.of.Bike.Commuter_ohm', 'bike.commuter.density_ohm',
       'Intersection_Density_ohm', 'Commercial.Area_ohm',
       'Industrial.Area_ohm', 'Residential_Road_ohm', 'Retail.Area_ohm',
       'Grass.Area_ohm', 'Park.Area_ohm

In [62]:
bodata0=bo1_data[top_corr_features]
bodata00=bodata0.drop(['Distance.to.Grass', 'Distance.to.Grass.Center','cycleway_track_all_binary','tertiary_bike_lane_binary','tertiary_no_bike_lane_binary','sep_bikeway_binary','BikeFac_ohm','stv_nc_adb'],axis=1)

In [63]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bodata00
X_dataf = bodata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bodata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 1.29 s


,feature,VIF
4,maxspeed_ohm,1.709036
10,sep_onstreet_binary,1.750054
6,pct_African_American_ohm,1.812633
13,log_stv_nc_adb,2.140862
12,secondary_bike_lane_binary,2.466597
1,Commercial.Area_ohm,2.948145
7,pct_at_least_college_education_ohm,3.017191
8,Median_HH_income_ohm,3.131795
2,Water.Area_ohm,3.267349
3,Tertiary_ohm,3.314963


In [64]:
vif_data['feature']

1                    Commercial.Area_ohm
2                         Water.Area_ohm
3                           Tertiary_ohm
4                           maxspeed_ohm
5                          pct_white_ohm
6               pct_African_American_ohm
7     pct_at_least_college_education_ohm
8                   Median_HH_income_ohm
9                              slope_ohm
10                   sep_onstreet_binary
11                        BikeFac_binary
12            secondary_bike_lane_binary
13                        log_stv_nc_adb
14                        Park_acres_ohm
Name: feature, dtype: object

# Boulder 2 miles Buffer

In [213]:
bo1_data1=bo1_data.loc[:,~bo1_data.columns.str.contains('_otm', case=False)]
bo1_data2=bo1_data1.loc[:,~bo1_data1.columns.str.contains('_qm', case=False)]
bo1_data3=bo1_data2.loc[:,~bo1_data2.columns.str.contains('_hm', case=False)]
bo1_data4=bo1_data3.loc[:,~bo1_data3.columns.str.contains('_tfm', case=False)]
bo1_data5=bo1_data4.loc[:,~bo1_data4.columns.str.contains('_om', case=False)]
bo1_data6=bo1_data5.loc[:,~bo1_data5.columns.str.contains('_ohm', case=False)]
bo1_data6.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,tertiary_bike_lane_binary,BikeFac_tm,BikeFac_onstreet_tm,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Distance_to_CBD_mi,Distance_to_Water_Body_mi
63,206.745402,102.547945,14.027397,218,1,0,0,0,0,0,...,0,54.112700,9.845608,4.640035,2.709875,4.494468,5.389072,10.029106,3.691834,0.883893
64,479.543452,29.739726,10.452055,5359,0,0,1,0,0,0,...,0,138.808788,75.010821,3.425556,2.438169,3.010013,8.586719,12.012275,0.903281,0.392384
65,607.273809,52.397260,16.424658,4518,0,1,0,0,0,0,...,0,143.322249,85.646262,3.977759,2.857886,3.610177,8.416046,12.393805,1.048750,0.245676
66,502.851786,29.904110,9.972603,3750,0,1,0,0,0,0,...,0,143.861848,83.710853,3.430889,2.395402,3.041256,8.229778,11.660667,0.929956,0.572558
67,601.530238,20.808219,9.095890,2382,0,0,0,1,0,0,...,0,117.554770,60.384953,3.082287,2.312128,2.542572,7.776115,10.858402,0.159742,0.398035


In [214]:
corrMatrix1 = bo1_data6.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'cycleway_lane_binary', 'cycleway_track_all_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Residential.Area', 'Distance.to.Residential.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_school',
       'min_dist_to_college', 'min_dist_to_university', 'Point.Speed',
       'Point.Bridge', 'Total.Commuter_tm', 'Bike.Commuter_tm',
       'Percentage.of.Bike.Commuter_tm', 'bike.commuter.density_tm',
       'Intersection_Density_tm', 'Commercial.Area_tm', 'Industrial.Area_tm',
       'Residential_Road_tm', 'Retail.Area_tm', 'Grass.Area_tm',
       'Park.Area_tm', 'Water.A

In [221]:
bodata0=bo1_data[top_corr_features]
bodata00=bodata0.drop(['pct_at_least_college_education_tm','secondary_bike_lane_binary','Distance.to.Grass', 'Distance.to.Grass.Center','cycleway_track_all_binary','tertiary_bike_lane_binary','tertiary_no_bike_lane_binary','sep_bikeway_binary','BikeFac_tm','stv_nc_adb'],axis=1)

In [222]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bodata00
X_dataf = bodata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bodata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 1.24 s


,feature,VIF
11,sep_bikeway_tm,1.780458
12,log_stv_nc_adb,1.799638
9,sep_onstreet_binary,1.921294
5,maxspeed_tm,2.070121
6,pct_African_American_tm,2.129924
10,BikeFac_binary,2.583500
3,Water.Area_tm,2.593783
1,Commercial.Area_tm,2.692472
2,Industrial.Area_tm,3.402045
8,slope_tm,3.560997


In [223]:
vif_data['feature']

1          Commercial.Area_tm
2          Industrial.Area_tm
3               Water.Area_tm
4                 Tertiary_tm
5                 maxspeed_tm
6     pct_African_American_tm
7               Median.Age_tm
8                    slope_tm
9         sep_onstreet_binary
10             BikeFac_binary
11             sep_bikeway_tm
12             log_stv_nc_adb
Name: feature, dtype: object

# Charlotte All Buffers

In [229]:
ch_data=data[data['region']=='Charlotte']
ch_data.head()

,Unnamed: 0,site_id,site_name,type,year,region,travel_pattern,AADBT,stv_adb,stv_c_adb,...,Bike.Share.best_tm,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi,rowIndex
102,103,P001CHA,Blue Line Trail,permanent,2019,Charlotte,3.0,393.654456,18.547945,5.958904,...,NaN,0.0000,0.000000,0.041610,33.852053,85.171864,198.519985,1.556811,0.766116,103
103,104,P002CHA,Pecan Ave,permanent,2019,Charlotte,3.0,160.742460,16.082192,4.602740,...,NaN,0.0000,0.000000,2.249798,39.927741,81.717367,197.069588,1.371700,0.347870,104
104,105,P003CHA,Selwyn Avenue,permanent,2019,Charlotte,3.0,39.364881,15.465753,2.945205,...,NaN,0.0000,0.000000,4.910771,29.877785,80.588010,136.432429,3.008143,0.482522,105
105,106,P004CHA,South Tryon Street,permanent,2019,Charlotte,1.0,149.292520,11.630137,4.780822,...,NaN,0.0000,0.617506,1.755423,8.325280,69.409236,156.062736,0.690422,0.370499,106
106,107,P005CHA,Stonewall Street,permanent,2019,Charlotte,2.0,33.307341,2.438356,1.136986,...,NaN,0.5032,17.150246,41.189231,77.882806,112.212650,180.740409,0.509815,0.247150,107


In [230]:
# Clean the data
an_data=ch_data._get_numeric_data()
nan_values = an_data. isna()
nan_columns = nan_values. any()
columns_with_nan = an_data. columns[nan_columns]. tolist()
print(columns_with_nan)

['Bike.Share.Origin_otm', 'Bike.Share.Destination_otm', 'Bike.Share.Crossing_otm', 'Bike.Share.route_otm', 'Bike.Share.Origin_qm', 'Bike.Share.Destination_qm', 'Bike.Share.Crossing_qm', 'Bike.Share.route_qm', 'Bike.Share.Origin_hm', 'Bike.Share.Destination_hm', 'Bike.Share.Crossing_hm', 'Bike.Share.route_hm', 'Bike.Share.Origin_tfm', 'Bike.Share.Destination_tfm', 'Bike.Share.Crossing_tfm', 'Bike.Share.route_tfm', 'Bike.Share.Origin_om', 'Bike.Share.Destination_om', 'Bike.Share.Crossing_om', 'Bike.Share.route_om', 'Bike.Share.Origin_ohm', 'Bike.Share.Destination_ohm', 'Bike.Share.Crossing_ohm', 'Bike.Share.route_ohm', 'Bike.Share.Origin_tm', 'Bike.Share.Destination_tm', 'Bike.Share.Crossing_tm', 'Bike.Share.route_tm', 'Bike.Share.Origin_otm_net', 'Bike.Share.Destination_otm_net', 'Bike.Share.Crossing_otm_net', 'Bike.Share.route_otm_net', 'Bike.Share.Origin_qm_net', 'Bike.Share.Destination_qm_net', 'Bike.Share.Crossing_qm_net', 'Bike.Share.route_qm_net', 'Bike.Share.Origin_hm_net', 'Bike

In [231]:
ch1_data=an_data.drop(an_data. columns[nan_columns]. tolist(),axis=1)
ch1_data=ch1_data.drop(['Unnamed: 0','year','latitude','longitude','rowIndex'],axis=1)
ch1_data.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,log_stl_raw,log_stv_stl,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
102,3.0,393.654456,18.547945,5.958904,1236,0,0,0,0,0,...,7.120444,10.093315,0.0000,0.000000,0.041610,33.852053,85.171864,198.519985,1.556811,0.766116
103,3.0,160.742460,16.082192,4.602740,740,0,0,1,0,0,...,6.608001,9.446037,0.0000,0.000000,2.249798,39.927741,81.717367,197.069588,1.371700,0.347870
104,3.0,39.364881,15.465753,2.945205,328,0,0,0,0,0,...,5.796058,8.597340,0.0000,0.000000,4.910771,29.877785,80.588010,136.432429,3.008143,0.482522
105,1.0,149.292520,11.630137,4.780822,2049,0,1,0,0,0,...,7.625595,10.161681,0.0000,0.617506,1.755423,8.325280,69.409236,156.062736,0.690422,0.370499
106,2.0,33.307341,2.438356,1.136986,1443,0,1,0,0,0,...,7.275172,8.510166,0.5032,17.150246,41.189231,77.882806,112.212650,180.740409,0.509815,0.247150


In [232]:
ch1_data=ch1_data.loc[:,~ch1_data.columns.str.contains('_net', case=False)] 
ch1_data.shape

(14, 399)

In [233]:
corrMatrix1 = ch1_data.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.40]
top_corr_features

Index(['travel_pattern', 'AADBT', 'secondary_binary', 'cycleway_binary',
       'Distance.to.Park', 'Distance.to.forest', 'Distance.to.Forest.Center',
       'min_dist_to_university', 'Total.Commuter_otm', 'Bike.Commuter_otm',
       'Percentage.of.Bike.Commuter_otm', 'bike.commuter.density_otm',
       'Intersection_Density_otm', 'Park.Area_otm', 'Bicycle.Parking_otm',
       'OSM_node_Density_otm', 'Secondary_otm', 'Path_otm',
       'cycleway_lane_all_otm', 'cycleway_track_all_otm', 'pct_white_otm',
       'pct_African_American_otm', 'pct_male_otm', 'pct_female_otm',
       'Median.Age_otm', 'HH_density_otm', 'population_density_otm',
       'employment_density_otm', 'Total.Commuter_qm', 'Bike.Commuter_qm',
       'bike.commuter.density_qm', 'Intersection_Density_qm', 'Park.Area_qm',
       'Water.Area_qm', 'OSM_node_Density_qm', 'Path_qm', 'Footway_qm',
       'cycleway_track_all_qm', 'maxspeed_qm', 'Median.Age_qm',
       'HH_density_qm', 'population_density_qm', 'employment_densi

In [235]:
chdata0=ch1_data[top_corr_features]
chdata00=chdata0.drop(['travel_pattern','secondary_no_bike_lane_binary','arterial_no_bike_lane_binary'],axis=1)

In [236]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

chdata00
X_dataf = chdata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = chdata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 1.23 s


,feature,VIF
6,log_stv_nc_adb,1.248546
5,arterial_binary,1.780787
1,Residential_Road_tm,2.250857
4,sep_bikeway_otm,3.144930
7,Park_acres_otm,3.320255
2,slope_tm,3.339120
3,BikeFac_binary,3.486780
8,Park_acres_om,3.953527


In [237]:
vif_data['feature']

1    Residential_Road_tm
2               slope_tm
3         BikeFac_binary
4        sep_bikeway_otm
5        arterial_binary
6         log_stv_nc_adb
7         Park_acres_otm
8          Park_acres_om
Name: feature, dtype: object

# Charlotte 0.1 mile buffer

In [238]:
ch1_data2=ch1_data.loc[:,~ch1_data.columns.str.contains('_qm', case=False)] 
ch1_data3=ch1_data2.loc[:,~ch1_data2.columns.str.contains('_hm', case=False)] 
ch1_data4=ch1_data3.loc[:,~ch1_data3.columns.str.contains('_tfm', case=False)] 
ch1_data5=ch1_data4.loc[:,~ch1_data4.columns.str.contains('_om', case=False)] 
ch1_data6=ch1_data5.loc[:,~ch1_data5.columns.str.contains('_ohm', case=False)] 
ch1_data7=ch1_data6.loc[:,~ch1_data6.columns.str.contains('_tm', case=False)]
ch1_data8=ch1_data7.loc[:,~ch1_data7.columns.str.contains('_otm', case=False)]
ch1_data7.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,BikeFac_otm,BikeFac_onstreet_otm,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Park_acres_otm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
102,3.0,393.654456,18.547945,5.958904,1236,0,0,0,0,0,...,0.214234,0.214234,2.972870,1.940022,2.609264,7.120444,10.093315,0.0000,1.556811,0.766116
103,3.0,160.742460,16.082192,4.602740,740,0,0,1,0,0,...,0.000000,0.000000,2.838037,1.723256,2.524083,6.608001,9.446037,0.0000,1.371700,0.347870
104,3.0,39.364881,15.465753,2.945205,328,0,0,0,0,0,...,0.399539,0.399539,2.801283,1.372501,2.604211,5.796058,8.597340,0.0000,3.008143,0.482522
105,1.0,149.292520,11.630137,4.780822,2049,0,1,0,0,0,...,0.000000,0.000000,2.536086,1.754546,2.060426,7.625595,10.161681,0.0000,0.690422,0.370499
106,2.0,33.307341,2.438356,1.136986,1443,0,1,0,0,0,...,0.314059,0.314059,1.234993,0.759397,0.833505,7.275172,8.510166,0.5032,0.509815,0.247150


In [239]:
corrMatrix1 = ch1_data7.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['travel_pattern', 'AADBT', 'stv_adb', 'secondary_binary',
       'cycleway_binary', 'cycleway_track_all_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_university', 'Point.Bridge',
       'Total.Commuter_otm', 'Bike.Commuter_otm',
       'Percentage.of.Bike.Commuter_otm', 'bike.commuter.density_otm',
       'Intersection_Density_otm', 'Residential_Road_otm', 'Grass.Area_otm',
       'Park.Area_otm', 'Water.Area_otm', 'Forest.Area_otm',
       'Bicycle.Parking_otm', 'OSM_node_Density_otm', 'Secondary_otm',
       'Tertiary_otm', 'Path_otm', 'Cycleway_otm', 'Footway_otm',
       'cycleway_lane_all_otm', 'cycleway_track_all_otm', 'maxspeed_otm',
       'bridge_otm', 'pct_white_otm', 'pct_African

In [241]:
chdata0=ch1_data[top_corr_features]
chdata00=chdata0.drop(['travel_pattern','Distance.to.Grass', 'Distance.to.Grass.Center','Percentage.of.Bike.Commuter_otm', 'bike.commuter.density_otm', 'cycleway_lane_all_otm', 'cycleway_track_all_otm',
       'arterial_no_bike_lane_binary', 'secondary_no_bike_lane_binary'],axis=1)

In [242]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

chdata00
X_dataf = chdata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = chdata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 571 ms


,feature,VIF
2,slope_otm,1.134654
4,BikeFac_onstreet_otm,1.335895
5,log_stv_stl,2.213263
6,Park_acres_otm,2.310949
1,Number.of.jobs_otm,2.461839
8,Distance_to_Water_Body_mi,3.086499
3,arterial_binary,3.209696
7,Distance_to_CBD_mi,3.343849


In [243]:
vif_data['feature']

1           Number.of.jobs_otm
2                    slope_otm
3              arterial_binary
4         BikeFac_onstreet_otm
5                  log_stv_stl
6               Park_acres_otm
7           Distance_to_CBD_mi
8    Distance_to_Water_Body_mi
Name: feature, dtype: object

# Charlotte 0.25 mile buffer

In [254]:
ch1_data9=ch1_data.loc[:,ch1_data.columns.str.contains('_qm', case=False)]
ch1_data9=pd.concat([ch1_data8, ch1_data9.reindex(ch1_data8.index)], axis=1)
ch1_data9.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,employment_density_qm,Number.of.jobs_qm,slope_qm,Temp_avg_qm,Hum_avg_qm,Precipitation_qm,sep_bikeway_qm,BikeFac_qm,BikeFac_onstreet_qm,Park_acres_qm
102,3.0,393.654456,18.547945,5.958904,1236,0,0,0,0,0,...,3840.927050,3287.110707,2.329721,63.593151,65.583288,0.148247,2722.681115,0.515659,0.515659,0.000000
103,3.0,160.742460,16.082192,4.602740,740,0,0,1,0,0,...,3563.368551,1845.948921,2.045256,63.593151,65.583288,0.148247,335.260062,0.063496,0.063496,0.000000
104,3.0,39.364881,15.465753,2.945205,328,0,0,0,0,0,...,2192.028633,255.161299,0.000000,63.593151,65.583288,0.148247,12389.663170,3.262960,3.241485,0.000000
105,1.0,149.292520,11.630137,4.780822,2049,0,1,0,0,0,...,4786.541100,9939.205036,0.000000,63.593151,65.583288,0.148247,2260.046710,0.460286,0.460286,0.617506
106,2.0,33.307341,2.438356,1.136986,1443,0,1,0,0,0,...,1316.185512,2295.090523,2.826991,63.593151,65.583288,0.148247,2336.101663,1.269365,1.269365,17.150246


In [255]:
corrMatrix1 = ch1_data9.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['travel_pattern', 'AADBT', 'stv_adb', 'secondary_binary',
       'cycleway_binary', 'cycleway_track_all_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_university', 'Point.Bridge',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'arterial_binary', 'arterial_no_bike_lane_binary',
       'secondary_no_bike_lane_binary', 'log_stv_adb', 'log_stv_nc_adb',
       'log_stv_stl', 'Distance_to_CBD_mi', 'Distance_to_Water_Body_mi',
       'Total.Commuter_qm', 'Bike.Commuter_qm', 'bike.commuter.density_qm',
       'Intersection_Density_qm', 'Grass.Area_qm', 'Park.Area_qm',
       'Water.Area_qm', 'Forest.Area_qm', 'Bus.Stops_qm', 'college_qm',
       'OSM_node_

In [256]:
chdata0=ch1_data[top_corr_features]
chdata00=chdata0.drop(['travel_pattern','Distance.to.Grass', 'Distance.to.Grass.Center','cycleway_track_all_binary','arterial_no_bike_lane_binary',
       'secondary_no_bike_lane_binary'],axis=1)

In [257]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

chdata00
X_dataf = chdata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = chdata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 521 ms


,feature,VIF
5,Median.Age_qm,1.326732
4,Student.Access_qm,1.421798
8,BikeFac_onstreet_qm,1.495212
7,slope_qm,1.740493
2,pct_African_American_qm,1.790427
6,employment_density_qm,2.127947
3,pct_female_qm,2.277449
9,Park_acres_qm,2.380486
1,bridge_qm,2.468559


In [253]:
vif_data['feature']

1                  bridge_qm
2    pct_African_American_qm
3              pct_female_qm
4          Student.Access_qm
5              Median.Age_qm
6      employment_density_qm
7                   slope_qm
8        BikeFac_onstreet_qm
9              Park_acres_qm
Name: feature, dtype: object

# Charlotte 0.50 mile buffer

In [31]:
ch1_data9=ch1_data.loc[:,ch1_data.columns.str.contains('hm', case=False)]
ch1_data10=pd.concat([ch1_data8, ch1_data9.reindex(ch1_data8.index)], axis=1)
ch1_data11=ch1_data10.loc[:,~ch1_data10.columns.str.contains('ohm', case=False)]
ch1_data11.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,Temp_avg_hm_net,Hum_avg_hm_net,Precipitation_hm_net,sep_bikeway_hm,BikeFac_hm,BikeFac_onstreet_hm,sep_bikeway_hm_net,BikeFac_hm_net,school_college_hm,Park_acres_hm
102,3.0,393.654456,18.547945,5.958904,1236,0,0,0,0,0,...,63.593151,65.583288,0.148247,5835.840189,1.110100,1.110100,5089.053649,0.963836,1,0.041610
103,3.0,160.742460,16.082192,4.602740,740,0,0,1,0,0,...,63.593151,65.583288,0.148247,13271.965720,2.550605,2.550605,2583.938599,0.489382,2,2.249798
104,3.0,39.364881,15.465753,2.945205,328,0,0,0,0,0,...,63.593151,65.583288,0.148247,63359.579250,13.384398,13.185903,19805.975640,5.135609,0,4.910771
105,1.0,149.292520,11.630137,4.780822,2049,0,1,0,0,0,...,63.593151,65.583288,0.148247,8359.182301,2.707669,2.707669,6110.769132,1.816037,1,1.755423
106,2.0,33.307341,2.438356,1.136986,1443,0,1,0,0,0,...,63.593151,65.583288,0.148247,17349.073320,4.363642,4.363642,14287.398580,3.559167,3,41.189231


In [32]:
corrMatrix1 = ch1_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['travel_pattern', 'AADBT', 'stv_adb', 'secondary_binary',
       'cycleway_binary', 'cycleway_track_all_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_university', 'Point.Bridge',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'arterial_binary', 'arterial_no_bike_lane_binary',
       'secondary_no_bike_lane_binary', 'log_stv_adb', 'log_stv_nc_adb',
       'log_stv_stl', 'Distance_to_CBD_mi', 'Distance_to_Water_Body_mi',
       'Total.Commuter_hm', 'Bike.Commuter_hm',
       'Percentage.of.Bike.Commuter_hm', 'bike.commuter.density_hm',
       'Intersection_Density_hm', 'Residential_Road_hm', 'Grass.Area_hm',
       'Park.Area_hm', 'Forest.Area_hm', 

In [33]:
chdata0=ch1_data[top_corr_features]
chdata00=chdata0.drop(['Distance.to.Grass', 'Distance.to.Grass.Center','arterial_no_bike_lane_binary',
       'secondary_no_bike_lane_binary','OSM_node_Density_hm_net'],axis=1)

In [34]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

chdata00
X_dataf = chdata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = chdata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\regression\linear_model.py:1738: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 2.58 s


,feature,VIF
1,log_stv_stl,1.161723
2,population_density_hm_net,1.369138
6,Park_acres_hm,1.414585
3,slope_hm_net,1.431063
4,BikeFac_onstreet_hm,1.645257
5,school_college_hm,2.140957


In [36]:
vif_data['feature']

1                  log_stv_stl
2    population_density_hm_net
3                 slope_hm_net
4          BikeFac_onstreet_hm
5            school_college_hm
6                Park_acres_hm
Name: feature, dtype: object

# Charlotte 0.75 mile buffer

In [42]:
ch1_data9=ch1_data.loc[:,ch1_data.columns.str.contains('tfm', case=False)]
ch1_data10=pd.concat([ch1_data8, ch1_data9.reindex(ch1_data8.index)], axis=1)
ch1_data11=ch1_data10.loc[:,~ch1_data10.columns.str.contains('ohm', case=False)]
ch1_data11.shape

(14, 152)

In [43]:
corrMatrix1 = ch1_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['travel_pattern', 'AADBT', 'stv_adb', 'secondary_binary',
       'cycleway_binary', 'cycleway_track_all_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_university', 'Point.Bridge',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'arterial_binary', 'arterial_no_bike_lane_binary',
       'secondary_no_bike_lane_binary', 'log_stv_adb', 'log_stv_nc_adb',
       'log_stv_stl', 'Distance_to_CBD_mi', 'Distance_to_Water_Body_mi',
       'Total.Commuter_tfm', 'Intersection_Density_tfm', 'Park.Area_tfm',
       'Forest.Area_tfm', 'Bicycle.Parking_tfm', 'Bus.Stops_tfm', 'School_tfm',
       'college_tfm', 'University_tfm', 'OSM_node_Density_tfm', 'Primary_tfm',

In [53]:
chdata0=ch1_data[top_corr_features]
chdata00=chdata0.drop(['travel_pattern','Distance.to.Grass', 'Distance.to.Grass.Center','arterial_no_bike_lane_binary',
       'secondary_no_bike_lane_binary','OSM_node_Density_tfm','Grass.Area_tfm_net','cycleway_lane_all_tfm_net',
       'cycleway_track_all_tfm_net','Percentage.of.Bike.Commuter_tfm_net','cycleway_track_all_binary'],axis=1)

In [54]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

chdata00
X_dataf = chdata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = chdata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\regression\linear_model.py:1738: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 2.19 s


,feature,VIF
4,slope_tfm_net,1.122747
1,log_stv_stl,1.190325
5,BikeFac_onstreet_tfm,1.829336
6,Park_acres_tfm,2.284820
3,Number.of.jobs_tfm_net,2.536351
2,Median.Age_tfm_net,3.922839


In [55]:
vif_data['feature']

1               log_stv_stl
2        Median.Age_tfm_net
3    Number.of.jobs_tfm_net
4             slope_tfm_net
5      BikeFac_onstreet_tfm
6            Park_acres_tfm
Name: feature, dtype: object

# Charlotte 1.0 mile buffer

In [60]:
ch1_data9=ch1_data.loc[:,ch1_data.columns.str.contains('_om', case=False)]
#be1_data10=be1_data.loc[:,be1_data.columns.str.contains('_om_net', case=False)]
ch1_data11=pd.concat([ch1_data9, ch1_data8.reindex(ch1_data9.index)], axis=1)
#be1_data12=pd.concat([be1_data8, be1_data11.reindex(be1_data8.index)], axis=1)
ch1_data11.head()

,Total.Commuter_om,Bike.Commuter_om,Percentage.of.Bike.Commuter_om,bike.commuter.density_om,Intersection_Density_om,Commercial.Area_om,Industrial.Area_om,Residential_Road_om,Retail.Area_om,Grass.Area_om,...,primary_bike_lane_binary,secondary_bike_lane_binary,tertiary_bike_lane_binary,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Distance_to_CBD_mi,Distance_to_Water_Body_mi
102,10471.922800,163.177552,1.558239,51.941455,131.781376,180766.9978,8.998661e+05,227266.3809,1.369224e+06,5.252148e+05,...,0,0,0,2.972870,1.940022,2.609264,7.120444,10.093315,1.556811,0.766116
103,9485.705957,44.407918,0.468156,14.135596,151.835064,0.0000,0.000000e+00,246902.0092,1.741495e+06,8.886883e+04,...,0,0,1,2.838037,1.723256,2.524083,6.608001,9.446037,1.371700,0.347870
104,5829.775885,17.843155,0.306069,5.679699,86.581001,0.0000,0.000000e+00,157075.7558,1.533610e+06,1.086468e+05,...,0,0,0,2.801283,1.372501,2.604211,5.796058,8.597340,3.008143,0.482522
105,13652.006780,180.329246,1.320899,57.401054,155.018189,180766.9978,1.078900e+06,136333.1498,6.268159e+04,1.225613e+06,...,0,0,0,2.536086,1.754546,2.060426,7.625595,10.161681,0.690422,0.370499
106,9092.930231,50.346334,0.553687,16.025868,172.207065,0.0000,0.000000e+00,184605.7376,5.730821e+05,4.788002e+05,...,0,0,0,1.234993,0.759397,0.833505,7.275172,8.510166,0.509815,0.247150


In [67]:
corrMatrix1 = ch1_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['Total.Commuter_om', 'Intersection_Density_om', 'Commercial.Area_om',
       'Retail.Area_om', 'Park.Area_om', 'Forest.Area_om',
       'Bicycle.Parking_om', 'Bus.Stops_om', 'School_om', 'college_om',
       ...
       'sep_onstreet_binary', 'BikeFac_binary', 'arterial_binary',
       'arterial_no_bike_lane_binary', 'secondary_no_bike_lane_binary',
       'log_stv_adb', 'log_stv_nc_adb', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi'],
      dtype='object', length=102)

In [68]:
chdata0=ch1_data[top_corr_features]
chdata00=chdata0.drop(['cycleway_lane_all_om', 'cycleway_track_all_om','cycleway_lane_all_om_net', 'cycleway_track_all_om_net','cycleway_lane_all_om_net', 'cycleway_track_all_om_net','cycleway_track_all_binary','Distance.to.Grass','arterial_no_bike_lane_binary',
       'secondary_no_bike_lane_binary'],axis=1)

In [69]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

chdata00
X_dataf = chdata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = chdata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\regression\linear_model.py:1738: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 2.72 s


,feature,VIF
2,min_dist_to_university,1.424184
1,min_dist_to_school,1.593945
7,Distance_to_Water_Body_mi,1.600344
6,log_stv_stl,1.747630
5,arterial_binary,1.753266
4,BikeFac_binary,2.454213
3,Point.Bridge,2.848838


In [70]:
vif_data['feature']

1           min_dist_to_school
2       min_dist_to_university
3                 Point.Bridge
4               BikeFac_binary
5              arterial_binary
6                  log_stv_stl
7    Distance_to_Water_Body_mi
Name: feature, dtype: object

# Charlotte1.5 mile buffer

In [71]:
ch1_data1=ch1_data.loc[:,~ch1_data.columns.str.contains('_otm', case=False)]
ch1_data2=ch1_data1.loc[:,~ch1_data1.columns.str.contains('_qm', case=False)]
ch1_data3=ch1_data2.loc[:,~ch1_data2.columns.str.contains('_hm', case=False)]
ch1_data4=ch1_data3.loc[:,~ch1_data3.columns.str.contains('_tfm', case=False)]
ch1_data5=ch1_data4.loc[:,~ch1_data4.columns.str.contains('_om', case=False)]
ch1_data6=ch1_data5.loc[:,~ch1_data5.columns.str.contains('_tm', case=False)]
ch1_data6.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,sep_bikeway_ohm_net,BikeFac_ohm_net,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
102,3.0,393.654456,18.547945,5.958904,1236,0,0,0,0,0,...,5089.053649,0.963836,2.972870,1.940022,2.609264,7.120444,10.093315,198.519985,1.556811,0.766116
103,3.0,160.742460,16.082192,4.602740,740,0,0,1,0,0,...,2583.938599,0.489382,2.838037,1.723256,2.524083,6.608001,9.446037,197.069588,1.371700,0.347870
104,3.0,39.364881,15.465753,2.945205,328,0,0,0,0,0,...,19805.975640,5.135609,2.801283,1.372501,2.604211,5.796058,8.597340,136.432429,3.008143,0.482522
105,1.0,149.292520,11.630137,4.780822,2049,0,1,0,0,0,...,6110.769132,1.816037,2.536086,1.754546,2.060426,7.625595,10.161681,156.062736,0.690422,0.370499
106,2.0,33.307341,2.438356,1.136986,1443,0,1,0,0,0,...,14287.398580,3.559167,1.234993,0.759397,0.833505,7.275172,8.510166,180.740409,0.509815,0.247150


In [72]:
corrMatrix1 = ch1_data6.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['travel_pattern', 'AADBT', 'stv_adb', 'secondary_binary',
       'cycleway_binary', 'cycleway_track_all_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center',
       ...
       'BikeFac_ohm', 'BikeFac_onstreet_ohm', 'sep_bikeway_ohm_net',
       'BikeFac_ohm_net', 'log_stv_adb', 'log_stv_nc_adb', 'log_stv_stl',
       'Park_acres_ohm', 'Distance_to_CBD_mi', 'Distance_to_Water_Body_mi'],
      dtype='object', length=101)

In [84]:
chdata0=ch1_data[top_corr_features]
chdata00=chdata0.drop(['travel_pattern','Distance.to.Grass', 'Distance.to.Grass.Center','cycleway_track_all_binary','secondary_no_bike_lane_binary','arterial_no_bike_lane_binary','arterial_no_bike_lane_binary','BikeFac_onstreet_ohm'],axis=1)

In [85]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

chdata00
X_dataf = chdata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = chdata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\regression\linear_model.py:1738: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 2.93 s


,feature,VIF
1,BikeFac_binary,1.186911
2,arterial_binary,1.333118
6,Park_acres_ohm,1.505109
7,Distance_to_Water_Body_mi,2.053013
5,log_stv_stl,2.291328
4,BikeFac_ohm_net,2.504065
3,BikeFac_ohm,2.817086


In [86]:
vif_data['feature']

1               BikeFac_binary
2              arterial_binary
3                  BikeFac_ohm
4              BikeFac_ohm_net
5                  log_stv_stl
6               Park_acres_ohm
7    Distance_to_Water_Body_mi
Name: feature, dtype: object

# Charlotte 2 mile buffer

In [87]:
ch1_data1=ch1_data.loc[:,~ch1_data.columns.str.contains('_otm', case=False)]
ch1_data2=ch1_data1.loc[:,~ch1_data1.columns.str.contains('_qm', case=False)]
ch1_data3=ch1_data2.loc[:,~ch1_data2.columns.str.contains('_hm', case=False)]
ch1_data4=ch1_data3.loc[:,~ch1_data3.columns.str.contains('_tfm', case=False)]
ch1_data5=ch1_data4.loc[:,~ch1_data4.columns.str.contains('_om', case=False)]
ch1_data6=ch1_data5.loc[:,~ch1_data5.columns.str.contains('_ohm', case=False)]
ch1_data6.head()

,travel_pattern,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,...,BikeFac_onstreet_tm,sep_bikeway_tm_net,BikeFac_tm_net,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Distance_to_CBD_mi,Distance_to_Water_Body_mi
102,3.0,393.654456,18.547945,5.958904,1236,0,0,0,0,0,...,54.107696,5089.053649,0.963836,2.972870,1.940022,2.609264,7.120444,10.093315,1.556811,0.766116
103,3.0,160.742460,16.082192,4.602740,740,0,0,1,0,0,...,88.457491,2583.938599,0.489382,2.838037,1.723256,2.524083,6.608001,9.446037,1.371700,0.347870
104,3.0,39.364881,15.465753,2.945205,328,0,0,0,0,0,...,84.831249,19805.975640,5.135609,2.801283,1.372501,2.604211,5.796058,8.597340,3.008143,0.482522
105,1.0,149.292520,11.630137,4.780822,2049,0,1,0,0,0,...,89.746434,6110.769132,1.816037,2.536086,1.754546,2.060426,7.625595,10.161681,0.690422,0.370499
106,2.0,33.307341,2.438356,1.136986,1443,0,1,0,0,0,...,96.590607,14287.398580,3.559167,1.234993,0.759397,0.833505,7.275172,8.510166,0.509815,0.247150


In [88]:
corrMatrix1 = ch1_data6.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['travel_pattern', 'AADBT', 'stv_adb', 'secondary_binary',
       'cycleway_binary', 'cycleway_track_all_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_university', 'Point.Bridge',
       'Total.Commuter_tm', 'Bike.Commuter_tm',
       'Percentage.of.Bike.Commuter_tm', 'bike.commuter.density_tm',
       'Intersection_Density_tm', 'Residential_Road_tm', 'Park.Area_tm',
       'Water.Area_tm', 'Forest.Area_tm', 'Bicycle.Parking_tm', 'School_tm',
       'college_tm', 'University_tm', 'OSM_node_Density_tm', 'Secondary_tm',
       'Tertiary_tm', 'Path_tm', 'Cycleway_tm', 'Footway_tm',
       'cycleway_lane_all_tm', 'cycleway_track_all_tm', 'maxspeed_tm',
       'pct_white_tm', 'pct_African_America

In [89]:
chdata0=ch1_data[top_corr_features]
chdata00=chdata0.drop(['travel_pattern','Distance.to.Grass', 'Distance.to.Grass.Center','cycleway_track_all_binary','secondary_no_bike_lane_binary','arterial_no_bike_lane_binary','arterial_no_bike_lane_binary','BikeFac_onstreet_tm'],axis=1)

In [90]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

chdata00
X_dataf = chdata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = chdata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\regression\linear_model.py:1738: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\mxm5116\AppData\Roaming\Python\Python38\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 2.61 s


,feature,VIF
1,BikeFac_binary,1.151286
6,Distance_to_Water_Body_mi,1.348050
2,arterial_binary,1.424350
4,sep_bikeway_tm_net,1.907099
5,log_stv_stl,1.907710
3,BikeFac_tm,3.086951


In [91]:
vif_data['feature']

1               BikeFac_binary
2              arterial_binary
3                   BikeFac_tm
4           sep_bikeway_tm_net
5                  log_stv_stl
6    Distance_to_Water_Body_mi
Name: feature, dtype: object